In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''

import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    punctuations = '''!()_;:,<>?@#$^.'''

    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''

In [3]:
Numbers15 = [
    ['Numbers','Num','15','001','1',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','15','002','2',"Speak unto the children of Israel, and say unto them, When ye be come into the land of your habitations, which I give unto you,"],
    ['Numbers','Num','15','003','3',"And will make an offering by fire unto the LORD, a burnt offering, or a sacrifice in performing a vow, or in a freewill offering, or in your solemn feasts, to make a sweet savour unto the LORD, of the herd, or of the flock:"],
    ['Numbers','Num','15','004','4',"Then shall he that offereth his offering unto the LORD bring a meat offering of a tenth deal of flour mingled with the fourth \emph{part} of an hin of oil."],
    ['Numbers','Num','15','005','5',"And the fourth \emph{part} of an hin of wine for a drink offering shalt thou prepare with the burnt offering or sacrifice, for one lamb."],
    ['Numbers','Num','15','006','6',"Or for a ram, thou shalt prepare \emph{for} a meat offering two tenth deals of flour mingled with the third \emph{part} of an hin of oil."],
    ['Numbers','Num','15','007','7',"And for a drink offering thou shalt offer the third \emph{part} of an hin of wine, \emph{for} a sweet savour unto the LORD."],
    ['Numbers','Num','15','008','8',"And when thou preparest a bullock \emph{for} a burnt offering, or \emph{for} a sacrifice in performing a vow, or peace offerings unto the LORD:"],
    ['Numbers','Num','15','009','9',"Then shall he bring with a bullock a meat offering of three tenth deals of flour mingled with half an hin of oil."],
    ['Numbers','Num','15','010','10',"And thou shalt bring for a drink offering half an hin of wine, \emph{for} an offering made by fire, of a sweet savour unto the LORD."],
    ['Numbers','Num','15','011','11',"Thus shall it be done for one bullock, or for one ram, or for a lamb, or a kid."],
    ['Numbers','Num','15','012','12',"According to the number that ye shall prepare, so shall ye do to every one according to their number."],
    ['Numbers','Num','15','013','13',"All that are born of the country shall do these things after this manner, in offering an offering made by fire, of a sweet savour unto the LORD."],
    ['Numbers','Num','15','014','14',"And if a stranger sojourn with you, or whosoever \emph{be} among you in your generations, and will offer an offering made by fire, of a sweet savour unto the LORD; as ye do, so he shall do."],
    ['Numbers','Num','15','015','15',"One ordinance \emph{shall} \emph{be} \emph{both} for you of the congregation, and also for the stranger that sojourneth \emph{with} \emph{you}, an ordinance for ever in your generations: as ye \emph{are}, so shall the stranger be before the LORD."],
    ['Numbers','Num','15','016','16',"One law and one manner shall be for you, and for the stranger that sojourneth with you."],
    ['Numbers','Num','15','017','17',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','15','018','18',"Speak unto the children of Israel, and say unto them, When ye come into the land whither I bring you,"],
    ['Numbers','Num','15','019','19',"Then it shall be, that, when ye eat of the bread of the land, ye shall offer up an heave offering unto the LORD."],
    ['Numbers','Num','15','020','20',"Ye shall offer up a cake of the first of your dough \emph{for} an heave offering: as \emph{ye} \emph{do} the heave offering of the threshingfloor, so shall ye heave it."],
    ['Numbers','Num','15','021','21',"Of the first of your dough ye shall give unto the LORD an heave offering in your generations."],
    ['Numbers','Num','15','022','22',"And if ye have erred, and not observed all these commandments, which the LORD hath spoken unto Moses,"],
    ['Numbers','Num','15','023','23',"\emph{Even} all that the LORD hath commanded you by the hand of Moses, from the day that the LORD commanded \emph{Moses}, and henceforward among your generations;"],
    ['Numbers','Num','15','024','24',"Then it shall be, if \emph{ought} be committed by ignorance without the knowledge of the congregation, that all the congregation shall offer one young bullock for a burnt offering, for a sweet savour unto the LORD, with his meat offering, and his drink offering, according to the manner, and one kid of the goats for a sin offering."],
    ['Numbers','Num','15','025','25',"And the priest shall make an atonement for all the congregation of the children of Israel, and it shall be forgiven them; for it \emph{is} ignorance: and they shall bring their offering, a sacrifice made by fire unto the LORD, and their sin offering before the LORD, for their ignorance:"],
    ['Numbers','Num','15','026','26',"And it shall be forgiven all the congregation of the children of Israel, and the stranger that sojourneth among them; seeing all the people \emph{were} in ignorance."],
    ['Numbers','Num','15','027','27',"And if any soul sin through ignorance, then he shall bring a she goat of the first year for a sin offering."],
    ['Numbers','Num','15','028','28',"And the priest shall make an atonement for the soul that sinneth ignorantly, when he sinneth by ignorance before the LORD, to make an atonement for him; and it shall be forgiven him."],
    ['Numbers','Num','15','029','29',"Ye shall have one law for him that sinneth through ignorance, \emph{both} \emph{for} him that is born among the children of Israel, and for the stranger that sojourneth among them."],
    ['Numbers','Num','15','030','30',"But the soul that doeth \emph{ought} presumptuously, \emph{whether} \emph{he} \emph{be} born in the land, or a stranger, the same reproacheth the LORD; and that soul shall be cut off from among his people."],
    ['Numbers','Num','15','031','31',"Because he hath despised the word of the LORD, and hath broken his commandment, that soul shall utterly be cut off; his iniquity \emph{shall} \emph{be} upon him."],
    ['Numbers','Num','15','032','32',"And while the children of Israel were in the wilderness, they found a man that gathered sticks upon the sabbath day."],
    ['Numbers','Num','15','033','33',"And they that found him gathering sticks brought him unto Moses and Aaron, and unto all the congregation."],
    ['Numbers','Num','15','034','34',"And they put him in ward, because it was not declared what should be done to him."],
    ['Numbers','Num','15','035','35',"And the LORD said unto Moses, The man shall be surely put to death: all the congregation shall stone him with stones without the camp."],
    ['Numbers','Num','15','036','36',"And all the congregation brought him without the camp, and stoned him with stones, and he died; as the LORD commanded Moses."],
    ['Numbers','Num','15','037','37',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','15','038','38',"Speak unto the children of Israel, and bid them that they make them fringes in the borders of their garments throughout their generations, and that they put upon the fringe of the borders a ribband of blue:"],
    ['Numbers','Num','15','039','39',"And it shall be unto you for a fringe, that ye may look upon it, and remember all the commandments of the LORD, and do them; and that ye seek not after your own heart and your own eyes, after which ye use to go a whoring:"],
    ['Numbers','Num','15','040','40',"That ye may remember, and do all my commandments, and be holy unto your God."],
    ['Numbers','Num','15','041','41',"I \emph{am} the LORD your God, which brought you out of the land of Egypt, to be your God: I \emph{am} the LORD your God."]
]

In [4]:
# %load OT04-NumbersChapters.py
Numbers01 = [
    ['Numbers','Numbers','01','01','1',"And the LORD spake unto Moses in the wilderness of Sinai, in the tabernacle of the congregation, on the first \emph{day} of the second month, in the second year after they were come out of the land of Egypt, saying,"],
    ['Numbers','Numbers','01','02','2',"Take ye the sum of all the congregation of the children of Israel, after their families, by the house of their fathers, with the number of \emph{their} names, every male by their polls; "],
    ['Numbers','Numbers','01','03','3',"From twenty years old and upward, all that are able to go forth to war in Israel: thou and Aaron shall number them by their armies."],
    ['Numbers','Numbers','01','04','4',"And with you there shall be a man of every tribe; every one head of the house of his fathers."],
    ['Numbers','Numbers','01','05','5',"And these \emph{are} the names of the men that shall stand with you: of \emph{the} \emph{tribe} \emph{of} Reuben; Elizur the son of Shedeur."],
    ['Numbers','Numbers','01','06','6',"Of Simeon; Shelumiel the son of Zurishaddai. "],
    ['Numbers','Numbers','01','07','7',"Of Judah; Nahshon the son of Amminadab. "],
    ['Numbers','Numbers','01','08','8',"Of Issachar; Nethaneel the son of Zuar. "],
    ['Numbers','Numbers','01','09','9',"Of Zebulun; Eliab the son of Helon."],
    ['Numbers','Numbers','01','10','10',"Of the children of Joseph: of Ephraim; Elishama the son of Ammihud: of Manasseh; Gamaliel the son of Pedahzur."],
    ['Numbers','Numbers','01','11','11',"Of Benjamin; Abidan the son of Gideoni. "],
    ['Numbers','Numbers','01','12','12',"Of Dan; Ahiezer the son of Ammishaddai. "],
    ['Numbers','Numbers','01','13','13',"Of Asher; Pagiel the son of Ocran. "],
    ['Numbers','Numbers','01','14','14',"Of Gad; Eliasaph the son of Deuel."],
    ['Numbers','Numbers','01','15','15',"Of Naphtali; Ahira the son of Enan. "],
    ['Numbers','Numbers','01','16','16',"These \emph{were} the renowned of the congregation, princes of the tribes of their fathers, heads of thousands in Israel."],
    ['Numbers','Numbers','01','17','17',"And Moses and Aaron took these men which are expressed by \emph{their} names:"],
    ['Numbers','Numbers','01','18','18',"And they assembled all the congregation together on the first \emph{day} of the second month, and they declared their pedigrees after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, by their polls."],
    ['Numbers','Numbers','01','19','19',"As the LORD commanded Moses, so he numbered them in the wilderness of Sinai. "],
    ['Numbers','Numbers','01','20','20',"And the children of Reuben, Israel's eldest son, by their generations, after their families, by the house of their fathers, according to the number of the names, by their polls, every male from twenty years old and upward, all that were able to go forth to war; "],
    ['Numbers','Numbers','01','21','21',"Those that were numbered of them, \emph{even} of the tribe of Reuben, \emph{were} forty and six thousand and five hundred."],
    ['Numbers','Numbers','01','22','22',"Of the children of Simeon, by their generations, after their families, by the house of their fathers, those that were numbered of them, according to the number of the names, by their polls, every male from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','23','23',"Those that were numbered of them, \emph{even} of the tribe of Simeon, \emph{were} fifty and nine thousand and three hundred. "],
    ['Numbers','Numbers','01','24','24',"Of the children of Gad, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','25','25',"Those that were numbered of them, \emph{even} of the tribe of Gad, \emph{were} forty and five thousand six hundred and fifty. "],
    ['Numbers','Numbers','01','26','26',"Of the children of Judah, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','27','27',"Those that were numbered of them, \emph{even} of the tribe of Judah, \emph{were} threescore and fourteen thousand and six hundred."],
    ['Numbers','Numbers','01','28','28',"Of the children of Issachar, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','29','29',"Those that were numbered of them, \emph{even} of the tribe of Issachar, \emph{were} fifty and four thousand and four hundred."],
    ['Numbers','Numbers','01','30','30',"Of the children of Zebulun, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','31','31',"Those that were numbered of them, \emph{even} of the tribe of Zebulun, \emph{were} fifty and seven thousand and four hundred."],
    ['Numbers','Numbers','01','32','32',"Of the children of Joseph, \emph{namely}, of the children of Ephraim, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','09','33','33',"Those that were numbered of them, \emph{even} of the tribe of Ephraim, \emph{were} forty thousand and five hundred."],
    ['Numbers','Numbers','01','34','34',"Of the children of Manasseh, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','35','35',"Those that were numbered of them, \emph{even} of the tribe of Manasseh, \emph{were} thirty and two thousand and two hundred."],
    ['Numbers','Numbers','01','36','36',"Of the children of Benjamin, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','37','37',"Those that were numbered of them, \emph{even} of the tribe of Benjamin, \emph{were} thirty and five thousand and four hundred. "],
    ['Numbers','Numbers','01','38','38',"Of the children of Dan, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','39','39',"Those that were numbered of them, \emph{even} of the tribe of Dan, \emph{were} threescore and two thousand and seven hundred."],
    ['Numbers','Numbers','01','40','40',"Of the children of Asher, by their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','41','41',"Those that were numbered of them, \emph{even} of the tribe of Asher, \emph{were} forty and one thousand and five hundred."],
    ['Numbers','Numbers','01','42','42',"Of the children of Naphtali, throughout their generations, after their families, by the house of their fathers, according to the number of the names, from twenty years old and upward, all that were able to go forth to war;"],
    ['Numbers','Numbers','01','43','43',"Those that were numbered of them, \emph{even} of the tribe of Naphtali, \emph{were} fifty and three thousand and four hundred."],
    ['Numbers','Numbers','01','44','44',"These \emph{are} those that were numbered, which Moses and Aaron numbered, and the princes of Israel, \emph{being} twelve men: each one was for the house of his fathers. "],
    ['Numbers','Numbers','01','45','45',"So were all those that were numbered of the children of Israel, by the house of their fathers, from twenty years old and upward, all that were able to go forth to war in Israel;"],
    ['Numbers','Numbers','01','46','46',"Even all they that were numbered were six hundred thousand and three thousand and five hundred and fifty."],
    ['Numbers','Numbers','01','47','47',"But the Levites after the tribe of their fathers were not numbered among them. "],
    ['Numbers','Numbers','01','48','48',"For the LORD had spoken unto Moses, saying,"],
    ['Numbers','Numbers','01','49','49',"Only thou shalt not number the tribe of Levi, neither take the sum of them among the children of Israel:"],
    ['Numbers','Numbers','01','50','50',"But thou shalt appoint the Levites over the tabernacle of testimony, and over all the vessels thereof, and over all things that \emph{belong} to it: they shall bear the tabernacle, and all the vessels thereof; and they shall minister unto it, and shall encamp round about the tabernacle."],
    ['Numbers','Numbers','01','51','51',"And when the tabernacle setteth forward, the Levites shall take it down: and when the tabernacle is to be pitched, the Levites shall set it up: and the stranger that cometh nigh shall be put to death."],
    ['Numbers','Numbers','01','52','52',"And the children of Israel shall pitch their tents, every man by his own camp, and every man by his own standard, throughout their hosts. "],
    ['Numbers','Numbers','01','53','53',"But the Levites shall pitch round about the tabernacle of testimony, that there be no wrath upon the congregation of the children of Israel: and the Levites shall keep the charge of the tabernacle of testimony."],
    ['Numbers','Numbers','01','54','54',"And the children of Israel did according to all that the LORD commanded Moses, so did they."]]

Numbers02 = [
    ['Numbers','Num','02','001','1',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Numbers','Num','02','002','2',"Every man of the children of Israel shall pitch by his own standard, with the ensign of their father's house: far off about the tabernacle of the congregation shall they pitch."],
['Numbers','Num','02','003','3',"And on the east side toward the rising of the sun shall they of the standard of the camp of Judah pitch throughout their armies: and Nahshon the son of Amminadab \emph{shall} \emph{be} captain of the children of Judah."],
['Numbers','Num','02','004','4',"And his host, and those that were numbered of them, \emph{were} threescore and fourteen thousand and six hundred."],
['Numbers','Num','02','005','5',"And those that do pitch next unto him \emph{shall} \emph{be} the tribe of Issachar: and Nethaneel the son of Zuar \emph{shall} \emph{be} captain of the children of Issachar."],
['Numbers','Num','02','006','6',"And his host, and those that were numbered thereof, \emph{were} fifty and four thousand and four hundred."],
['Numbers','Num','02','007','7',"\emph{Then} the tribe of Zebulun: and Eliab the son of Helon \emph{shall} \emph{be} captain of the children of Zebulun."],
['Numbers','Num','02','008','8',"And his host, and those that were numbered thereof, \emph{were} fifty and seven thousand and four hundred."],
['Numbers','Num','02','009','9',"All that were numbered in the camp of Judah \emph{were} an hundred thousand and fourscore thousand and six thousand and four hundred, throughout their armies. These shall first set forth."],
['Numbers','Num','02','010','10',"On the south side \emph{shall} \emph{be} the standard of the camp of Reuben according to their armies: and the captain of the children of Reuben \emph{shall} \emph{be} Elizur the son of Shedeur."],
['Numbers','Num','02','011','11',"And his host, and those that were numbered thereof, \emph{were} forty and six thousand and five hundred."],
['Numbers','Num','02','012','12',"And those which pitch by him \emph{shall} \emph{be} the tribe of Simeon: and the captain of the children of Simeon \emph{shall} \emph{be} Shelumiel the son of Zurishaddai."],
['Numbers','Num','02','013','13',"And his host, and those that were numbered of them, \emph{were} fifty and nine thousand and three hundred."],
['Numbers','Num','02','014','14',"Then the tribe of Gad: and the captain of the sons of Gad \emph{shall} \emph{be} Eliasaph the son of Reuel."],
['Numbers','Num','02','015','15',"And his host, and those that were numbered of them, \emph{were} forty and five thousand and six hundred and fifty."],
['Numbers','Num','02','016','16',"All that were numbered in the camp of Reuben \emph{were} an hundred thousand and fifty and one thousand and four hundred and fifty, throughout their armies. And they shall set forth in the second rank."],
['Numbers','Num','02','017','17',"Then the tabernacle of the congregation shall set forward with the camp of the Levites in the midst of the camp: as they encamp, so shall they set forward, every man in his place by their standards."],
['Numbers','Num','02','018','18',"On the west side \emph{shall} \emph{be} the standard of the camp of Ephraim according to their armies: and the captain of the sons of Ephraim \emph{shall} \emph{be} Elishama the son of Ammihud."],
['Numbers','Num','02','019','19',"And his host, and those that were numbered of them, \emph{were} forty thousand and five hundred."],
['Numbers','Num','02','020','20',"And by him \emph{shall} \emph{be} the tribe of Manasseh: and the captain of the children of Manasseh \emph{shall} \emph{be} Gamaliel the son of Pedahzur."],
['Numbers','Num','02','021','21',"And his host, and those that were numbered of them, \emph{were} thirty and two thousand and two hundred."],
['Numbers','Num','02','022','22',"Then the tribe of Benjamin: and the captain of the sons of Benjamin \emph{shall} \emph{be} Abidan the son of Gideoni."],
['Numbers','Num','02','023','23',"And his host, and those that were numbered of them, \emph{were} thirty and five thousand and four hundred."],
['Numbers','Num','02','024','24',"All that were numbered of the camp of Ephraim \emph{were} an hundred thousand and eight thousand and an hundred, throughout their armies. And they shall go forward in the third rank."],
['Numbers','Num','02','025','25',"The standard of the camp of Dan \emph{shall} \emph{be} on the north side by their armies: and the captain of the children of Dan \emph{shall} \emph{be} Ahiezer the son of Ammishaddai."],
['Numbers','Num','02','026','26',"And his host, and those that were numbered of them, \emph{were} threescore and two thousand and seven hundred."],
['Numbers','Num','02','027','27',"And those that encamp by him \emph{shall} \emph{be} the tribe of Asher: and the captain of the children of Asher \emph{shall} \emph{be} Pagiel the son of Ocran."],
['Numbers','Num','02','028','28',"And his host, and those that were numbered of them, \emph{were} forty and one thousand and five hundred."],
['Numbers','Num','02','029','29',"Then the tribe of Naphtali: and the captain of the children of Naphtali \emph{shall} \emph{be} Ahira the son of Enan."],
['Numbers','Num','02','030','30',"And his host, and those that were numbered of them, \emph{were} fifty and three thousand and four hundred."],
['Numbers','Num','02','031','31',"All they that were numbered in the camp of Dan \emph{were} an hundred thousand and fifty and seven thousand and six hundred. They shall go hindmost with their standards."],
['Numbers','Num','02','032','32',"These \emph{are} those which were numbered of the children of Israel by the house of their fathers: all those that were numbered of the camps throughout their hosts \emph{were} six hundred thousand and three thousand and five hundred and fifty."],
['Numbers','Num','02','033','33',"But the Levites were not numbered among the children of Israel; as the LORD commanded Moses."],
['Numbers','Num','02','034','34',"And the children of Israel did according to all that the LORD commanded Moses: so they pitched by their standards, and so they set forward, every one after their families, according to the house of their fathers."]
]

Numbers03 = [
    ['Numbers','Num','03','001','1',"These also \emph{are} the generations of Aaron and Moses in the day \emph{that} the LORD spake with Moses in mount Sinai."],
['Numbers','Num','03','002','2',"And these \emph{are} the names of the sons of Aaron; Nadab the firstborn, and Abihu, Eleazar, and Ithamar."],
['Numbers','Num','03','003','3',"These \emph{are} the names of the sons of Aaron, the priests which were anointed, whom he consecrated to minister in the priest's office."],
['Numbers','Num','03','004','4',"And Nadab and Abihu died before the LORD, when they offered strange fire before the LORD, in the wilderness of Sinai, and they had no children: and Eleazar and Ithamar ministered in the priest's office in the sight of Aaron their father."],
['Numbers','Num','03','005','5',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','03','006','6',"Bring the tribe of Levi near, and present them before Aaron the priest, that they may minister unto him."],
['Numbers','Num','03','007','7',"And they shall keep his charge, and the charge of the whole congregation before the tabernacle of the congregation, to do the service of the tabernacle."],
['Numbers','Num','03','008','8',"And they shall keep all the instruments of the tabernacle of the congregation, and the charge of the children of Israel, to do the service of the tabernacle."],
['Numbers','Num','03','009','9',"And thou shalt give the Levites unto Aaron and to his sons: they \emph{are} wholly given unto him out of the children of Israel."],
['Numbers','Num','03','010','10',"And thou shalt appoint Aaron and his sons, and they shall wait on their priest's office: and the stranger that cometh nigh shall be put to death."],
['Numbers','Num','03','011','11',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','03','012','12',"And I, behold, I have taken the Levites from among the children of Israel instead of all the firstborn that openeth the matrix among the children of Israel: therefore the Levites shall be mine;"],
['Numbers','Num','03','013','13',"Because all the firstborn \emph{are} mine; \emph{for} on the day that I smote all the firstborn in the land of Egypt I hallowed unto me all the firstborn in Israel, both man and beast: mine shall they be: I \emph{am} the LORD."],
['Numbers','Num','03','014','14',"And the LORD spake unto Moses in the wilderness of Sinai, saying,"],
['Numbers','Num','03','015','15',"Number the children of Levi after the house of their fathers, by their families: every male from a month old and upward shalt thou number them."],
['Numbers','Num','03','016','16',"And Moses numbered them according to the word of the LORD, as he was commanded."],
['Numbers','Num','03','017','17',"And these were the sons of Levi by their names; Gershon, and Kohath, and Merari."],
['Numbers','Num','03','018','18',"And these \emph{are} the names of the sons of Gershon by their families; Libni, and Shimei."],
['Numbers','Num','03','019','19',"And the sons of Kohath by their families; Amram, and Izehar, Hebron, and Uzziel."],
['Numbers','Num','03','020','20',"And the sons of Merari by their families; Mahli, and Mushi. These \emph{are} the families of the Levites according to the house of their fathers."],
['Numbers','Num','03','021','21',"Of Gershon \emph{was} the family of the Libnites, and the family of the Shimites: these \emph{are} the families of the Gershonites."],
['Numbers','Num','03','022','22',"Those that were numbered of them, according to the number of all the males, from a month old and upward, \emph{even} those that were numbered of them \emph{were} seven thousand and five hundred."],
['Numbers','Num','03','023','23',"The families of the Gershonites shall pitch behind the tabernacle westward."],
['Numbers','Num','03','024','24',"And the chief of the house of the father of the Gershonites \emph{shall} \emph{be} Eliasaph the son of Lael."],
['Numbers','Num','03','025','25',"And the charge of the sons of Gershon in the tabernacle of the congregation \emph{shall} \emph{be} the tabernacle, and the tent, the covering thereof, and the hanging for the door of the tabernacle of the congregation,"],
['Numbers','Num','03','026','26',"And the hangings of the court, and the curtain for the door of the court, which \emph{is} by the tabernacle, and by the altar round about, and the cords of it for all the service thereof."],
['Numbers','Num','03','027','27',"And of Kohath \emph{was} the family of the Amramites, and the family of the Izeharites, and the family of the Hebronites, and the family of the Uzzielites: these \emph{are} the families of the Kohathites."],
['Numbers','Num','03','028','28',"In the number of all the males, from a month old and upward, \emph{were} eight thousand and six hundred, keeping the charge of the sanctuary."],
['Numbers','Num','03','029','29',"The families of the sons of Kohath shall pitch on the side of the tabernacle southward."],
['Numbers','Num','03','030','30',"And the chief of the house of the father of the families of the Kohathites \emph{shall} \emph{be} Elizaphan the son of Uzziel."],
['Numbers','Num','03','031','31',"And their charge \emph{shall} \emph{be} the ark, and the table, and the candlestick, and the altars, and the vessels of the sanctuary wherewith they minister, and the hanging, and all the service thereof."],
['Numbers','Num','03','032','32',"And Eleazar the son of Aaron the priest \emph{shall} \emph{be} chief over the chief of the Levites, \emph{and} \emph{have} the oversight of them that keep the charge of the sanctuary."],
['Numbers','Num','03','033','33',"Of Merari \emph{was} the family of the Mahlites, and the family of the Mushites: these \emph{are} the families of Merari."],
['Numbers','Num','03','034','34',"And those that were numbered of them, according to the number of all the males, from a month old and upward, \emph{were} six thousand and two hundred."],
['Numbers','Num','03','035','35',"And the chief of the house of the father of the families of Merari \emph{was} Zuriel the son of Abihail: \emph{these} shall pitch on the side of the tabernacle northward."],
['Numbers','Num','03','036','36',"And \emph{under} the custody and charge of the sons of Merari \emph{shall} \emph{be} the boards of the tabernacle, and the bars thereof, and the pillars thereof, and the sockets thereof, and all the vessels thereof, and all that serveth thereto,"],
['Numbers','Num','03','037','37',"And the pillars of the court round about, and their sockets, and their pins, and their cords."],
['Numbers','Num','03','038','38',"But those that encamp before the tabernacle toward the east, \emph{even} before the tabernacle of the congregation eastward, \emph{shall} \emph{be} Moses, and Aaron and his sons, keeping the charge of the sanctuary for the charge of the children of Israel; and the stranger that cometh nigh shall be put to death."],
['Numbers','Num','03','039','39',"All that were numbered of the Levites, which Moses and Aaron numbered at the commandment of the LORD, throughout their families, all the males from a month old and upward, \emph{were} twenty and two thousand."],
['Numbers','Num','03','040','40',"And the LORD said unto Moses, Number all the firstborn of the males of the children of Israel from a month old and upward, and take the number of their names."],
['Numbers','Num','03','041','41',"And thou shalt take the Levites for me (I \emph{am} the LORD) instead of all the firstborn among the children of Israel; and the cattle of the Levites instead of all the firstlings among the cattle of the children of Israel."],
['Numbers','Num','03','042','42',"And Moses numbered, as the LORD commanded him, all the firstborn among the children of Israel."],
['Numbers','Num','03','043','43',"And all the firstborn males by the number of names, from a month old and upward, of those that were numbered of them, were twenty and two thousand two hundred and threescore and thirteen."],
['Numbers','Num','03','044','44',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','03','045','45',"Take the Levites instead of all the firstborn among the children of Israel, and the cattle of the Levites instead of their cattle; and the Levites shall be mine: I \emph{am} the LORD."],
['Numbers','Num','03','046','46',"And for those that are to be redeemed of the two hundred and threescore and thirteen of the firstborn of the children of Israel, which are more than the Levites;"],
['Numbers','Num','03','047','47',"Thou shalt even take five shekels apiece by the poll, after the shekel of the sanctuary shalt thou take \emph{them}: (the shekel \emph{is} twenty gerahs:)"],
['Numbers','Num','03','048','48',"And thou shalt give the money, wherewith the odd number of them is to be redeemed, unto Aaron and to his sons."],
['Numbers','Num','03','049','49',"And Moses took the redemption money of them that were over and above them that were redeemed by the Levites:"],
['Numbers','Num','03','050','50',"Of the firstborn of the children of Israel took he the money; a thousand three hundred and threescore and five \emph{shekels}, after the shekel of the sanctuary:"],
['Numbers','Num','03','051','51',"And Moses gave the money of them that were redeemed unto Aaron and to his sons, according to the word of the LORD, as the LORD commanded Moses."]
]

Numbers04 = [
    ['Numbers','Num','04','001','1',"And the LORD spake unto Moses and unto Aaron, saying,"],
    ['Numbers','Num','04','002','2',"Take the sum of the sons of Kohath from among the sons of Levi, after their families, by the house of their fathers,"],
    ['Numbers','Num','04','003','3',"From thirty years old and upward even until fifty years old, all that enter into the host, to do the work in the tabernacle of the congregation."],
    ['Numbers','Num','04','004','4',"This \emph{shall} \emph{be} the service of the sons of Kohath in the tabernacle of the congregation, \emph{about} the most holy things:"],
    ['Numbers','Num','04','005','5',"And when the camp setteth forward, Aaron shall come, and his sons, and they shall take down the covering vail, and cover the ark of testimony with it:"],
    ['Numbers','Num','04','006','6',"And shall put thereon the covering of badgers' skins, and shall spread over \emph{it} a cloth wholly of blue, and shall put in the staves thereof."],
    ['Numbers','Num','04','007','7',"And upon the table of shewbread they shall spread a cloth of blue, and put thereon the dishes, and the spoons, and the bowls, and covers to cover withal: and the continual bread shall be thereon:"],
    ['Numbers','Num','04','008','8',"And they shall spread upon them a cloth of scarlet, and cover the same with a covering of badgers' skins, and shall put in the staves thereof."],
    ['Numbers','Num','04','009','9',"And they shall take a cloth of blue, and cover the candlestick of the light, and his lamps, and his tongs, and his snuffdishes, and all the oil vessels thereof, wherewith they minister unto it:"],
    ['Numbers','Num','04','010','10',"And they shall put it and all the vessels thereof within a covering of badgers' skins, and shall put \emph{it} upon a bar."],
['Numbers','Num','04','011','11',"And upon the golden altar they shall spread a cloth of blue, and cover it with a covering of badgers' skins, and shall put to the staves thereof:"],
['Numbers','Num','04','012','12',"And they shall take all the instruments of ministry, wherewith they minister in the sanctuary, and put \emph{them} in a cloth of blue, and cover them with a covering of badgers' skins, and shall put \emph{them} on a bar:"],
['Numbers','Num','04','013','13',"And they shall take away the ashes from the altar, and spread a purple cloth thereon:"],
['Numbers','Num','04','014','14',"And they shall put upon it all the vessels thereof, wherewith they minister about it, \emph{even} the censers, the fleshhooks, and the shovels, and the basons, all the vessels of the altar; and they shall spread upon it a covering of badgers' skins, and put to the staves of it."],
['Numbers','Num','04','015','15',"And when Aaron and his sons have made an end of covering the sanctuary, and all the vessels of the sanctuary, as the camp is to set forward; after that, the sons of Kohath shall come to bear \emph{it}: but they shall not touch \emph{any} holy thing, lest they die. These \emph{things} \emph{are} the burden of the sons of Kohath in the tabernacle of the congregation."],
['Numbers','Num','04','016','16',"And to the office of Eleazar the son of Aaron the priest \emph{pertaineth} the oil for the light, and the sweet incense, and the daily meat offering, and the anointing oil, \emph{and} the oversight of all the tabernacle, and of all that therein \emph{is}, in the sanctuary, and in the vessels thereof."],
['Numbers','Num','04','017','17',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Numbers','Num','04','018','18',"Cut ye not off the tribe of the families of the Kohathites from among the Levites:"],
['Numbers','Num','04','019','19',"But thus do unto them, that they may live, and not die, when they approach unto the most holy things: Aaron and his sons shall go in, and appoint them every one to his service and to his burden:"],
['Numbers','Num','04','020','20',"But they shall not go in to see when the holy things are covered, lest they die."],
['Numbers','Num','04','021','21',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','04','022','22',"Take also the sum of the sons of Gershon, throughout the houses of their fathers, by their families;"],
['Numbers','Num','04','023','23',"From thirty years old and upward until fifty years old shalt thou number them; all that enter in to perform the service, to do the work in the tabernacle of the congregation."],
['Numbers','Num','04','024','24',"This \emph{is} the service of the families of the Gershonites, to serve, and for burdens:"],
['Numbers','Num','04','025','25',"And they shall bear the curtains of the tabernacle, and the tabernacle of the congregation, his covering, and the covering of the badgers' skins that \emph{is} above upon it, and the hanging for the door of the tabernacle of the congregation,"],
['Numbers','Num','04','026','26',"And the hangings of the court, and the hanging for the door of the gate of the court, which \emph{is} by the tabernacle and by the altar round about, and their cords, and all the instruments of their service, and all that is made for them: so shall they serve."],
['Numbers','Num','04','027','27',"At the appointment of Aaron and his sons shall be all the service of the sons of the Gershonites, in all their burdens, and in all their service: and ye shall appoint unto them in charge all their burdens."],
['Numbers','Num','04','028','28',"This \emph{is} the service of the families of the sons of Gershon in the tabernacle of the congregation: and their charge \emph{shall} \emph{be} under the hand of Ithamar the son of Aaron the priest."],
['Numbers','Num','04','029','29',"As for the sons of Merari, thou shalt number them after their families, by the house of their fathers;"],
['Numbers','Num','04','030','30',"From thirty years old and upward even unto fifty years old shalt thou number them, every one that entereth into the service, to do the work of the tabernacle of the congregation."],
['Numbers','Num','04','031','31',"And this \emph{is} the charge of their burden, according to all their service in the tabernacle of the congregation; the boards of the tabernacle, and the bars thereof, and the pillars thereof, and sockets thereof,"],
['Numbers','Num','04','032','32',"And the pillars of the court round about, and their sockets, and their pins, and their cords, with all their instruments, and with all their service: and by name ye shall reckon the instruments of the charge of their burden."],
['Numbers','Num','04','033','33',"This \emph{is} the service of the families of the sons of Merari, according to all their service, in the tabernacle of the congregation, under the hand of Ithamar the son of Aaron the priest."],
['Numbers','Num','04','034','34',"And Moses and Aaron and the chief of the congregation numbered the sons of the Kohathites after their families, and after the house of their fathers,"],
['Numbers','Num','04','035','35',"From thirty years old and upward even unto fifty years old, every one that entereth into the service, for the work in the tabernacle of the congregation:"],
['Numbers','Num','04','036','36',"And those that were numbered of them by their families were two thousand seven hundred and fifty."],
['Numbers','Num','04','037','37',"These \emph{were} they that were numbered of the families of the Kohathites, all that might do service in the tabernacle of the congregation, which Moses and Aaron did number according to the commandment of the LORD by the hand of Moses."],
['Numbers','Num','04','038','38',"And those that were numbered of the sons of Gershon, throughout their families, and by the house of their fathers,"],
['Numbers','Num','04','039','39',"From thirty years old and upward even unto fifty years old, every one that entereth into the service, for the work in the tabernacle of the congregation,"],
    ['Numbers','Num','04','040','40',"Even those that were numbered of them, throughout their families, by the house of their fathers, were two thousand and six hundred and thirty."],
    ['Numbers','Num','04','041','41',"These \emph{are} they that were numbered of the families of the sons of Gershon, of all that might do service in the tabernacle of the congregation, whom Moses and Aaron did number according to the commandment of the LORD."],
    ['Numbers','Num','04','042','42',"And those that were numbered of the families of the sons of Merari, throughout their families, by the house of their fathers,"],
    ['Numbers','Num','04','043','43',"From thirty years old and upward even unto fifty years old, every one that entereth into the service, for the work in the tabernacle of the congregation,"],
    ['Numbers','Num','04','044','44',"Even those that were numbered of them after their families, were three thousand and two hundred."],
    ['Numbers','Num','04','045','45',"These \emph{be} those that were numbered of the families of the sons of Merari, whom Moses and Aaron numbered according to the word of the LORD by the hand of Moses."],
    ['Numbers','Num','04','046','46',"All those that were numbered of the Levites, whom Moses and Aaron and the chief of Israel numbered, after their families, and after the house of their fathers,"],
    ['Numbers','Num','04','047','47',"From thirty years old and upward even unto fifty years old, every one that came to do the service of the ministry, and the service of the burden in the tabernacle of the congregation,"],
    ['Numbers','Num','04','048','48',"Even those that were numbered of them, were eight thousand and five hundred and fourscore."],
    ['Numbers','Num','04','049','49',"According to the commandment of the LORD they were numbered by the hand of Moses, every one according to his service, and according to his burden: thus were they numbered of him, as the LORD commanded Moses."]
]

Numbers05 = [
    ['Numbers','Num','05','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','05','002','2',"Command the children of Israel, that they put out of the camp every leper, and every one that hath an issue, and whosoever is defiled by the dead:"],
['Numbers','Num','05','003','3',"Both male and female shall ye put out, without the camp shall ye put them; that they defile not their camps, in the midst whereof I dwell."],
['Numbers','Num','05','004','4',"And the children of Israel did so, and put them out without the camp: as the LORD spake unto Moses, so did the children of Israel."],
['Numbers','Num','05','005','5',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','05','006','6',"Speak unto the children of Israel, When a man or woman shall commit any sin that men commit, to do a trespass against the LORD, and that person be guilty;"],
['Numbers','Num','05','007','7',"Then they shall confess their sin which they have done: and he shall recompense his trespass with the principal thereof, and add unto it the fifth \emph{part} thereof, and give \emph{it} unto \emph{him} against whom he hath trespassed."],
['Numbers','Num','05','008','8',"But if the man have no kinsman to recompense the trespass unto, let the trespass be recompensed unto the LORD, \emph{even} to the priest; beside the ram of the atonement, whereby an atonement shall be made for him."],
['Numbers','Num','05','009','9',"And every offering of all the holy things of the children of Israel, which they bring unto the priest, shall be his."],
['Numbers','Num','05','010','10',"And every man's hallowed things shall be his: whatsoever any man giveth the priest, it shall be his."],
['Numbers','Num','05','011','11',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','05','012','12',"Speak unto the children of Israel, and say unto them, If any man's wife go aside, and commit a trespass against him,"],
['Numbers','Num','05','013','13',"And a man lie with her carnally, and it be hid from the eyes of her husband, and be kept close, and she be defiled, and \emph{there} \emph{be} no witness against her, neither she be taken \emph{with} \emph{the} \emph{manner};"],
['Numbers','Num','05','014','14',"And the spirit of jealousy come upon him, and he be jealous of his wife, and she be defiled: or if the spirit of jealousy come upon him, and he be jealous of his wife, and she be not defiled:"],
['Numbers','Num','05','015','15',"Then shall the man bring his wife unto the priest, and he shall bring her offering for her, the tenth \emph{part} of an ephah of barley meal; he shall pour no oil upon it, nor put frankincense thereon; for it \emph{is} an offering of jealousy, an offering of memorial, bringing iniquity to remembrance."],
['Numbers','Num','05','016','16',"And the priest shall bring her near, and set her before the LORD:"],
['Numbers','Num','05','017','17',"And the priest shall take holy water in an earthen vessel; and of the dust that is in the floor of the tabernacle the priest shall take, and put \emph{it} into the water:"],
['Numbers','Num','05','018','18',"And the priest shall set the woman before the LORD, and uncover the woman's head, and put the offering of memorial in her hands, which \emph{is} the jealousy offering: and the priest shall have in his hand the bitter water that causeth the curse:"],
['Numbers','Num','05','019','19',"And the priest shall charge her by an oath, and say unto the woman, If no man have lain with thee, and if thou hast not gone aside to uncleanness \emph{with} \emph{another} instead of thy husband, be thou free from this bitter water that causeth the curse:"],
['Numbers','Num','05','020','20',"But if thou hast gone aside \emph{to} \emph{another} instead of thy husband, and if thou be defiled, and some man have lain with thee beside thine husband:"],
['Numbers','Num','05','021','21',"Then the priest shall charge the woman with an oath of cursing, and the priest shall say unto the woman, The LORD make thee a curse and an oath among thy people, when the LORD doth make thy thigh to rot, and thy belly to swell;"],
['Numbers','Num','05','022','22',"And this water that causeth the curse shall go into thy bowels, to make \emph{thy} belly to swell, and \emph{thy} thigh to rot: And the woman shall say, Amen, amen."],
['Numbers','Num','05','023','23',"And the priest shall write these curses in a book, and he shall blot \emph{them} out with the bitter water:"],
['Numbers','Num','05','024','24',"And he shall cause the woman to drink the bitter water that causeth the curse: and the water that causeth the curse shall enter into her, \emph{and} \emph{become} bitter."],
['Numbers','Num','05','025','25',"Then the priest shall take the jealousy offering out of the woman's hand, and shall wave the offering before the LORD, and offer it upon the altar:"],
['Numbers','Num','05','026','26',"And the priest shall take an handful of the offering, \emph{even} the memorial thereof, and burn \emph{it} upon the altar, and afterward shall cause the woman to drink the water."],
['Numbers','Num','05','027','27',"And when he hath made her to drink the water, then it shall come to pass, \emph{that}, if she be defiled, and have done trespass against her husband, that the water that causeth the curse shall enter into her, \emph{and} \emph{become} bitter, and her belly shall swell, and her thigh shall rot: and the woman shall be a curse among her people."],
['Numbers','Num','05','028','28',"And if the woman be not defiled, but be clean; then she shall be free, and shall conceive seed."],
['Numbers','Num','05','029','29',"This \emph{is} the law of jealousies, when a wife goeth aside \emph{to} \emph{another} instead of her husband, and is defiled;"],
['Numbers','Num','05','030','30',"Or when the spirit of jealousy cometh upon him, and he be jealous over his wife, and shall set the woman before the LORD, and the priest shall execute upon her all this law."],
['Numbers','Num','05','031','31',"Then shall the man be guiltless from iniquity, and this woman shall bear her iniquity."]
]

Numbers06 = [
    ['Numbers','Num','06','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','06','002','2',"Speak unto the children of Israel, and say unto them, When either man or woman shall separate \emph{themselves} to vow a vow of a Nazarite, to separate \emph{themselves} unto the LORD:"],
['Numbers','Num','06','003','3',"He shall separate \emph{himself} from wine and strong drink, and shall drink no vinegar of wine, or vinegar of strong drink, neither shall he drink any liquor of grapes, nor eat moist grapes, or dried."],
['Numbers','Num','06','004','4',"All the days of his separation shall he eat nothing that is made of the vine tree, from the kernels even to the husk."],
['Numbers','Num','06','005','5',"All the days of the vow of his separation there shall no razor come upon his head: until the days be fulfilled, in the which he separateth \emph{himself} unto the LORD, he shall be holy, \emph{and} shall let the locks of the hair of his head grow."],
['Numbers','Num','06','006','6',"All the days that he separateth \emph{himself} unto the LORD he shall come at no dead body."],
['Numbers','Num','06','007','7',"He shall not make himself unclean for his father, or for his mother, for his brother, or for his sister, when they die: because the consecration of his God \emph{is} upon his head."],
['Numbers','Num','06','008','8',"All the days of his separation he \emph{is} holy unto the LORD."],
['Numbers','Num','06','009','9',"And if any man die very suddenly by him, and he hath defiled the head of his consecration; then he shall shave his head in the day of his cleansing, on the seventh day shall he shave it."],
['Numbers','Num','06','010','10',"And on the eighth day he shall bring two turtles, or two young pigeons, to the priest, to the door of the tabernacle of the congregation:"],
['Numbers','Num','06','011','11',"And the priest shall offer the one for a sin offering, and the other for a burnt offering, and make an atonement for him, for that he sinned by the dead, and shall hallow his head that same day."],
['Numbers','Num','06','012','12',"And he shall consecrate unto the LORD the days of his separation, and shall bring a lamb of the first year for a trespass offering: but the days that were before shall be lost, because his separation was defiled."],
['Numbers','Num','06','013','13',"And this \emph{is} the law of the Nazarite, when the days of his separation are fulfilled: he shall be brought unto the door of the tabernacle of the congregation:"],
['Numbers','Num','06','014','14',"And he shall offer his offering unto the LORD, one he lamb of the first year without blemish for a burnt offering, and one ewe lamb of the first year without blemish for a sin offering, and one ram without blemish for peace offerings,"],
['Numbers','Num','06','015','15',"And a basket of unleavened bread, cakes of fine flour mingled with oil, and wafers of unleavened bread anointed with oil, and their meat offering, and their drink offerings."],
['Numbers','Num','06','016','16',"And the priest shall bring \emph{them} before the LORD, and shall offer his sin offering, and his burnt offering:"],
['Numbers','Num','06','017','17',"And he shall offer the ram \emph{for} a sacrifice of peace offerings unto the LORD, with the basket of unleavened bread: the priest shall offer also his meat offering, and his drink offering."],
['Numbers','Num','06','018','18',"And the Nazarite shall shave the head of his separation \emph{at} the door of the tabernacle of the congregation, and shall take the hair of the head of his separation, and put \emph{it} in the fire which \emph{is} under the sacrifice of the peace offerings."],
['Numbers','Num','06','019','19',"And the priest shall take the sodden shoulder of the ram, and one unleavened cake out of the basket, and one unleavened wafer, and shall put \emph{them} upon the hands of the Nazarite, after \emph{the} \emph{hair} \emph{of} his separation is shaven:"],
['Numbers','Num','06','020','20',"And the priest shall wave them \emph{for} a wave offering before the LORD: this \emph{is} holy for the priest, with the wave breast and heave shoulder: and after that the Nazarite may drink wine."],
['Numbers','Num','06','021','21',"This \emph{is} the law of the Nazarite who hath vowed, \emph{and} \emph{of} his offering unto the LORD for his separation, beside \emph{that} that his hand shall get: according to the vow which he vowed, so he must do after the law of his separation."],
['Numbers','Num','06','022','22',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','06','023','23',"Speak unto Aaron and unto his sons, saying, On this wise ye shall bless the children of Israel, saying unto them,"],
['Numbers','Num','06','024','24',"The LORD bless thee, and keep thee:"],
['Numbers','Num','06','025','25',"The LORD make his face shine upon thee, and be gracious unto thee:"],
    ['Numbers','Num','06','026','26',"The LORD lift up his countenance upon thee, and give thee peace."],
    ['Numbers','Num','06','027','27',"And they shall put my name upon the children of Israel; and I will bless them."]
]

Numbers07 = [
     ['Numbers','Num','07','001','1',"And it came to pass on the day that Moses had fully set up the tabernacle, and had anointed it, and sanctified it, and all the instruments thereof, both the altar and all the vessels thereof, and had anointed them, and sanctified them;"],
     ['Numbers','Num','07','002','2',"That the princes of Israel, heads of the house of their fathers, who \emph{were} the princes of the tribes, and were over them that were numbered, offered:"],
     ['Numbers','Num','07','003','3',"And they brought their offering before the LORD, six covered wagons, and twelve oxen; a wagon for two of the princes, and for each one an ox: and they brought them before the tabernacle."],
     ['Numbers','Num','07','004','4',"And the LORD spake unto Moses, saying,"],
     ['Numbers','Num','07','005','5',"Take \emph{it} of them, that they may be to do the service of the tabernacle of the congregation; and thou shalt give them unto the Levites, to every man according to his service."],
['Numbers','Num','07','006','6',"And Moses took the wagons and the oxen, and gave them unto the Levites."],
['Numbers','Num','07','007','7',"Two wagons and four oxen he gave unto the sons of Gershon, according to their service:"],
['Numbers','Num','07','008','8',"And four wagons and eight oxen he gave unto the sons of Merari, according unto their service, under the hand of Ithamar the son of Aaron the priest."],
['Numbers','Num','07','009','9',"But unto the sons of Kohath he gave none: because the service of the sanctuary belonging unto them \emph{was} \emph{that} they should bear upon their shoulders."],
['Numbers','Num','07','010','10',"And the princes offered for dedicating of the altar in the day that it was anointed, even the princes offered their offering before the altar."],
['Numbers','Num','07','011','11',"And the LORD said unto Moses, They shall offer their offering, each prince on his day, for the dedicating of the altar."],
['Numbers','Num','07','012','12',"And he that offered his offering the first day was Nahshon the son of Amminadab, of the tribe of Judah:"],
['Numbers','Num','07','013','13',"And his offering \emph{was} one silver charger, the weight thereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them \emph{were} full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','014','14',"One spoon of ten \emph{shekels} of gold, full of incense:"],
['Numbers','Num','07','015','15',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','016','16',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','017','17',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Nahshon the son of Amminadab."],
['Numbers','Num','07','018','18',"On the second day Nethaneel the son of Zuar, prince of Issachar, did offer:"],
['Numbers','Num','07','019','19',"He offered \emph{for} his offering one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','020','20',"One spoon of gold of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','021','21',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','022','22',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','023','23',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Nethaneel the son of Zuar."],
['Numbers','Num','07','024','24',"On the third day Eliab the son of Helon, prince of the children of Zebulun, \emph{did} \emph{offer}:"],
['Numbers','Num','07','025','25',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','026','26',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','027','27',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','028','28',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','029','29',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Eliab the son of Helon."],
['Numbers','Num','07','030','30',"On the fourth day Elizur the son of Shedeur, prince of the children of Reuben, \emph{did} \emph{offer}:"],
['Numbers','Num','07','031','31',"His offering \emph{was} one silver charger of the weight of an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','032','32',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','033','33',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','034','34',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','035','35',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Elizur the son of Shedeur."],
['Numbers','Num','07','036','36',"On the fifth day Shelumiel the son of Zurishaddai, prince of the children of Simeon, \emph{did} \emph{offer}:"],
['Numbers','Num','07','037','37',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','038','38',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','039','39',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','040','40',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','041','41',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Shelumiel the son of Zurishaddai."],
['Numbers','Num','07','042','42',"On the sixth day Eliasaph the son of Deuel, prince of the children of Gad, \emph{offered}:"],
['Numbers','Num','07','043','43',"His offering \emph{was} one silver charger of the weight of an hundred and thirty \emph{shekels}, a silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','044','44',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','045','45',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','046','46',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','047','47',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Eliasaph the son of Deuel."],
['Numbers','Num','07','048','48',"On the seventh day Elishama the son of Ammihud, prince of the children of Ephraim, \emph{offered}:"],
['Numbers','Num','07','049','49',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','050','50',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','051','51',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','052','52',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','053','53',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Elishama the son of Ammihud."],
['Numbers','Num','07','054','54',"On the eighth day \emph{offered} Gamaliel the son of Pedahzur, prince of the children of Manasseh:"],
['Numbers','Num','07','055','55',"His offering \emph{was} one silver charger of the weight of an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','056','56',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','057','57',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','058','58',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','059','59',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Gamaliel the son of Pedahzur."],
['Numbers','Num','07','060','60',"On the ninth day Abidan the son of Gideoni, prince of the children of Benjamin, \emph{offered}:"],
['Numbers','Num','07','061','61',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','062','62',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','063','63',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','064','64',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','065','65',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Abidan the son of Gideoni."],
['Numbers','Num','07','066','66',"On the tenth day Ahiezer the son of Ammishaddai, prince of the children of Dan, \emph{offered}:"],
['Numbers','Num','07','067','67',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','068','68',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','069','69',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','070','70',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','071','71',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Ahiezer the son of Ammishaddai."],
['Numbers','Num','07','072','72',"On the eleventh day Pagiel the son of Ocran, prince of the children of Asher, \emph{offered}:"],
['Numbers','Num','07','073','73',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','074','74',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','075','75',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','076','76',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','077','77',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Pagiel the son of Ocran."],
['Numbers','Num','07','078','78',"On the twelfth day Ahira the son of Enan, prince of the children of Naphtali, \emph{offered}:"],
['Numbers','Num','07','079','79',"His offering \emph{was} one silver charger, the weight whereof \emph{was} an hundred and thirty \emph{shekels}, one silver bowl of seventy shekels, after the shekel of the sanctuary; both of them full of fine flour mingled with oil for a meat offering:"],
['Numbers','Num','07','080','80',"One golden spoon of ten \emph{shekels}, full of incense:"],
['Numbers','Num','07','081','81',"One young bullock, one ram, one lamb of the first year, for a burnt offering:"],
['Numbers','Num','07','082','82',"One kid of the goats for a sin offering:"],
['Numbers','Num','07','083','83',"And for a sacrifice of peace offerings, two oxen, five rams, five he goats, five lambs of the first year: this \emph{was} the offering of Ahira the son of Enan."],
['Numbers','Num','07','084','84',"This \emph{was} the dedication of the altar, in the day when it was anointed, by the princes of Israel: twelve chargers of silver, twelve silver bowls, twelve spoons of gold:"],
    ['Numbers','Num','07','085','85',"Each charger of silver \emph{weighing} an hundred and thirty \emph{shekels}, each bowl seventy: all the silver vessels \emph{weighed} two thousand and four hundred \emph{shekels}, after the shekel of the sanctuary:"],
    ['Numbers','Num','07','086','86',"The golden spoons \emph{were} twelve, full of incense, \emph{weighing} ten \emph{shekels} apiece, after the shekel of the sanctuary: all the gold of the spoons \emph{was} an hundred and twenty \emph{shekels}."],
    ['Numbers','Num','07','087','87',"All the oxen for the burnt offering \emph{were} twelve bullocks, the rams twelve, the lambs of the first year twelve, with their meat offering: and the kids of the goats for sin offering twelve."],
    ['Numbers','Num','07','088','88',"And all the oxen for the sacrifice of the peace offerings \emph{were} twenty and four bullocks, the rams sixty, the he goats sixty, the lambs of the first year sixty. This \emph{was} the dedication of the altar, after that it was anointed."],
    ['Numbers','Num','07','089','89',"And when Moses was gone into the tabernacle of the congregation to speak with him, then he heard the voice of one speaking unto him from off the mercy seat that \emph{was} upon the ark of testimony, from between the two cherubims: and he spake unto him."]
]

Numbers08 = [
    ['Numbers','Num','08','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','08','002','2',"Speak unto Aaron, and say unto him, When thou lightest the lamps, the seven lamps shall give light over against the candlestick."],
['Numbers','Num','08','003','3',"And Aaron did so; he lighted the lamps thereof over against the candlestick, as the LORD commanded Moses."],
['Numbers','Num','08','004','4',"And this work of the candlestick \emph{was} \emph{of} beaten gold, unto the shaft thereof, unto the flowers thereof, \emph{was} beaten work: according unto the pattern which the LORD had shewed Moses, so he made the candlestick."],
['Numbers','Num','08','005','5',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','08','006','6',"Take the Levites from among the children of Israel, and cleanse them."],
['Numbers','Num','08','007','7',"And thus shalt thou do unto them, to cleanse them: Sprinkle water of purifying upon them, and let them shave all their flesh, and let them wash their clothes, and \emph{so} make themselves clean."],
['Numbers','Num','08','008','8',"Then let them take a young bullock with his meat offering, \emph{even} fine flour mingled with oil, and another young bullock shalt thou take for a sin offering."],
['Numbers','Num','08','009','9',"And thou shalt bring the Levites before the tabernacle of the congregation: and thou shalt gather the whole assembly of the children of Israel together:"],
['Numbers','Num','08','010','10',"And thou shalt bring the Levites before the LORD: and the children of Israel shall put their hands upon the Levites:"],
['Numbers','Num','08','011','11',"And Aaron shall offer the Levites before the LORD \emph{for} an offering of the children of Israel, that they may execute the service of the LORD."],
['Numbers','Num','08','012','12',"And the Levites shall lay their hands upon the heads of the bullocks: and thou shalt offer the one \emph{for} a sin offering, and the other \emph{for} a burnt offering, unto the LORD, to make an atonement for the Levites."],
['Numbers','Num','08','013','13',"And thou shalt set the Levites before Aaron, and before his sons, and offer them \emph{for} an offering unto the LORD."],
['Numbers','Num','08','014','14',"Thus shalt thou separate the Levites from among the children of Israel: and the Levites shall be mine."],
['Numbers','Num','08','015','15',"And after that shall the Levites go in to do the service of the tabernacle of the congregation: and thou shalt cleanse them, and offer them \emph{for} an offering."],
['Numbers','Num','08','016','16',"For they \emph{are} wholly given unto me from among the children of Israel; instead of such as open every womb, \emph{even} \emph{instead} \emph{of} the firstborn of all the children of Israel, have I taken them unto me."],
['Numbers','Num','08','017','17',"For all the firstborn of the children of Israel \emph{are} mine, \emph{both} man and beast: on the day that I smote every firstborn in the land of Egypt I sanctified them for myself."],
['Numbers','Num','08','018','18',"And I have taken the Levites for all the firstborn of the children of Israel."],
['Numbers','Num','08','019','19',"And I have given the Levites \emph{as} a gift to Aaron and to his sons from among the children of Israel, to do the service of the children of Israel in the tabernacle of the congregation, and to make an atonement for the children of Israel: that there be no plague among the children of Israel, when the children of Israel come nigh unto the sanctuary."],
['Numbers','Num','08','020','20',"And Moses, and Aaron, and all the congregation of the children of Israel, did to the Levites according unto all that the LORD commanded Moses concerning the Levites, so did the children of Israel unto them."],
['Numbers','Num','08','021','21',"And the Levites were purified, and they washed their clothes; and Aaron offered them \emph{as} an offering before the LORD; and Aaron made an atonement for them to cleanse them."],
['Numbers','Num','08','022','22',"And after that went the Levites in to do their service in the tabernacle of the congregation before Aaron, and before his sons: as the LORD had commanded Moses concerning the Levites, so did they unto them."],
['Numbers','Num','08','023','23',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','08','024','24',"This \emph{is} \emph{it} that \emph{belongeth} unto the Levites: from twenty and five years old and upward they shall go in to wait upon the service of the tabernacle of the congregation:"],
['Numbers','Num','08','025','25',"And from the age of fifty years they shall cease waiting upon the service \emph{thereof}, and shall serve no more:"],
['Numbers','Num','08','026','26',"But shall minister with their brethren in the tabernacle of the congregation, to keep the charge, and shall do no service. Thus shalt thou do unto the Levites touching their charge."]
]

Numbers09 = [
    ['Numbers','Num','09','001','1',"And the LORD spake unto Moses in the wilderness of Sinai, in the first month of the second year after they were come out of the land of Egypt, saying,"],
    ['Numbers','Num','09','002','2',"Let the children of Israel also keep the passover at his appointed season."],
    ['Numbers','Num','09','003','3',"In the fourteenth day of this month, at even, ye shall keep it in his appointed season: according to all the rites of it, and according to all the ceremonies thereof, shall ye keep it."],
    ['Numbers','Num','09','004','4',"And Moses spake unto the children of Israel, that they should keep the passover."],
    ['Numbers','Num','09','005','5',"And they kept the passover on the fourteenth day of the first month at even in the wilderness of Sinai: according to all that the LORD commanded Moses, so did the children of Israel."],
    ['Numbers','Num','09','006','6',"And there were certain men, who were defiled by the dead body of a man, that they could not keep the passover on that day: and they came before Moses and before Aaron on that day:"],
    ['Numbers','Num','09','007','7',"And those men said unto him, We \emph{are} defiled by the dead body of a man: wherefore are we kept back, that we may not offer an offering of the LORD in his appointed season among the children of Israel?"],
    ['Numbers','Num','09','008','8',"And Moses said unto them, Stand still, and I will hear what the LORD will command concerning you."],
    ['Numbers','Num','09','009','9',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','09','010','10',"Speak unto the children of Israel, saying, If any man of you or of your posterity shall be unclean by reason of a dead body, or \emph{be} in a journey afar off, yet he shall keep the passover unto the LORD."],
    ['Numbers','Num','09','011','11',"The fourteenth day of the second month at even they shall keep it, \emph{and} eat it with unleavened bread and bitter \emph{herbs}."],
    ['Numbers','Num','09','012','12',"They shall leave none of it unto the morning, nor break any bone of it: according to all the ordinances of the passover they shall keep it."],
    ['Numbers','Num','09','013','13',"But the man that \emph{is} clean, and is not in a journey, and forbeareth to keep the passover, even the same soul shall be cut off from among his people: because he brought not the offering of the LORD in his appointed season, that man shall bear his sin."],
    ['Numbers','Num','09','014','14',"And if a stranger shall sojourn among you, and will keep the passover unto the LORD; according to the ordinance of the passover, and according to the manner thereof, so shall he do: ye shall have one ordinance, both for the stranger, and for him that was born in the land."],
    ['Numbers','Num','09','015','15',"And on the day that the tabernacle was reared up the cloud covered the tabernacle, \emph{namely}, the tent of the testimony: and at even there was upon the tabernacle as it were the appearance of fire, until the morning."],
    ['Numbers','Num','09','016','16',"So it was alway: the cloud covered it \emph{by} \emph{day}, and the appearance of fire by night."],
    ['Numbers','Num','09','017','17',"And when the cloud was taken up from the tabernacle, then after that the children of Israel journeyed: and in the place where the cloud abode, there the children of Israel pitched their tents."],
    ['Numbers','Num','09','018','18',"At the commandment of the LORD the children of Israel journeyed, and at the commandment of the LORD they pitched: as long as the cloud abode upon the tabernacle they rested in their tents."],
    ['Numbers','Num','09','019','19',"And when the cloud tarried long upon the tabernacle many days, then the children of Israel kept the charge of the LORD, and journeyed not."],
    ['Numbers','Num','09','020','20',"And \emph{so} it was, when the cloud was a few days upon the tabernacle; according to the commandment of the LORD they abode in their tents, and according to the commandment of the LORD they journeyed."],
    ['Numbers','Num','09','021','21',"And \emph{so} it was, when the cloud abode from even unto the morning, and \emph{that} the cloud was taken up in the morning, then they journeyed: whether \emph{it} \emph{was} by day or by night that the cloud was taken up, they journeyed."],
    ['Numbers','Num','09','022','22',"Or \emph{whether} \emph{it} \emph{were} two days, or a month, or a year, that the cloud tarried upon the tabernacle, remaining thereon, the children of Israel abode in their tents, and journeyed not: but when it was taken up, they journeyed."],
    ['Numbers','Num','09','023','23',"At the commandment of the LORD they rested in the tents, and at the commandment of the LORD they journeyed: they kept the charge of the LORD, at the commandment of the LORD by the hand of Moses."]
    ]

Numbers10 = [
    ['Numbers','Num','10','001','1',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','10','002','2',"Make thee two trumpets of silver; of a whole piece shalt thou make them: that thou mayest use them for the calling of the assembly, and for the journeying of the camps."],
    ['Numbers','Num','10','003','3',"And when they shall blow with them, all the assembly shall assemble themselves to thee at the door of the tabernacle of the congregation."],
['Numbers','Num','10','004','4',"And if they blow \emph{but} with one \emph{trumpet}, then the princes, \emph{which} \emph{are} heads of the thousands of Israel, shall gather themselves unto thee."],
['Numbers','Num','10','005','5',"When ye blow an alarm, then the camps that lie on the east parts shall go forward."],
['Numbers','Num','10','006','6',"When ye blow an alarm the second time, then the camps that lie on the south side shall take their journey: they shall blow an alarm for their journeys."],
['Numbers','Num','10','007','7',"But when the congregation is to be gathered together, ye shall blow, but ye shall not sound an alarm."],
['Numbers','Num','10','008','8',"And the sons of Aaron, the priests, shall blow with the trumpets; and they shall be to you for an ordinance for ever throughout your generations."],
['Numbers','Num','10','009','9',"And if ye go to war in your land against the enemy that oppresseth you, then ye shall blow an alarm with the trumpets; and ye shall be remembered before the LORD your God, and ye shall be saved from your enemies."],
['Numbers','Num','10','010','10',"Also in the day of your gladness, and in your solemn days, and in the beginnings of your months, ye shall blow with the trumpets over your burnt offerings, and over the sacrifices of your peace offerings; that they may be to you for a memorial before your God: I \emph{am} the LORD your God."],
['Numbers','Num','10','011','11',"And it came to pass on the twentieth \emph{day} of the second month, in the second year, that the cloud was taken up from off the tabernacle of the testimony."],
['Numbers','Num','10','012','12',"And the children of Israel took their journeys out of the wilderness of Sinai; and the cloud rested in the wilderness of Paran."],
['Numbers','Num','10','013','13',"And they first took their journey according to the commandment of the LORD by the hand of Moses."],
['Numbers','Num','10','014','14',"In the first \emph{place} went the standard of the camp of the children of Judah according to their armies: and over his host \emph{was} Nahshon the son of Amminadab."],
['Numbers','Num','10','015','15',"And over the host of the tribe of the children of Issachar \emph{was} Nethaneel the son of Zuar."],
['Numbers','Num','10','016','16',"And over the host of the tribe of the children of Zebulun \emph{was} Eliab the son of Helon."],
['Numbers','Num','10','017','17',"And the tabernacle was taken down; and the sons of Gershon and the sons of Merari set forward, bearing the tabernacle."],
['Numbers','Num','10','018','18',"And the standard of the camp of Reuben set forward according to their armies: and over his host \emph{was} Elizur the son of Shedeur."],
['Numbers','Num','10','019','19',"And over the host of the tribe of the children of Simeon \emph{was} Shelumiel the son of Zurishaddai."],
['Numbers','Num','10','020','20',"And over the host of the tribe of the children of Gad \emph{was} Eliasaph the son of Deuel."],
['Numbers','Num','10','021','21',"And the Kohathites set forward, bearing the sanctuary: and \emph{the} \emph{other} did set up the tabernacle against they came."],
['Numbers','Num','10','022','22',"And the standard of the camp of the children of Ephraim set forward according to their armies: and over his host \emph{was} Elishama the son of Ammihud."],
['Numbers','Num','10','023','23',"And over the host of the tribe of the children of Manasseh \emph{was} Gamaliel the son of Pedahzur."],
['Numbers','Num','10','024','24',"And over the host of the tribe of the children of Benjamin \emph{was} Abidan the son of Gideoni."],
['Numbers','Num','10','025','25',"And the standard of the camp of the children of Dan set forward, \emph{which} \emph{was} the rereward of all the camps throughout their hosts: and over his host \emph{was} Ahiezer the son of Ammishaddai."],
['Numbers','Num','10','026','26',"And over the host of the tribe of the children of Asher \emph{was} Pagiel the son of Ocran."],
['Numbers','Num','10','027','27',"And over the host of the tribe of the children of Naphtali \emph{was} Ahira the son of Enan."],
['Numbers','Num','10','028','28',"Thus \emph{were} the journeyings of the children of Israel according to their armies, when they set forward."],
['Numbers','Num','10','029','29',"And Moses said unto Hobab, the son of Raguel the Midianite, Moses' father in law, We are journeying unto the place of which the LORD said, I will give it you: come thou with us, and we will do thee good: for the LORD hath spoken good concerning Israel."],
['Numbers','Num','10','030','30',"And he said unto him, I will not go; but I will depart to mine own land, and to my kindred."],
['Numbers','Num','10','031','31',"And he said, Leave us not, I pray thee; forasmuch as thou knowest how we are to encamp in the wilderness, and thou mayest be to us instead of eyes."],
['Numbers','Num','10','032','32',"And it shall be, if thou go with us, yea, it shall be, that what goodness the LORD shall do unto us, the same will we do unto thee."],
    ['Numbers','Num','10','033','33',"And they departed from the mount of the LORD three days' journey: and the ark of the covenant of the LORD went before them in the three days' journey, to search out a resting place for them."],
    ['Numbers','Num','10','034','34',"And the cloud of the LORD \emph{was} upon them by day, when they went out of the camp."],
    ['Numbers','Num','10','035','35',"And it came to pass, when the ark set forward, that Moses said, Rise up, LORD, and let thine enemies be scattered; and let them that hate thee flee before thee."],
    ['Numbers','Num','10','036','36',"And when it rested, he said, Return, O LORD, unto the many thousands of Israel."]
]

Numbers11 = [
    ['Numbers','Num','11','001','1',"And \emph{when} the people complained, it displeased the LORD: and the LORD heard \emph{it}; and his anger was kindled; and the fire of the LORD burnt among them, and consumed \emph{them} \emph{that} \emph{were} in the uttermost parts of the camp."],
    ['Numbers','Num','11','002','2',"And the people cried unto Moses; and when Moses prayed unto the LORD, the fire was quenched."],
    ['Numbers','Num','11','003','3',"And he called the name of the place Taberah: because the fire of the LORD burnt among them."],
    ['Numbers','Num','11','004','4',"And the mixt multitude that \emph{was} among them fell a lusting: and the children of Israel also wept again, and said, Who shall give us flesh to eat?"],
['Numbers','Num','11','005','5',"We remember the fish, which we did eat in Egypt freely; the cucumbers, and the melons, and the leeks, and the onions, and the garlick:"],
['Numbers','Num','11','006','6',"But now our soul \emph{is} dried away: \emph{there} \emph{is} nothing at all, beside this manna, \emph{before} our eyes."],
['Numbers','Num','11','007','7',"And the manna \emph{was} as coriander seed, and the colour thereof as the colour of bdellium."],
['Numbers','Num','11','008','8',"\emph{And} the people went about, and gathered \emph{it}, and ground \emph{it} in mills, or beat \emph{it} in a mortar, and baked \emph{it} in pans, and made cakes of it: and the taste of it was as the taste of fresh oil."],
['Numbers','Num','11','009','9',"And when the dew fell upon the camp in the night, the manna fell upon it."],
['Numbers','Num','11','010','10',"Then Moses heard the people weep throughout their families, every man in the door of his tent: and the anger of the LORD was kindled greatly; Moses also was displeased."],
['Numbers','Num','11','011','11',"And Moses said unto the LORD, Wherefore hast thou afflicted thy servant? and wherefore have I not found favour in thy sight, that thou layest the burden of all this people upon me?"],
['Numbers','Num','11','012','12',"Have I conceived all this people? have I begotten them, that thou shouldest say unto me, Carry them in thy bosom, as a nursing father beareth the sucking child, unto the land which thou swarest unto their fathers?"],
['Numbers','Num','11','013','13',"Whence should I have flesh to give unto all this people? for they weep unto me, saying, Give us flesh, that we may eat."],
['Numbers','Num','11','014','14',"I am not able to bear all this people alone, because \emph{it} \emph{is} too heavy for me."],
['Numbers','Num','11','015','15',"And if thou deal thus with me, kill me, I pray thee, out of hand, if I have found favour in thy sight; and let me not see my wretchedness."],
['Numbers','Num','11','016','16',"And the LORD said unto Moses, Gather unto me seventy men of the elders of Israel, whom thou knowest to be the elders of the people, and officers over them; and bring them unto the tabernacle of the congregation, that they may stand there with thee."],
['Numbers','Num','11','017','17',"And I will come down and talk with thee there: and I will take of the spirit which \emph{is} upon thee, and will put \emph{it} upon them; and they shall bear the burden of the people with thee, that thou bear \emph{it} not thyself alone."],
['Numbers','Num','11','018','18',"And say thou unto the people, Sanctify yourselves against to morrow, and ye shall eat flesh: for ye have wept in the ears of the LORD, saying, Who shall give us flesh to eat? for \emph{it} \emph{was} well with us in Egypt: therefore the LORD will give you flesh, and ye shall eat."],
['Numbers','Num','11','019','19',"Ye shall not eat one day, nor two days, nor five days, neither ten days, nor twenty days;"],
['Numbers','Num','11','020','20',"\emph{But} even a whole month, until it come out at your nostrils, and it be loathsome unto you: because that ye have despised the LORD which \emph{is} among you, and have wept before him, saying, Why came we forth out of Egypt?"],
['Numbers','Num','11','021','21',"And Moses said, The people, among whom I \emph{am}, \emph{are} six hundred thousand footmen; and thou hast said, I will give them flesh, that they may eat a whole month."],
['Numbers','Num','11','022','22',"Shall the flocks and the herds be slain for them, to suffice them? or shall all the fish of the sea be gathered together for them, to suffice them?"],
['Numbers','Num','11','023','23',"And the LORD said unto Moses, Is the LORD'S hand waxed short? thou shalt see now whether my word shall come to pass unto thee or not."],
['Numbers','Num','11','024','24',"And Moses went out, and told the people the words of the LORD, and gathered the seventy men of the elders of the people, and set them round about the tabernacle."],
['Numbers','Num','11','025','25',"And the LORD came down in a cloud, and spake unto him, and took of the spirit that \emph{was} upon him, and gave \emph{it} unto the seventy elders: and it came to pass, \emph{that}, when the spirit rested upon them, they prophesied, and did not cease."],
['Numbers','Num','11','026','26',"But there remained two \emph{of} \emph{the} men in the camp, the name of the one \emph{was} Eldad, and the name of the other Medad: and the spirit rested upon them; and they \emph{were} of them that were written, but went not out unto the tabernacle: and they prophesied in the camp."],
['Numbers','Num','11','027','27',"And there ran a young man, and told Moses, and said, Eldad and Medad do prophesy in the camp."],
['Numbers','Num','11','028','28',"And Joshua the son of Nun, the servant of Moses, \emph{one} of his young men, answered and said, My lord Moses, forbid them."],
['Numbers','Num','11','029','29',"And Moses said unto him, Enviest thou for my sake? would God that all the LORD'S people were prophets, \emph{and} that the LORD would put his spirit upon them!"],
['Numbers','Num','11','030','30',"And Moses gat him into the camp, he and the elders of Israel."],
    ['Numbers','Num','11','031','31',"And there went forth a wind from the LORD, and brought quails from the sea, and let \emph{them} fall by the camp, as it were a day's journey on this side, and as it were a day's journey on the other side, round about the camp, and as it were two cubits \emph{high} upon the face of the earth."],
    ['Numbers','Num','11','032','32',"And the people stood up all that day, and all \emph{that} night, and all the next day, and they gathered the quails: he that gathered least gathered ten homers: and they spread \emph{them} all abroad for themselves round about the camp."],
    ['Numbers','Num','11','033','33',"And while the flesh \emph{was} yet between their teeth, ere it was chewed, the wrath of the LORD was kindled against the people, and the LORD smote the people with a very great plague."],
    ['Numbers','Num','11','034','34',"And he called the name of that place Kibroth-hattaavah: because there they buried the people that lusted."],
    ['Numbers','Num','11','035','35',"\emph{And} the people journeyed from Kibroth-hattaavah unto Hazeroth; and abode at Hazeroth."]
]

Numbers12 = [
    ['Numbers','Num','12','001','1',"And Miriam and Aaron spake against Moses because of the Ethiopian woman whom he had married: for he had married an Ethiopian woman."],
    ['Numbers','Num','12','002','2',"And they said, Hath the LORD indeed spoken only by Moses? hath he not spoken also by us? And the LORD heard \emph{it}."],
    ['Numbers','Num','12','003','3',"(Now the man Moses \emph{was} very meek, above all the men which \emph{were} upon the face of the earth.)"],
    ['Numbers','Num','12','004','4',"And the LORD spake suddenly unto Moses, and unto Aaron, and unto Miriam, Come out ye three unto the tabernacle of the congregation. And they three came out."],
    ['Numbers','Num','12','005','5',"And the LORD came down in the pillar of the cloud, and stood \emph{in} the door of the tabernacle, and called Aaron and Miriam: and they both came forth."],
    ['Numbers','Num','12','006','6',"And he said, Hear now my words: If there be a prophet among you, \emph{I} the LORD will make myself known unto him in a vision, \emph{and} will speak unto him in a dream."],
    ['Numbers','Num','12','007','7',"My servant Moses \emph{is} not so, who \emph{is} faithful in all mine house."],
    ['Numbers','Num','12','008','8',"With him will I speak mouth to mouth, even apparently, and not in dark speeches; and the similitude of the LORD shall he behold: wherefore then were ye not afraid to speak against my servant Moses?"],
    ['Numbers','Num','12','009','9',"And the anger of the LORD was kindled against them; and he departed."],
    ['Numbers','Num','12','010','10',"And the cloud departed from off the tabernacle; and, behold, Miriam \emph{became} leprous, \emph{white} as snow: and Aaron looked upon Miriam, and, behold, \emph{she} \emph{was} leprous."],
    ['Numbers','Num','12','011','11',"And Aaron said unto Moses, Alas, my lord, I beseech thee, lay not the sin upon us, wherein we have done foolishly, and wherein we have sinned."],
    ['Numbers','Num','12','012','12',"Let her not be as one dead, of whom the flesh is half consumed when he cometh out of his mother's womb."],
    ['Numbers','Num','12','013','13',"And Moses cried unto the LORD, saying, Heal her now, O God, I beseech thee."],
    ['Numbers','Num','12','014','14',"And the LORD said unto Moses, If her father had but spit in her face, should she not be ashamed seven days? let her be shut out from the camp seven days, and after that let her be received in \emph{again}."],
    ['Numbers','Num','12','015','15',"And Miriam was shut out from the camp seven days: and the people journeyed not till Miriam was brought in \emph{again}."],
    ['Numbers','Num','12','016','16',"And afterward the people removed from Hazeroth, and pitched in the wilderness of Paran."]
]

Numbers13 = [
    ['Numbers','Num','13','001','1',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','13','002','2',"Send thou men, that they may search the land of Canaan, which I give unto the children of Israel: of every tribe of their fathers shall ye send a man, every one a ruler among them."],
    ['Numbers','Num','13','003','3',"And Moses by the commandment of the LORD sent them from the wilderness of Paran: all those men \emph{were} heads of the children of Israel."],
    ['Numbers','Num','13','004','4',"And these \emph{were} their names: of the tribe of Reuben, Shammua the son of Zaccur."],
    ['Numbers','Num','13','005','5',"Of the tribe of Simeon, Shaphat the son of Hori."],
    ['Numbers','Num','13','006','6',"Of the tribe of Judah, Caleb the son of Jephunneh."],
    ['Numbers','Num','13','007','7',"Of the tribe of Issachar, Igal the son of Joseph."],
    ['Numbers','Num','13','008','8',"Of the tribe of Ephraim, Oshea the son of Nun."],
    ['Numbers','Num','13','009','9',"Of the tribe of Benjamin, Palti the son of Raphu."],
    ['Numbers','Num','13','010','10',"Of the tribe of Zebulun, Gaddiel the son of Sodi."],
    ['Numbers','Num','13','011','11',"Of the tribe of Joseph, \emph{namely}, of the tribe of Manasseh, Gaddi the son of Susi."],
    ['Numbers','Num','13','012','12',"Of the tribe of Dan, Ammiel the son of Gemalli."],
    ['Numbers','Num','13','013','13',"Of the tribe of Asher, Sethur the son of Michael."],
    ['Numbers','Num','13','014','14',"Of the tribe of Naphtali, Nahbi the son of Vophsi."],
    ['Numbers','Num','13','015','15',"Of the tribe of Gad, Geuel the son of Machi."],
    ['Numbers','Num','13','016','16',"These \emph{are} the names of the men which Moses sent to spy out the land. And Moses called Oshea the son of Nun Jehoshua."],
    ['Numbers','Num','13','017','17',"And Moses sent them to spy out the land of Canaan, and said unto them, Get you up this \emph{way} southward, and go up into the mountain:"],
    ['Numbers','Num','13','018','18',"And see the land, what it \emph{is}; and the people that dwelleth therein, whether they \emph{be} strong or weak, few or many;"],
    ['Numbers','Num','13','019','19',"And what the land \emph{is} that they dwell in, whether it \emph{be} good or bad; and what cities \emph{they} \emph{be} that they dwell in, whether in tents, or in strong holds;"],
    ['Numbers','Num','13','020','20',"And what the land \emph{is}, whether it \emph{be} fat or lean, whether there be wood therein, or not. And be ye of good courage, and bring of the fruit of the land. Now the time \emph{was} the time of the firstripe grapes."],
    ['Numbers','Num','13','021','21',"So they went up, and searched the land from the wilderness of Zin unto Rehob, as men come to Hamath."],
    ['Numbers','Num','13','022','22',"And they ascended by the south, and came unto Hebron; where Ahiman, Sheshai, and Talmai, the children of Anak, \emph{were}. (Now Hebron was built seven years before Zoan in Egypt.)"],
    ['Numbers','Num','13','023','23',"And they came unto the brook of Eshcol, and cut down from thence a branch with one cluster of grapes, and they bare it between two upon a staff; and \emph{they} \emph{brought} of the pomegranates, and of the figs."],
    ['Numbers','Num','13','024','24',"The place was called the brook Eshcol, because of the cluster of grapes which the children of Israel cut down from thence."],
    ['Numbers','Num','13','025','25',"And they returned from searching of the land after forty days."],
    ['Numbers','Num','13','026','26',"And they went and came to Moses, and to Aaron, and to all the congregation of the children of Israel, unto the wilderness of Paran, to Kadesh; and brought back word unto them, and unto all the congregation, and shewed them the fruit of the land."],
    ['Numbers','Num','13','027','27',"And they told him, and said, We came unto the land whither thou sentest us, and surely it floweth with milk and honey; and this \emph{is} the fruit of it."],
    ['Numbers','Num','13','028','28',"Nevertheless the people \emph{be} strong that dwell in the land, and the cities \emph{are} walled, \emph{and} very great: and moreover we saw the children of Anak there."],
    ['Numbers','Num','13','029','29',"The Amalekites dwell in the land of the south: and the Hittites, and the Jebusites, and the Amorites, dwell in the mountains: and the Canaanites dwell by the sea, and by the coast of Jordan."],
    ['Numbers','Num','13','030','30',"And Caleb stilled the people before Moses, and said, Let us go up at once, and possess it; for we are well able to overcome it."],
    ['Numbers','Num','13','031','31',"But the men that went up with him said, We be not able to go up against the people; for they \emph{are} stronger than we."],
    ['Numbers','Num','13','032','32',"And they brought up an evil report of the land which they had searched unto the children of Israel, saying, The land, through which we have gone to search it, \emph{is} a land that eateth up the inhabitants thereof; and all the people that we saw in it \emph{are} men of a great stature."],
    ['Numbers','Num','13','033','33',"And there we saw the giants, the sons of Anak, \emph{which} \emph{come} of the giants: and we were in our own sight as grasshoppers, and so we were in their sight."]
]

Numbers14 = [
     ['Numbers','Num','14','001','1',"And all the congregation lifted up their voice, and cried; and the people wept that night."],
     ['Numbers','Num','14','002','2',"And all the children of Israel murmured against Moses and against Aaron: and the whole congregation said unto them, Would God that we had died in the land of Egypt! or would God we had died in this wilderness!"],
     ['Numbers','Num','14','003','3',"And wherefore hath the LORD brought us unto this land, to fall by the sword, that our wives and our children should be a prey? were it not better for us to return into Egypt?"],
     ['Numbers','Num','14','004','4',"And they said one to another, Let us make a captain, and let us return into Egypt."],
['Numbers','Num','14','005','5',"Then Moses and Aaron fell on their faces before all the assembly of the congregation of the children of Israel."],
['Numbers','Num','14','006','6',"And Joshua the son of Nun, and Caleb the son of Jephunneh, \emph{which} \emph{were} of them that searched the land, rent their clothes:"],
['Numbers','Num','14','007','7',"And they spake unto all the company of the children of Israel, saying, The land, which we passed through to search it, \emph{is} an exceeding good land."],
['Numbers','Num','14','008','8',"If the LORD delight in us, then he will bring us into this land, and give it us; a land which floweth with milk and honey."],
['Numbers','Num','14','009','9',"Only rebel not ye against the LORD, neither fear ye the people of the land; for they \emph{are} bread for us: their defence is departed from them, and the LORD \emph{is} with us: fear them not."],
['Numbers','Num','14','010','10',"But all the congregation bade stone them with stones. And the glory of the LORD appeared in the tabernacle of the congregation before all the children of Israel."],
['Numbers','Num','14','011','11',"And the LORD said unto Moses, How long will this people provoke me? and how long will it be ere they believe me, for all the signs which I have shewed among them?"],
['Numbers','Num','14','012','12',"I will smite them with the pestilence, and disinherit them, and will make of thee a greater nation and mightier than they."],
['Numbers','Num','14','013','13',"And Moses said unto the LORD, Then the Egyptians shall hear \emph{it}, (for thou broughtest up this people in thy might from among them;)"],
['Numbers','Num','14','014','14',"And they will tell \emph{it} to the inhabitants of this land: \emph{for} they have heard that thou LORD \emph{art} among this people, that thou LORD art seen face to face, and \emph{that} thy cloud standeth over them, and \emph{that} thou goest before them, by day time in a pillar of a cloud, and in a pillar of fire by night."],
['Numbers','Num','14','015','15',"Now \emph{if} thou shalt kill \emph{all} this people as one man, then the nations which have heard the fame of thee will speak, saying,"],
['Numbers','Num','14','016','16',"Because the LORD was not able to bring this people into the land which he sware unto them, therefore he hath slain them in the wilderness."],
['Numbers','Num','14','017','17',"And now, I beseech thee, let the power of my Lord be great, according as thou hast spoken, saying,"],
['Numbers','Num','14','018','18',"The LORD \emph{is} longsuffering, and of great mercy, forgiving iniquity and transgression, and by no means clearing \emph{the} \emph{guilty}, visiting the iniquity of the fathers upon the children unto the third and fourth \emph{generation}."],
['Numbers','Num','14','019','19',"Pardon, I beseech thee, the iniquity of this people according unto the greatness of thy mercy, and as thou hast forgiven this people, from Egypt even until now."],
['Numbers','Num','14','020','20',"And the LORD said, I have pardoned according to thy word:"],
['Numbers','Num','14','021','21',"But \emph{as} truly \emph{as} I live, all the earth shall be filled with the glory of the LORD."],
['Numbers','Num','14','022','22',"Because all those men which have seen my glory, and my miracles, which I did in Egypt and in the wilderness, and have tempted me now these ten times, and have not hearkened to my voice;"],
['Numbers','Num','14','023','23',"Surely they shall not see the land which I sware unto their fathers, neither shall any of them that provoked me see it:"],
['Numbers','Num','14','024','24',"But my servant Caleb, because he had another spirit with him, and hath followed me fully, him will I bring into the land whereinto he went; and his seed shall possess it."],
['Numbers','Num','14','025','25',"(Now the Amalekites and the Canaanites dwelt in the valley.) To morrow turn you, and get you into the wilderness by the way of the Red sea."],
['Numbers','Num','14','026','26',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Numbers','Num','14','027','27',"How long \emph{shall} \emph{I} \emph{bear} \emph{with} this evil congregation, which murmur against me? I have heard the murmurings of the children of Israel, which they murmur against me."],
['Numbers','Num','14','028','28',"Say unto them, \emph{As} \emph{truly} \emph{as} I live, saith the LORD, as ye have spoken in mine ears, so will I do to you:"],
['Numbers','Num','14','029','29',"Your carcases shall fall in this wilderness; and all that were numbered of you, according to your whole number, from twenty years old and upward, which have murmured against me,"],
['Numbers','Num','14','030','30',"Doubtless ye shall not come into the land, \emph{concerning} which I sware to make you dwell therein, save Caleb the son of Jephunneh, and Joshua the son of Nun."],
['Numbers','Num','14','031','31',"But your little ones, which ye said should be a prey, them will I bring in, and they shall know the land which ye have despised."],
['Numbers','Num','14','032','32',"But \emph{as} \emph{for} you, your carcases, they shall fall in this wilderness."],
['Numbers','Num','14','033','33',"And your children shall wander in the wilderness forty years, and bear your whoredoms, until your carcases be wasted in the wilderness."],
['Numbers','Num','14','034','34',"After the number of the days in which ye searched the land, \emph{even} forty days, each day for a year, shall ye bear your iniquities, \emph{even} forty years, and ye shall know my breach of promise."],
['Numbers','Num','14','035','35',"I the LORD have said, I will surely do it unto all this evil congregation, that are gathered together against me: in this wilderness they shall be consumed, and there they shall die."],
['Numbers','Num','14','036','36',"And the men, which Moses sent to search the land, who returned, and made all the congregation to murmur against him, by bringing up a slander upon the land,"],
['Numbers','Num','14','037','37',"Even those men that did bring up the evil report upon the land, died by the plague before the LORD."],
['Numbers','Num','14','038','38',"But Joshua the son of Nun, and Caleb the son of Jephunneh, \emph{which} \emph{were} of the men that went to search the land, lived \emph{still}."],
['Numbers','Num','14','039','39',"And Moses told these sayings unto all the children of Israel: and the people mourned greatly."],
['Numbers','Num','14','040','40',"And they rose up early in the morning, and gat them up into the top of the mountain, saying, Lo, we \emph{be} \emph{here}, and will go up unto the place which the LORD hath promised: for we have sinned."],
['Numbers','Num','14','041','41',"And Moses said, Wherefore now do ye transgress the commandment of the LORD? but it shall not prosper."],
    ['Numbers','Num','14','042','42',"Go not up, for the LORD \emph{is} not among you; that ye be not smitten before your enemies."],
    ['Numbers','Num','14','043','43',"For the Amalekites and the Canaanites \emph{are} there before you, and ye shall fall by the sword: because ye are turned away from the LORD, therefore the LORD will not be with you."],
    ['Numbers','Num','14','044','44',"But they presumed to go up unto the hill top: nevertheless the ark of the covenant of the LORD, and Moses, departed not out of the camp."],
    ['Numbers','Num','14','045','45',"Then the Amalekites came down, and the Canaanites which dwelt in that hill, and smote them, and discomfited them, \emph{even} unto Hormah."]
]

Numbers15 = [
    ['Numbers','Num','15','001','1',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','15','002','2',"Speak unto the children of Israel, and say unto them, When ye be come into the land of your habitations, which I give unto you,"],
    ['Numbers','Num','15','003','3',"And will make an offering by fire unto the LORD, a burnt offering, or a sacrifice in performing a vow, or in a freewill offering, or in your solemn feasts, to make a sweet savour unto the LORD, of the herd, or of the flock:"],
    ['Numbers','Num','15','004','4',"Then shall he that offereth his offering unto the LORD bring a meat offering of a tenth deal of flour mingled with the fourth \emph{part} of an hin of oil."],
    ['Numbers','Num','15','005','5',"And the fourth \emph{part} of an hin of wine for a drink offering shalt thou prepare with the burnt offering or sacrifice, for one lamb."],
    ['Numbers','Num','15','006','6',"Or for a ram, thou shalt prepare \emph{for} a meat offering two tenth deals of flour mingled with the third \emph{part} of an hin of oil."],
['Numbers','Num','15','007','7',"And for a drink offering thou shalt offer the third \emph{part} of an hin of wine, \emph{for} a sweet savour unto the LORD."],
['Numbers','Num','15','008','8',"And when thou preparest a bullock \emph{for} a burnt offering, or \emph{for} a sacrifice in performing a vow, or peace offerings unto the LORD:"],
['Numbers','Num','15','009','9',"Then shall he bring with a bullock a meat offering of three tenth deals of flour mingled with half an hin of oil."],
['Numbers','Num','15','010','10',"And thou shalt bring for a drink offering half an hin of wine, \emph{for} an offering made by fire, of a sweet savour unto the LORD."],
['Numbers','Num','15','011','11',"Thus shall it be done for one bullock, or for one ram, or for a lamb, or a kid."],
['Numbers','Num','15','012','12',"According to the number that ye shall prepare, so shall ye do to every one according to their number."],
['Numbers','Num','15','013','13',"All that are born of the country shall do these things after this manner, in offering an offering made by fire, of a sweet savour unto the LORD."],
['Numbers','Num','15','014','14',"And if a stranger sojourn with you, or whosoever \emph{be} among you in your generations, and will offer an offering made by fire, of a sweet savour unto the LORD; as ye do, so he shall do."],
['Numbers','Num','15','015','15',"One ordinance \emph{shall} \emph{be} \emph{both} for you of the congregation, and also for the stranger that sojourneth \emph{with} \emph{you}, an ordinance for ever in your generations: as ye \emph{are}, so shall the stranger be before the LORD."],
['Numbers','Num','15','016','16',"One law and one manner shall be for you, and for the stranger that sojourneth with you."],
['Numbers','Num','15','017','17',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','15','018','18',"Speak unto the children of Israel, and say unto them, When ye come into the land whither I bring you,"],
['Numbers','Num','15','019','19',"Then it shall be, that, when ye eat of the bread of the land, ye shall offer up an heave offering unto the LORD."],
['Numbers','Num','15','020','20',"Ye shall offer up a cake of the first of your dough \emph{for} an heave offering: as \emph{ye} \emph{do} the heave offering of the threshingfloor, so shall ye heave it."],
['Numbers','Num','15','021','21',"Of the first of your dough ye shall give unto the LORD an heave offering in your generations."],
['Numbers','Num','15','022','22',"And if ye have erred, and not observed all these commandments, which the LORD hath spoken unto Moses,"],
['Numbers','Num','15','023','23',"\emph{Even} all that the LORD hath commanded you by the hand of Moses, from the day that the LORD commanded \emph{Moses}, and henceforward among your generations;"],
['Numbers','Num','15','024','24',"Then it shall be, if \emph{ought} be committed by ignorance without the knowledge of the congregation, that all the congregation shall offer one young bullock for a burnt offering, for a sweet savour unto the LORD, with his meat offering, and his drink offering, according to the manner, and one kid of the goats for a sin offering."],
['Numbers','Num','15','025','25',"And the priest shall make an atonement for all the congregation of the children of Israel, and it shall be forgiven them; for it \emph{is} ignorance: and they shall bring their offering, a sacrifice made by fire unto the LORD, and their sin offering before the LORD, for their ignorance:"],
['Numbers','Num','15','026','26',"And it shall be forgiven all the congregation of the children of Israel, and the stranger that sojourneth among them; seeing all the people \emph{were} in ignorance."],
['Numbers','Num','15','027','27',"And if any soul sin through ignorance, then he shall bring a she goat of the first year for a sin offering."],
['Numbers','Num','15','028','28',"And the priest shall make an atonement for the soul that sinneth ignorantly, when he sinneth by ignorance before the LORD, to make an atonement for him; and it shall be forgiven him."],
['Numbers','Num','15','029','29',"Ye shall have one law for him that sinneth through ignorance, \emph{both} \emph{for} him that is born among the children of Israel, and for the stranger that sojourneth among them."],
['Numbers','Num','15','030','30',"But the soul that doeth \emph{ought} presumptuously, \emph{whether} \emph{he} \emph{be} born in the land, or a stranger, the same reproacheth the LORD; and that soul shall be cut off from among his people."],
['Numbers','Num','15','031','31',"Because he hath despised the word of the LORD, and hath broken his commandment, that soul shall utterly be cut off; his iniquity \emph{shall} \emph{be} upon him."],
['Numbers','Num','15','032','32',"And while the children of Israel were in the wilderness, they found a man that gathered sticks upon the sabbath day."],
['Numbers','Num','15','033','33',"And they that found him gathering sticks brought him unto Moses and Aaron, and unto all the congregation."],
['Numbers','Num','15','034','34',"And they put him in ward, because it was not declared what should be done to him."],
    ['Numbers','Num','15','035','35',"And the LORD said unto Moses, The man shall be surely put to death: all the congregation shall stone him with stones without the camp."],
    ['Numbers','Num','15','036','36',"And all the congregation brought him without the camp, and stoned him with stones, and he died; as the LORD commanded Moses."],
    ['Numbers','Num','15','037','37',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','15','038','38',"Speak unto the children of Israel, and bid them that they make them fringes in the borders of their garments throughout their generations, and that they put upon the fringe of the borders a ribband of blue:"],
    ['Numbers','Num','15','039','39',"And it shall be unto you for a fringe, that ye may look upon it, and remember all the commandments of the LORD, and do them; and that ye seek not after your own heart and your own eyes, after which ye use to go a whoring:"],
    ['Numbers','Num','15','040','40',"That ye may remember, and do all my commandments, and be holy unto your God."],
    ['Numbers','Num','15','041','41',"I \emph{am} the LORD your God, which brought you out of the land of Egypt, to be your God: I \emph{am} the LORD your God."]
]

Numbers16 = [
    ['Numbers','Num','16','001','1',"Now Korah, the son of Izhar, the son of Kohath, the son of Levi, and Dathan and Abiram, the sons of Eliab, and On, the son of Peleth, sons of Reuben, took \emph{men}:"],
['Numbers','Num','16','002','2',"And they rose up before Moses, with certain of the children of Israel, two hundred and fifty princes of the assembly, famous in the congregation, men of renown:"],
['Numbers','Num','16','003','3',"And they gathered themselves together against Moses and against Aaron, and said unto them, \emph{Ye} \emph{take} too much upon you, seeing all the congregation \emph{are} holy, every one of them, and the LORD \emph{is} among them: wherefore then lift ye up yourselves above the congregation of the LORD?"],
['Numbers','Num','16','004','4',"And when Moses heard \emph{it}, he fell upon his face:"],
['Numbers','Num','16','005','5',"And he spake unto Korah and unto all his company, saying, Even to morrow the LORD will shew who \emph{are} his, and \emph{who} \emph{is} holy; and will cause \emph{him} to come near unto him: even \emph{him} whom he hath chosen will he cause to come near unto him."],
['Numbers','Num','16','006','6',"This do; Take you censers, Korah, and all his company;"],
['Numbers','Num','16','007','7',"And put fire therein, and put incense in them before the LORD to morrow: and it shall be \emph{that} the man whom the LORD doth choose, he \emph{shall} \emph{be} holy: \emph{ye} \emph{take} too much upon you, ye sons of Levi."],
['Numbers','Num','16','008','8',"And Moses said unto Korah, Hear, I pray you, ye sons of Levi:"],
['Numbers','Num','16','009','9',"\emph{Seemeth} \emph{it} \emph{but} a small thing unto you, that the God of Israel hath separated you from the congregation of Israel, to bring you near to himself to do the service of the tabernacle of the LORD, and to stand before the congregation to minister unto them?"],
['Numbers','Num','16','010','10',"And he hath brought thee near \emph{to} \emph{him}, and all thy brethren the sons of Levi with thee: and seek ye the priesthood also?"],
['Numbers','Num','16','011','11',"For which cause \emph{both} thou and all thy company \emph{are} gathered together against the LORD: and what \emph{is} Aaron, that ye murmur against him?"],
['Numbers','Num','16','012','12',"And Moses sent to call Dathan and Abiram, the sons of Eliab: which said, We will not come up:"],
['Numbers','Num','16','013','13',"\emph{Is} \emph{it} a small thing that thou hast brought us up out of a land that floweth with milk and honey, to kill us in the wilderness, except thou make thyself altogether a prince over us?"],
['Numbers','Num','16','014','14',"Moreover thou hast not brought us into a land that floweth with milk and honey, or given us inheritance of fields and vineyards: wilt thou put out the eyes of these men? we will not come up."],
['Numbers','Num','16','015','15',"And Moses was very wroth, and said unto the LORD, Respect not thou their offering: I have not taken one ass from them, neither have I hurt one of them."],
['Numbers','Num','16','016','16',"And Moses said unto Korah, Be thou and all thy company before the LORD, thou, and they, and Aaron, to morrow:"],
['Numbers','Num','16','017','17',"And take every man his censer, and put incense in them, and bring ye before the LORD every man his censer, two hundred and fifty censers; thou also, and Aaron, each \emph{of} \emph{you} his censer."],
['Numbers','Num','16','018','18',"And they took every man his censer, and put fire in them, and laid incense thereon, and stood in the door of the tabernacle of the congregation with Moses and Aaron."],
['Numbers','Num','16','019','19',"And Korah gathered all the congregation against them unto the door of the tabernacle of the congregation: and the glory of the LORD appeared unto all the congregation."],
['Numbers','Num','16','020','20',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Numbers','Num','16','021','21',"Separate yourselves from among this congregation, that I may consume them in a moment."],
['Numbers','Num','16','022','22',"And they fell upon their faces, and said, O God, the God of the spirits of all flesh, shall one man sin, and wilt thou be wroth with all the congregation?"],
['Numbers','Num','16','023','23',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','16','024','24',"Speak unto the congregation, saying, Get you up from about the tabernacle of Korah, Dathan, and Abiram."],
['Numbers','Num','16','025','25',"And Moses rose up and went unto Dathan and Abiram; and the elders of Israel followed him."],
['Numbers','Num','16','026','26',"And he spake unto the congregation, saying, Depart, I pray you, from the tents of these wicked men, and touch nothing of their's, lest ye be consumed in all their sins."],
['Numbers','Num','16','027','27',"So they gat up from the tabernacle of Korah, Dathan, and Abiram, on every side: and Dathan and Abiram came out, and stood in the door of their tents, and their wives, and their sons, and their little children."],
['Numbers','Num','16','028','28',"And Moses said, Hereby ye shall know that the LORD hath sent me to do all these works; for \emph{I} \emph{have} not \emph{done} \emph{them} of mine own mind."],
['Numbers','Num','16','029','29',"If these men die the common death of all men, or if they be visited after the visitation of all men; \emph{then} the LORD hath not sent me."],
['Numbers','Num','16','030','30',"But if the LORD make a new thing, and the earth open her mouth, and swallow them up, with all that \emph{appertain} unto them, and they go down quick into the pit; then ye shall understand that these men have provoked the LORD."],
['Numbers','Num','16','031','31',"And it came to pass, as he had made an end of speaking all these words, that the ground clave asunder that \emph{was} under them:"],
['Numbers','Num','16','032','32',"And the earth opened her mouth, and swallowed them up, and their houses, and all the men that \emph{appertained} unto Korah, and all \emph{their} goods."],
['Numbers','Num','16','033','33',"They, and all that \emph{appertained} to them, went down alive into the pit, and the earth closed upon them: and they perished from among the congregation."],
['Numbers','Num','16','034','34',"And all Israel that \emph{were} round about them fled at the cry of them: for they said, Lest the earth swallow us up \emph{also}."],
['Numbers','Num','16','035','35',"And there came out a fire from the LORD, and consumed the two hundred and fifty men that offered incense."],
['Numbers','Num','16','036','36',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','16','037','37',"Speak unto Eleazar the son of Aaron the priest, that he take up the censers out of the burning, and scatter thou the fire yonder; for they are hallowed."],
['Numbers','Num','16','038','38',"The censers of these sinners against their own souls, let them make them broad plates \emph{for} a covering of the altar: for they offered them before the LORD, therefore they are hallowed: and they shall be a sign unto the children of Israel."],
['Numbers','Num','16','039','39',"And Eleazar the priest took the brasen censers, wherewith they that were burnt had offered; and they were made broad \emph{plates} \emph{for} a covering of the altar:"],
['Numbers','Num','16','040','40',"\emph{To} \emph{be} a memorial unto the children of Israel, that no stranger, which \emph{is} not of the seed of Aaron, come near to offer incense before the LORD; that he be not as Korah, and as his company: as the LORD said to him by the hand of Moses."],
['Numbers','Num','16','041','41',"But on the morrow all the congregation of the children of Israel murmured against Moses and against Aaron, saying, Ye have killed the people of the LORD."],
['Numbers','Num','16','042','42',"And it came to pass, when the congregation was gathered against Moses and against Aaron, that they looked toward the tabernacle of the congregation: and, behold, the cloud covered it, and the glory of the LORD appeared."],
['Numbers','Num','16','043','43',"And Moses and Aaron came before the tabernacle of the congregation."],
['Numbers','Num','16','044','44',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','16','045','45',"Get you up from among this congregation, that I may consume them as in a moment. And they fell upon their faces."],
['Numbers','Num','16','046','46',"And Moses said unto Aaron, Take a censer, and put fire therein from off the altar, and put on incense, and go quickly unto the congregation, and make an atonement for them: for there is wrath gone out from the LORD; the plague is begun."],
['Numbers','Num','16','047','47',"And Aaron took as Moses commanded, and ran into the midst of the congregation; and, behold, the plague was begun among the people: and he put on incense, and made an atonement for the people."],
['Numbers','Num','16','048','48',"And he stood between the dead and the living; and the plague was stayed."],
['Numbers','Num','16','049','49',"Now they that died in the plague were fourteen thousand and seven hundred, beside them that died about the matter of Korah."],
['Numbers','Num','16','050','50',"And Aaron returned unto Moses unto the door of the tabernacle of the congregation: and the plague was stayed."]
]

Numbers17 = [
    ['Numbers','Num','17','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','17','002','2',"Speak unto the children of Israel, and take of every one of them a rod according to the house of \emph{their} fathers, of all their princes according to the house of their fathers twelve rods: write thou every man's name upon his rod."],
['Numbers','Num','17','003','3',"And thou shalt write Aaron's name upon the rod of Levi: for one rod \emph{shall} \emph{be} for the head of the house of their fathers."],
['Numbers','Num','17','004','4',"And thou shalt lay them up in the tabernacle of the congregation before the testimony, where I will meet with you."],
['Numbers','Num','17','005','5',"And it shall come to pass, \emph{that} the man's rod, whom I shall choose, shall blossom: and I will make to cease from me the murmurings of the children of Israel, whereby they murmur against you."],
['Numbers','Num','17','006','6',"And Moses spake unto the children of Israel, and every one of their princes gave him a rod apiece, for each prince one, according to their fathers' houses, \emph{even} twelve rods: and the rod of Aaron \emph{was} among their rods."],
['Numbers','Num','17','007','7',"And Moses laid up the rods before the LORD in the tabernacle of witness."],
['Numbers','Num','17','008','8',"And it came to pass, that on the morrow Moses went into the tabernacle of witness; and, behold, the rod of Aaron for the house of Levi was budded, and brought forth buds, and bloomed blossoms, and yielded almonds."],
['Numbers','Num','17','009','9',"And Moses brought out all the rods from before the LORD unto all the children of Israel: and they looked, and took every man his rod."],
['Numbers','Num','17','010','10',"And the LORD said unto Moses, Bring Aaron's rod again before the testimony, to be kept for a token against the rebels; and thou shalt quite take away their murmurings from me, that they die not."],
['Numbers','Num','17','011','11',"And Moses did \emph{so}: as the LORD commanded him, so did he."],
['Numbers','Num','17','012','12',"And the children of Israel spake unto Moses, saying, Behold, we die, we perish, we all perish."],
['Numbers','Num','17','013','13',"Whosoever cometh any thing near unto the tabernacle of the LORD shall die: shall we be consumed with dying?"]
]

Numbers18 = [
    ['Numbers','Num','18','001','1',"And the LORD said unto Aaron, Thou and thy sons and thy father's house with thee shall bear the iniquity of the sanctuary: and thou and thy sons with thee shall bear the iniquity of your priesthood."],
['Numbers','Num','18','002','2',"And thy brethren also of the tribe of Levi, the tribe of thy father, bring thou with thee, that they may be joined unto thee, and minister unto thee: but thou and thy sons with thee \emph{shall} \emph{minister} before the tabernacle of witness."],
['Numbers','Num','18','003','3',"And they shall keep thy charge, and the charge of all the tabernacle: only they shall not come nigh the vessels of the sanctuary and the altar, that neither they, nor ye also, die."],
['Numbers','Num','18','004','4',"And they shall be joined unto thee, and keep the charge of the tabernacle of the congregation, for all the service of the tabernacle: and a stranger shall not come nigh unto you."],
['Numbers','Num','18','005','5',"And ye shall keep the charge of the sanctuary, and the charge of the altar: that there be no wrath any more upon the children of Israel."],
['Numbers','Num','18','006','6',"And I, behold, I have taken your brethren the Levites from among the children of Israel: to you \emph{they} \emph{are} given \emph{as} a gift for the LORD, to do the service of the tabernacle of the congregation."],
['Numbers','Num','18','007','7',"Therefore thou and thy sons with thee shall keep your priest's office for every thing of the altar, and within the vail; and ye shall serve: I have given your priest's office \emph{unto} \emph{you} as a service of gift: and the stranger that cometh nigh shall be put to death."],
['Numbers','Num','18','008','8',"And the LORD spake unto Aaron, Behold, I also have given thee the charge of mine heave offerings of all the hallowed things of the children of Israel; unto thee have I given them by reason of the anointing, and to thy sons, by an ordinance for ever."],
['Numbers','Num','18','009','9',"This shall be thine of the most holy things, \emph{reserved} from the fire: every oblation of their's, every meat offering of their's, and every sin offering of their's, and every trespass offering of their's, which they shall render unto me, \emph{shall} \emph{be} most holy for thee and for thy sons."],
['Numbers','Num','18','010','10',"In the most holy \emph{place} shalt thou eat it; every male shall eat it: it shall be holy unto thee."],
['Numbers','Num','18','011','11',"And this \emph{is} thine; the heave offering of their gift, with all the wave offerings of the children of Israel: I have given them unto thee, and to thy sons and to thy daughters with thee, by a statute for ever: every one that is clean in thy house shall eat of it."],
['Numbers','Num','18','012','12',"All the best of the oil, and all the best of the wine, and of the wheat, the firstfruits of them which they shall offer unto the LORD, them have I given thee."],
['Numbers','Num','18','013','13',"\emph{And} whatsoever is first ripe in the land, which they shall bring unto the LORD, shall be thine; every one that is clean in thine house shall eat \emph{of} it."],
['Numbers','Num','18','014','14',"Every thing devoted in Israel shall be thine."],
['Numbers','Num','18','015','15',"Every thing that openeth the matrix in all flesh, which they bring unto the LORD, \emph{whether} \emph{it} \emph{be} of men or beasts, shall be thine: nevertheless the firstborn of man shalt thou surely redeem, and the firstling of unclean beasts shalt thou redeem."],
['Numbers','Num','18','016','16',"And those that are to be redeemed from a month old shalt thou redeem, according to thine estimation, for the money of five shekels, after the shekel of the sanctuary, which \emph{is} twenty gerahs."],
['Numbers','Num','18','017','17',"But the firstling of a cow, or the firstling of a sheep, or the firstling of a goat, thou shalt not redeem; they \emph{are} holy: thou shalt sprinkle their blood upon the altar, and shalt burn their fat \emph{for} an offering made by fire, for a sweet savour unto the LORD."],
['Numbers','Num','18','018','18',"And the flesh of them shall be thine, as the wave breast and as the right shoulder are thine."],
['Numbers','Num','18','019','19',"All the heave offerings of the holy things, which the children of Israel offer unto the LORD, have I given thee, and thy sons and thy daughters with thee, by a statute for ever: it \emph{is} a covenant of salt for ever before the LORD unto thee and to thy seed with thee."],
['Numbers','Num','18','020','20',"And the LORD spake unto Aaron, Thou shalt have no inheritance in their land, neither shalt thou have any part among them: I \emph{am} thy part and thine inheritance among the children of Israel."],
['Numbers','Num','18','021','21',"And, behold, I have given the children of Levi all the tenth in Israel for an inheritance, for their service which they serve, \emph{even} the service of the tabernacle of the congregation."],
['Numbers','Num','18','022','22',"Neither must the children of Israel henceforth come nigh the tabernacle of the congregation, lest they bear sin, and die."],
['Numbers','Num','18','023','23',"But the Levites shall do the service of the tabernacle of the congregation, and they shall bear their iniquity: \emph{it} \emph{shall} \emph{be} a statute for ever throughout your generations, that among the children of Israel they have no inheritance."],
['Numbers','Num','18','024','24',"But the tithes of the children of Israel, which they offer \emph{as} an heave offering unto the LORD, I have given to the Levites to inherit: therefore I have said unto them, Among the children of Israel they shall have no inheritance."],
['Numbers','Num','18','025','25',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','18','026','26',"Thus speak unto the Levites, and say unto them, When ye take of the children of Israel the tithes which I have given you from them for your inheritance, then ye shall offer up an heave offering of it for the LORD, \emph{even} a tenth \emph{part} of the tithe."],
['Numbers','Num','18','027','27',"And \emph{this} your heave offering shall be reckoned unto you, as though \emph{it} \emph{were} the corn of the threshingfloor, and as the fulness of the winepress."],
['Numbers','Num','18','028','28',"Thus ye also shall offer an heave offering unto the LORD of all your tithes, which ye receive of the children of Israel; and ye shall give thereof the LORD'S heave offering to Aaron the priest."],
['Numbers','Num','18','029','29',"Out of all your gifts ye shall offer every heave offering of the LORD, of all the best thereof, \emph{even} the hallowed part thereof out of it."],
['Numbers','Num','18','030','30',"Therefore thou shalt say unto them, When ye have heaved the best thereof from it, then it shall be counted unto the Levites as the increase of the threshingfloor, and as the increase of the winepress."],
['Numbers','Num','18','031','31',"And ye shall eat it in every place, ye and your households: for it \emph{is} your reward for your service in the tabernacle of the congregation."],
['Numbers','Num','18','032','32',"And ye shall bear no sin by reason of it, when ye have heaved from it the best of it: neither shall ye pollute the holy things of the children of Israel, lest ye die."]
]

Numbers19 = [
    ['Numbers','Num','19','001','1',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Numbers','Num','19','002','2',"This \emph{is} the ordinance of the law which the LORD hath commanded, saying, Speak unto the children of Israel, that they bring thee a red heifer without spot, wherein \emph{is} no blemish, \emph{and} upon which never came yoke:"],
['Numbers','Num','19','003','3',"And ye shall give her unto Eleazar the priest, that he may bring her forth without the camp, and \emph{one} shall slay her before his face:"],
['Numbers','Num','19','004','4',"And Eleazar the priest shall take of her blood with his finger, and sprinkle of her blood directly before the tabernacle of the congregation seven times:"],
['Numbers','Num','19','005','5',"And \emph{one} shall burn the heifer in his sight; her skin, and her flesh, and her blood, with her dung, shall he burn:"],
['Numbers','Num','19','006','6',"And the priest shall take cedar wood, and hyssop, and scarlet, and cast \emph{it} into the midst of the burning of the heifer."],
['Numbers','Num','19','007','7',"Then the priest shall wash his clothes, and he shall bathe his flesh in water, and afterward he shall come into the camp, and the priest shall be unclean until the even."],
['Numbers','Num','19','008','8',"And he that burneth her shall wash his clothes in water, and bathe his flesh in water, and shall be unclean until the even."],
['Numbers','Num','19','009','9',"And a man \emph{that} \emph{is} clean shall gather up the ashes of the heifer, and lay \emph{them} up without the camp in a clean place, and it shall be kept for the congregation of the children of Israel for a water of separation: it \emph{is} a purification for sin."],
['Numbers','Num','19','010','10',"And he that gathereth the ashes of the heifer shall wash his clothes, and be unclean until the even: and it shall be unto the children of Israel, and unto the stranger that sojourneth among them, for a statute for ever."],
['Numbers','Num','19','011','11',"He that toucheth the dead body of any man shall be unclean seven days."],
['Numbers','Num','19','012','12',"He shall purify himself with it on the third day, and on the seventh day he shall be clean: but if he purify not himself the third day, then the seventh day he shall not be clean."],
['Numbers','Num','19','013','13',"Whosoever toucheth the dead body of any man that is dead, and purifieth not himself, defileth the tabernacle of the LORD; and that soul shall be cut off from Israel: because the water of separation was not sprinkled upon him, he shall be unclean; his uncleanness \emph{is} yet upon him."],
['Numbers','Num','19','014','14',"This \emph{is} the law, when a man dieth in a tent: all that come into the tent, and all that \emph{is} in the tent, shall be unclean seven days."],
['Numbers','Num','19','015','15',"And every open vessel, which hath no covering bound upon it, \emph{is} unclean."],
['Numbers','Num','19','016','16',"And whosoever toucheth one that is slain with a sword in the open fields, or a dead body, or a bone of a man, or a grave, shall be unclean seven days."],
['Numbers','Num','19','017','17',"And for an unclean \emph{person} they shall take of the ashes of the burnt heifer of purification for sin, and running water shall be put thereto in a vessel:"],
['Numbers','Num','19','018','18',"And a clean person shall take hyssop, and dip \emph{it} in the water, and sprinkle \emph{it} upon the tent, and upon all the vessels, and upon the persons that were there, and upon him that touched a bone, or one slain, or one dead, or a grave:"],
['Numbers','Num','19','019','19',"And the clean \emph{person} shall sprinkle upon the unclean on the third day, and on the seventh day: and on the seventh day he shall purify himself, and wash his clothes, and bathe himself in water, and shall be clean at even."],
['Numbers','Num','19','020','20',"But the man that shall be unclean, and shall not purify himself, that soul shall be cut off from among the congregation, because he hath defiled the sanctuary of the LORD: the water of separation hath not been sprinkled upon him; he \emph{is} unclean."],
['Numbers','Num','19','021','21',"And it shall be a perpetual statute unto them, that he that sprinkleth the water of separation shall wash his clothes; and he that toucheth the water of separation shall be unclean until even."],
['Numbers','Num','19','022','22',"And whatsoever the unclean \emph{person} toucheth shall be unclean; and the soul that toucheth \emph{it} shall be unclean until even."]
]

Numbers20 = [
    ['Numbers','Num','20','001','1',"Then came the children of Israel, \emph{even} the whole congregation, into the desert of Zin in the first month: and the people abode in Kadesh; and Miriam died there, and was buried there."],
['Numbers','Num','20','002','2',"And there was no water for the congregation: and they gathered themselves together against Moses and against Aaron."],
['Numbers','Num','20','003','3',"And the people chode with Moses, and spake, saying, Would God that we had died when our brethren died before the LORD!"],
['Numbers','Num','20','004','4',"And why have ye brought up the congregation of the LORD into this wilderness, that we and our cattle should die there?"],
['Numbers','Num','20','005','5',"And wherefore have ye made us to come up out of Egypt, to bring us in unto this evil place? it \emph{is} no place of seed, or of figs, or of vines, or of pomegranates; neither \emph{is} there any water to drink."],
['Numbers','Num','20','006','6',"And Moses and Aaron went from the presence of the assembly unto the door of the tabernacle of the congregation, and they fell upon their faces: and the glory of the LORD appeared unto them."],
['Numbers','Num','20','007','7',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','20','008','8',"Take the rod, and gather thou the assembly together, thou, and Aaron thy brother, and speak ye unto the rock before their eyes; and it shall give forth his water, and thou shalt bring forth to them water out of the rock: so thou shalt give the congregation and their beasts drink."],
['Numbers','Num','20','009','9',"And Moses took the rod from before the LORD, as he commanded him."],
['Numbers','Num','20','010','10',"And Moses and Aaron gathered the congregation together before the rock, and he said unto them, Hear now, ye rebels; must we fetch you water out of this rock?"],
['Numbers','Num','20','011','11',"And Moses lifted up his hand, and with his rod he smote the rock twice: and the water came out abundantly, and the congregation drank, and their beasts \emph{also}."],
['Numbers','Num','20','012','12',"And the LORD spake unto Moses and Aaron, Because ye believed me not, to sanctify me in the eyes of the children of Israel, therefore ye shall not bring this congregation into the land which I have given them."],
['Numbers','Num','20','013','13',"This \emph{is} the water of Meribah; because the children of Israel strove with the LORD, and he was sanctified in them."],
['Numbers','Num','20','014','14',"And Moses sent messengers from Kadesh unto the king of Edom, Thus saith thy brother Israel, Thou knowest all the travail that hath befallen us:"],
['Numbers','Num','20','015','15',"How our fathers went down into Egypt, and we have dwelt in Egypt a long time; and the Egyptians vexed us, and our fathers:"],
['Numbers','Num','20','016','16',"And when we cried unto the LORD, he heard our voice, and sent an angel, and hath brought us forth out of Egypt: and, behold, we \emph{are} in Kadesh, a city in the uttermost of thy border:"],
['Numbers','Num','20','017','17',"Let us pass, I pray thee, through thy country: we will not pass through the fields, or through the vineyards, neither will we drink \emph{of} the water of the wells: we will go by the king's \emph{high} way, we will not turn to the right hand nor to the left, until we have passed thy borders."],
['Numbers','Num','20','018','18',"And Edom said unto him, Thou shalt not pass by me, lest I come out against thee with the sword."],
['Numbers','Num','20','019','19',"And the children of Israel said unto him, We will go by the high way: and if I and my cattle drink of thy water, then I will pay for it: I will only, without \emph{doing} any thing \emph{else}, go through on my feet."],
['Numbers','Num','20','020','20',"And he said, Thou shalt not go through. And Edom came out against him with much people, and with a strong hand."],
['Numbers','Num','20','021','21',"Thus Edom refused to give Israel passage through his border: wherefore Israel turned away from him."],
['Numbers','Num','20','022','22',"And the children of Israel, \emph{even} the whole congregation, journeyed from Kadesh, and came unto mount Hor."],
['Numbers','Num','20','023','23',"And the LORD spake unto Moses and Aaron in mount Hor, by the coast of the land of Edom, saying,"],
['Numbers','Num','20','024','24',"Aaron shall be gathered unto his people: for he shall not enter into the land which I have given unto the children of Israel, because ye rebelled against my word at the water of Meribah."],
['Numbers','Num','20','025','25',"Take Aaron and Eleazar his son, and bring them up unto mount Hor:"],
['Numbers','Num','20','026','26',"And strip Aaron of his garments, and put them upon Eleazar his son: and Aaron shall be gathered \emph{unto} \emph{his} \emph{people}, and shall die there."],
['Numbers','Num','20','027','27',"And Moses did as the LORD commanded: and they went up into mount Hor in the sight of all the congregation."],
['Numbers','Num','20','028','28',"And Moses stripped Aaron of his garments, and put them upon Eleazar his son; and Aaron died there in the top of the mount: and Moses and Eleazar came down from the mount."],
['Numbers','Num','20','029','29',"And when all the congregation saw that Aaron was dead, they mourned for Aaron thirty days, \emph{even} all the house of Israel."]
]

Numbers21 = [
    ['Numbers','Num','21','001','1',"And \emph{when} king Arad the Canaanite, which dwelt in the south, heard tell that Israel came by the way of the spies; then he fought against Israel, and took \emph{some} of them prisoners."],
['Numbers','Num','21','002','2',"And Israel vowed a vow unto the LORD, and said, If thou wilt indeed deliver this people into my hand, then I will utterly destroy their cities."],
['Numbers','Num','21','003','3',"And the LORD hearkened to the voice of Israel, and delivered up the Canaanites; and they utterly destroyed them and their cities: and he called the name of the place Hormah."],
['Numbers','Num','21','004','4',"And they journeyed from mount Hor by the way of the Red sea, to compass the land of Edom: and the soul of the people was much discouraged because of the way."],
['Numbers','Num','21','005','5',"And the people spake against God, and against Moses, Wherefore have ye brought us up out of Egypt to die in the wilderness? for \emph{there} \emph{is} no bread, neither \emph{is} \emph{there} \emph{any} water; and our soul loatheth this light bread."],
['Numbers','Num','21','006','6',"And the LORD sent fiery serpents among the people, and they bit the people; and much people of Israel died."],
['Numbers','Num','21','007','7',"Therefore the people came to Moses, and said, We have sinned, for we have spoken against the LORD, and against thee; pray unto the LORD, that he take away the serpents from us. And Moses prayed for the people."],
['Numbers','Num','21','008','8',"And the LORD said unto Moses, Make thee a fiery serpent, and set it upon a pole: and it shall come to pass, that every one that is bitten, when he looketh upon it, shall live."],
['Numbers','Num','21','009','9',"And Moses made a serpent of brass, and put it upon a pole, and it came to pass, that if a serpent had bitten any man, when he beheld the serpent of brass, he lived."],
['Numbers','Num','21','010','10',"And the children of Israel set forward, and pitched in Oboth."],
['Numbers','Num','21','011','11',"And they journeyed from Oboth, and pitched at Ije-abarim, in the wilderness which \emph{is} before Moab, toward the sunrising."],
['Numbers','Num','21','012','12',"From thence they removed, and pitched in the valley of Zared."],
['Numbers','Num','21','013','13',"From thence they removed, and pitched on the other side of Arnon, which \emph{is} in the wilderness that cometh out of the coasts of the Amorites: for Arnon \emph{is} the border of Moab, between Moab and the Amorites."],
['Numbers','Num','21','014','14',"Wherefore it is said in the book of the wars of the LORD, What he did in the Red sea, and in the brooks of Arnon,"],
['Numbers','Num','21','015','15',"And at the stream of the brooks that goeth down to the dwelling of Ar, and lieth upon the border of Moab."],
['Numbers','Num','21','016','16',"And from thence \emph{they} \emph{went} to Beer: that \emph{is} the well whereof the LORD spake unto Moses, Gather the people together, and I will give them water."],
['Numbers','Num','21','017','17',"Then Israel sang this song, Spring up, O well; sing ye unto it:"],
['Numbers','Num','21','018','18',"The princes digged the well, the nobles of the people digged it, by \emph{the} \emph{direction} \emph{of} the lawgiver, with their staves. And from the wilderness \emph{they} \emph{went} to Mattanah:"],
['Numbers','Num','21','019','19',"And from Mattanah to Nahaliel: and from Nahaliel to Bamoth:"],
['Numbers','Num','21','020','20',"And from Bamoth \emph{in} the valley, that \emph{is} in the country of Moab, to the top of Pisgah, which looketh toward Jeshimon."],
['Numbers','Num','21','021','21',"And Israel sent messengers unto Sihon king of the Amorites, saying,"],
['Numbers','Num','21','022','22',"Let me pass through thy land: we will not turn into the fields, or into the vineyards; we will not drink \emph{of} the waters of the well: \emph{but} we will go along by the king's \emph{high} way, until we be past thy borders."],
['Numbers','Num','21','023','23',"And Sihon would not suffer Israel to pass through his border: but Sihon gathered all his people together, and went out against Israel into the wilderness: and he came to Jahaz, and fought against Israel."],
['Numbers','Num','21','024','24',"And Israel smote him with the edge of the sword, and possessed his land from Arnon unto Jabbok, even unto the children of Ammon: for the border of the children of Ammon \emph{was} strong."],
['Numbers','Num','21','025','25',"And Israel took all these cities: and Israel dwelt in all the cities of the Amorites, in Heshbon, and in all the villages thereof."],
['Numbers','Num','21','026','26',"For Heshbon \emph{was} the city of Sihon the king of the Amorites, who had fought against the former king of Moab, and taken all his land out of his hand, even unto Arnon."],
['Numbers','Num','21','027','27',"Wherefore they that speak in proverbs say, Come into Heshbon, let the city of Sihon be built and prepared:"],
['Numbers','Num','21','028','28',"For there is a fire gone out of Heshbon, a flame from the city of Sihon: it hath consumed Ar of Moab, \emph{and} the lords of the high places of Arnon."],
['Numbers','Num','21','029','29',"Woe to thee, Moab! thou art undone, O people of Chemosh: he hath given his sons that escaped, and his daughters, into captivity unto Sihon king of the Amorites."],
['Numbers','Num','21','030','30',"We have shot at them; Heshbon is perished even unto Dibon, and we have laid them waste even unto Nophah, which \emph{reacheth} unto Medeba."],
['Numbers','Num','21','031','31',"Thus Israel dwelt in the land of the Amorites."],
['Numbers','Num','21','032','32',"And Moses sent to spy out Jaazer, and they took the villages thereof, and drove out the Amorites that \emph{were} there."],
['Numbers','Num','21','033','33',"And they turned and went up by the way of Bashan: and Og the king of Bashan went out against them, he, and all his people, to the battle at Edrei."],
['Numbers','Num','21','034','34',"And the LORD said unto Moses, Fear him not: for I have delivered him into thy hand, and all his people, and his land; and thou shalt do to him as thou didst unto Sihon king of the Amorites, which dwelt at Heshbon."],
['Numbers','Num','21','035','35',"So they smote him, and his sons, and all his people, until there was none left him alive: and they possessed his land."]
]
Numbers22 = [
    ['Numbers','Num','22','001','1',"And the children of Israel set forward, and pitched in the plains of Moab on this side Jordan \emph{by} Jericho."],
['Numbers','Num','22','002','2',"And Balak the son of Zippor saw all that Israel had done to the Amorites."],
['Numbers','Num','22','003','3',"And Moab was sore afraid of the people, because they \emph{were} many: and Moab was distressed because of the children of Israel."],
['Numbers','Num','22','004','4',"And Moab said unto the elders of Midian, Now shall this company lick up all \emph{that} \emph{are} round about us, as the ox licketh up the grass of the field. And Balak the son of Zippor \emph{was} king of the Moabites at that time."],
['Numbers','Num','22','005','5',"He sent messengers therefore unto Balaam the son of Beor to Pethor, which \emph{is} by the river of the land of the children of his people, to call him, saying, Behold, there is a people come out from Egypt: behold, they cover the face of the earth, and they abide over against me:"],
['Numbers','Num','22','006','6',"Come now therefore, I pray thee, curse me this people; for they \emph{are} too mighty for me: peradventure I shall prevail, \emph{that} we may smite them, and \emph{that} I may drive them out of the land: for I wot that he whom thou blessest \emph{is} blessed, and he whom thou cursest is cursed."],
['Numbers','Num','22','007','7',"And the elders of Moab and the elders of Midian departed with the rewards of divination in their hand; and they came unto Balaam, and spake unto him the words of Balak."],
['Numbers','Num','22','008','8',"And he said unto them, Lodge here this night, and I will bring you word again, as the LORD shall speak unto me: and the princes of Moab abode with Balaam."],
['Numbers','Num','22','009','9',"And God came unto Balaam, and said, What men \emph{are} these with thee?"],
['Numbers','Num','22','010','10',"And Balaam said unto God, Balak the son of Zippor, king of Moab, hath sent unto me, \emph{saying},"],
['Numbers','Num','22','011','11',"Behold, \emph{there} \emph{is} a people come out of Egypt, which covereth the face of the earth: come now, curse me them; peradventure I shall be able to overcome them, and drive them out."],
['Numbers','Num','22','012','12',"And God said unto Balaam, Thou shalt not go with them; thou shalt not curse the people: for they \emph{are} blessed."],
['Numbers','Num','22','013','13',"And Balaam rose up in the morning, and said unto the princes of Balak, Get you into your land: for the LORD refuseth to give me leave to go with you."],
['Numbers','Num','22','014','14',"And the princes of Moab rose up, and they went unto Balak, and said, Balaam refuseth to come with us."],
['Numbers','Num','22','015','15',"And Balak sent yet again princes, more, and more honourable than they."],
['Numbers','Num','22','016','16',"And they came to Balaam, and said to him, Thus saith Balak the son of Zippor, Let nothing, I pray thee, hinder thee from coming unto me:"],
['Numbers','Num','22','017','17',"For I will promote thee unto very great honour, and I will do whatsoever thou sayest unto me: come therefore, I pray thee, curse me this people."],
['Numbers','Num','22','018','18',"And Balaam answered and said unto the servants of Balak, If Balak would give me his house full of silver and gold, I cannot go beyond the word of the LORD my God, to do less or more."],
['Numbers','Num','22','019','19',"Now therefore, I pray you, tarry ye also here this night, that I may know what the LORD will say unto me more."],
['Numbers','Num','22','020','20',"And God came unto Balaam at night, and said unto him, If the men come to call thee, rise up, \emph{and} go with them; but yet the word which I shall say unto thee, that shalt thou do."],
['Numbers','Num','22','021','21',"And Balaam rose up in the morning, and saddled his ass, and went with the princes of Moab."],
['Numbers','Num','22','022','22',"And God's anger was kindled because he went: and the angel of the LORD stood in the way for an adversary against him. Now he was riding upon his ass, and his two servants \emph{were} with him."],
['Numbers','Num','22','023','23',"And the ass saw the angel of the LORD standing in the way, and his sword drawn in his hand: and the ass turned aside out of the way, and went into the field: and Balaam smote the ass, to turn her into the way."],
['Numbers','Num','22','024','24',"But the angel of the LORD stood in a path of the vineyards, a wall \emph{being} on this side, and a wall on that side."],
['Numbers','Num','22','025','25',"And when the ass saw the angel of the LORD, she thrust herself unto the wall, and crushed Balaam's foot against the wall: and he smote her again."],
['Numbers','Num','22','026','26',"And the angel of the LORD went further, and stood in a narrow place, where \emph{was} no way to turn either to the right hand or to the left."],
['Numbers','Num','22','027','27',"And when the ass saw the angel of the LORD, she fell down under Balaam: and Balaam's anger was kindled, and he smote the ass with a staff."],
['Numbers','Num','22','028','28',"And the LORD opened the mouth of the ass, and she said unto Balaam, What have I done unto thee, that thou hast smitten me these three times?"],
['Numbers','Num','22','029','29',"And Balaam said unto the ass, Because thou hast mocked me: I would there were a sword in mine hand, for now would I kill thee."],
['Numbers','Num','22','030','30',"And the ass said unto Balaam, \emph{Am} not I thine ass, upon which thou hast ridden ever since \emph{I} \emph{was} thine unto this day? was I ever wont to do so unto thee? And he said, Nay."],
['Numbers','Num','22','031','31',"Then the LORD opened the eyes of Balaam, and he saw the angel of the LORD standing in the way, and his sword drawn in his hand: and he bowed down his head, and fell flat on his face."],
['Numbers','Num','22','032','32',"And the angel of the LORD said unto him, Wherefore hast thou smitten thine ass these three times? behold, I went out to withstand thee, because \emph{thy} way is perverse before me:"],
['Numbers','Num','22','033','33',"And the ass saw me, and turned from me these three times: unless she had turned from me, surely now also I had slain thee, and saved her alive."],
['Numbers','Num','22','034','34',"And Balaam said unto the angel of the LORD, I have sinned; for I knew not that thou stoodest in the way against me: now therefore, if it displease thee, I will get me back again."],
['Numbers','Num','22','035','35',"And the angel of the LORD said unto Balaam, Go with the men: but only the word that I shall speak unto thee, that thou shalt speak. So Balaam went with the princes of Balak."],
['Numbers','Num','22','036','36',"And when Balak heard that Balaam was come, he went out to meet him unto a city of Moab, which \emph{is} in the border of Arnon, which \emph{is} in the utmost coast."],
['Numbers','Num','22','037','37',"And Balak said unto Balaam, Did I not earnestly send unto thee to call thee? wherefore camest thou not unto me? am I not able indeed to promote thee to honour?"],
['Numbers','Num','22','038','38',"And Balaam said unto Balak, Lo, I am come unto thee: have I now any power at all to say any thing? the word that God putteth in my mouth, that shall I speak."],
['Numbers','Num','22','039','39',"And Balaam went with Balak, and they came unto Kirjath-huzoth."],
['Numbers','Num','22','040','40',"And Balak offered oxen and sheep, and sent to Balaam, and to the princes that \emph{were} with him."],
['Numbers','Num','22','041','41',"And it came to pass on the morrow, that Balak took Balaam, and brought him up into the high places of Baal, that thence he might see the utmost \emph{part} of the people."]        ]

Numbers23 = [
    ['Numbers','Num','23','001','1',"And Balaam said unto Balak, Build me here seven altars, and prepare me here seven oxen and seven rams."],
['Numbers','Num','23','002','2',"And Balak did as Balaam had spoken; and Balak and Balaam offered on \emph{every} altar a bullock and a ram."],
['Numbers','Num','23','003','3',"And Balaam said unto Balak, Stand by thy burnt offering, and I will go: peradventure the LORD will come to meet me: and whatsoever he sheweth me I will tell thee. And he went to an high place."],
['Numbers','Num','23','004','4',"And God met Balaam: and he said unto him, I have prepared seven altars, and I have offered upon \emph{every} altar a bullock and a ram."],
['Numbers','Num','23','005','5',"And the LORD put a word in Balaam's mouth, and said, Return unto Balak, and thus thou shalt speak."],
['Numbers','Num','23','006','6',"And he returned unto him, and, lo, he stood by his burnt sacrifice, he, and all the princes of Moab."],
['Numbers','Num','23','007','7',"And he took up his parable, and said, Balak the king of Moab hath brought me from Aram, out of the mountains of the east, \emph{saying}, Come, curse me Jacob, and come, defy Israel."],
['Numbers','Num','23','008','8',"How shall I curse, whom God hath not cursed? or how shall I defy, \emph{whom} the LORD hath not defied?"],
['Numbers','Num','23','009','9',"For from the top of the rocks I see him, and from the hills I behold him: lo, the people shall dwell alone, and shall not be reckoned among the nations."],
['Numbers','Num','23','010','10',"Who can count the dust of Jacob, and the number of the fourth \emph{part} of Israel? Let me die the death of the righteous, and let my last end be like his!"],
['Numbers','Num','23','011','11',"And Balak said unto Balaam, What hast thou done unto me? I took thee to curse mine enemies, and, behold, thou hast blessed \emph{them} altogether."],
['Numbers','Num','23','012','12',"And he answered and said, Must I not take heed to speak that which the LORD hath put in my mouth?"],
['Numbers','Num','23','013','13',"And Balak said unto him, Come, I pray thee, with me unto another place, from whence thou mayest see them: thou shalt see but the utmost part of them, and shalt not see them all: and curse me them from thence."],
['Numbers','Num','23','014','14',"And he brought him into the field of Zophim, to the top of Pisgah, and built seven altars, and offered a bullock and a ram on \emph{every} altar."],
['Numbers','Num','23','015','15',"And he said unto Balak, Stand here by thy burnt offering, while I meet \emph{the} \emph{LORD} yonder."],
['Numbers','Num','23','016','16',"And the LORD met Balaam, and put a word in his mouth, and said, Go again unto Balak, and say thus."],
['Numbers','Num','23','017','17',"And when he came to him, behold, he stood by his burnt offering, and the princes of Moab with him. And Balak said unto him, What hath the LORD spoken?"],
['Numbers','Num','23','018','18',"And he took up his parable, and said, Rise up, Balak, and hear; hearken unto me, thou son of Zippor:"],
['Numbers','Num','23','019','19',"God \emph{is} not a man, that he should lie; neither the son of man, that he should repent: hath he said, and shall he not do \emph{it}? or hath he spoken, and shall he not make it good?"],
['Numbers','Num','23','020','20',"Behold, I have received \emph{commandment} to bless: and he hath blessed; and I cannot reverse it."],
['Numbers','Num','23','021','21',"He hath not beheld iniquity in Jacob, neither hath he seen perverseness in Israel: the LORD his God \emph{is} with him, and the shout of a king \emph{is} among them."],
['Numbers','Num','23','022','22',"God brought them out of Egypt; he hath as it were the strength of an unicorn."],
['Numbers','Num','23','023','23',"Surely \emph{there} \emph{is} no enchantment against Jacob, neither \emph{is} \emph{there} any divination against Israel: according to this time it shall be said of Jacob and of Israel, What hath God wrought!"],
['Numbers','Num','23','024','24',"Behold, the people shall rise up as a great lion, and lift up himself as a young lion: he shall not lie down until he eat \emph{of} the prey, and drink the blood of the slain."],
['Numbers','Num','23','025','25',"And Balak said unto Balaam, Neither curse them at all, nor bless them at all."],
['Numbers','Num','23','026','26',"But Balaam answered and said unto Balak, Told not I thee, saying, All that the LORD speaketh, that I must do?"],
['Numbers','Num','23','027','27',"And Balak said unto Balaam, Come, I pray thee, I will bring thee unto another place; peradventure it will please God that thou mayest curse me them from thence."],
['Numbers','Num','23','028','28',"And Balak brought Balaam unto the top of Peor, that looketh toward Jeshimon."],
['Numbers','Num','23','029','29',"And Balaam said unto Balak, Build me here seven altars, and prepare me here seven bullocks and seven rams."],
['Numbers','Num','23','030','30',"And Balak did as Balaam had said, and offered a bullock and a ram on \emph{every} altar."]]

Numbers24 = [
    ['Numbers','Num','24','001','1',"And when Balaam saw that it pleased the LORD to bless Israel, he went not, as at other times, to seek for enchantments, but he set his face toward the wilderness."],
['Numbers','Num','24','002','2',"And Balaam lifted up his eyes, and he saw Israel abiding \emph{in} \emph{his} \emph{tents} according to their tribes; and the spirit of God came upon him."],
['Numbers','Num','24','003','3',"And he took up his parable, and said, Balaam the son of Beor hath said, and the man whose eyes are open hath said:"],
['Numbers','Num','24','004','4',"He hath said, which heard the words of God, which saw the vision of the Almighty, falling \emph{into} \emph{a} \emph{trance}, but having his eyes open:"],
['Numbers','Num','24','005','5',"How goodly are thy tents, O Jacob, \emph{and} thy tabernacles, O Israel!"],
['Numbers','Num','24','006','6',"As the valleys are they spread forth, as gardens by the river's side, as the trees of lign aloes which the LORD hath planted, \emph{and} as cedar trees beside the waters."],
['Numbers','Num','24','007','7',"He shall pour the water out of his buckets, and his seed \emph{shall} \emph{be} in many waters, and his king shall be higher than Agag, and his kingdom shall be exalted."],
['Numbers','Num','24','008','8',"God brought him forth out of Egypt; he hath as it were the strength of an unicorn: he shall eat up the nations his enemies, and shall break their bones, and pierce \emph{them} through with his arrows."],
['Numbers','Num','24','009','9',"He couched, he lay down as a lion, and as a great lion: who shall stir him up? Blessed \emph{is} he that blesseth thee, and cursed \emph{is} he that curseth thee."],
['Numbers','Num','24','010','10',"And Balak's anger was kindled against Balaam, and he smote his hands together: and Balak said unto Balaam, I called thee to curse mine enemies, and, behold, thou hast altogether blessed \emph{them} these three times."],
['Numbers','Num','24','011','11',"Therefore now flee thou to thy place: I thought to promote thee unto great honour; but, lo, the LORD hath kept thee back from honour."],
['Numbers','Num','24','012','12',"And Balaam said unto Balak, Spake I not also to thy messengers which thou sentest unto me, saying,"],
['Numbers','Num','24','013','13',"If Balak would give me his house full of silver and gold, I cannot go beyond the commandment of the LORD, to do \emph{either} good or bad of mine own mind; \emph{but} what the LORD saith, that will I speak?"],
['Numbers','Num','24','014','14',"And now, behold, I go unto my people: come \emph{therefore,} \emph{and} I will advertise thee what this people shall do to thy people in the latter days."],
['Numbers','Num','24','015','15',"And he took up his parable, and said, Balaam the son of Beor hath said, and the man whose eyes are open hath said:"],
['Numbers','Num','24','016','16',"He hath said, which heard the words of God, and knew the knowledge of the most High, \emph{which} saw the vision of the Almighty, falling \emph{into} \emph{a} \emph{trance}, but having his eyes open:"],
['Numbers','Num','24','017','17',"I shall see him, but not now: I shall behold him, but not nigh: there shall come a Star out of Jacob, and a Sceptre shall rise out of Israel, and shall smite the corners of Moab, and destroy all the children of Sheth."],
['Numbers','Num','24','018','18',"And Edom shall be a possession, Seir also shall be a possession for his enemies; and Israel shall do valiantly."],
['Numbers','Num','24','019','19',"Out of Jacob shall come he that shall have dominion, and shall destroy him that remaineth of the city."],
['Numbers','Num','24','020','20',"And when he looked on Amalek, he took up his parable, and said, Amalek \emph{was} the first of the nations; but his latter end \emph{shall} \emph{be} that he perish for ever."],
['Numbers','Num','24','021','21',"And he looked on the Kenites, and took up his parable, and said, Strong is thy dwellingplace, and thou puttest thy nest in a rock."],
['Numbers','Num','24','022','22',"Nevertheless the Kenite shall be wasted, until Asshur shall carry thee away captive."],
['Numbers','Num','24','023','23',"And he took up his parable, and said, Alas, who shall live when God doeth this!"],
['Numbers','Num','24','024','24',"And ships \emph{shall} \emph{come} from the coast of Chittim, and shall afflict Asshur, and shall afflict Eber, and he also shall perish for ever."],
['Numbers','Num','24','025','25',"And Balaam rose up, and went and returned to his place: and Balak also went his way."]]

Numbers25 = [
    ['Numbers','Num','25','001','1',"And Israel abode in Shittim, and the people began to commit whoredom with the daughters of Moab."],
['Numbers','Num','25','002','2',"And they called the people unto the sacrifices of their gods: and the people did eat, and bowed down to their gods."],
['Numbers','Num','25','003','3',"And Israel joined himself unto Baal-peor: and the anger of the LORD was kindled against Israel."],
['Numbers','Num','25','004','4',"And the LORD said unto Moses, Take all the heads of the people, and hang them up before the LORD against the sun, that the fierce anger of the LORD may be turned away from Israel."],
['Numbers','Num','25','005','5',"And Moses said unto the judges of Israel, Slay ye every one his men that were joined unto Baal-peor."],
['Numbers','Num','25','006','6',"And, behold, one of the children of Israel came and brought unto his brethren a Midianitish woman in the sight of Moses, and in the sight of all the congregation of the children of Israel, who \emph{were} weeping \emph{before} the door of the tabernacle of the congregation."],
['Numbers','Num','25','007','7',"And when Phinehas, the son of Eleazar, the son of Aaron the priest, saw \emph{it}, he rose up from among the congregation, and took a javelin in his hand;"],
['Numbers','Num','25','008','8',"And he went after the man of Israel into the tent, and thrust both of them through, the man of Israel, and the woman through her belly. So the plague was stayed from the children of Israel."],
['Numbers','Num','25','009','9',"And those that died in the plague were twenty and four thousand."],
['Numbers','Num','25','010','10',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','25','011','11',"Phinehas, the son of Eleazar, the son of Aaron the priest, hath turned my wrath away from the children of Israel, while he was zealous for my sake among them, that I consumed not the children of Israel in my jealousy."],
['Numbers','Num','25','012','12',"Wherefore say, Behold, I give unto him my covenant of peace:"],
['Numbers','Num','25','013','13',"And he shall have it, and his seed after him, \emph{even} the covenant of an everlasting priesthood; because he was zealous for his God, and made an atonement for the children of Israel."],
['Numbers','Num','25','014','14',"Now the name of the Israelite that was slain, \emph{even} that was slain with the Midianitish woman, \emph{was} Zimri, the son of Salu, a prince of a chief house among the Simeonites."],
['Numbers','Num','25','015','15',"And the name of the Midianitish woman that was slain \emph{was} Cozbi, the daughter of Zur; he \emph{was} head over a people, \emph{and} of a chief house in Midian."],
['Numbers','Num','25','016','16',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','25','017','17',"Vex the Midianites, and smite them:"],
['Numbers','Num','25','018','18',"For they vex you with their wiles, wherewith they have beguiled you in the matter of Peor, and in the matter of Cozbi, the daughter of a prince of Midian, their sister, which was slain in the day of the plague for Peor's sake."]]

Numbers26 = [
    ['Numbers','Num','26','001','1',"And it came to pass after the plague, that the LORD spake unto Moses and unto Eleazar the son of Aaron the priest, saying,"],
['Numbers','Num','26','002','2',"Take the sum of all the congregation of the children of Israel, from twenty years old and upward, throughout their fathers' house, all that are able to go to war in Israel."],
['Numbers','Num','26','003','3',"And Moses and Eleazar the priest spake with them in the plains of Moab by Jordan \emph{near} Jericho, saying,"],
['Numbers','Num','26','004','4',"\emph{Take} \emph{the} \emph{sum} \emph{of} \emph{the} \emph{people}, from twenty years old and upward; as the LORD commanded Moses and the children of Israel, which went forth out of the land of Egypt."],
['Numbers','Num','26','005','5',"Reuben, the eldest son of Israel: the children of Reuben; Hanoch, \emph{of} \emph{whom} \emph{cometh} the family of the Hanochites: of Pallu, the family of the Palluites:"],
['Numbers','Num','26','006','6',"Of Hezron, the family of the Hezronites: of Carmi, the family of the Carmites."],
['Numbers','Num','26','007','7',"These \emph{are} the families of the Reubenites: and they that were numbered of them were forty and three thousand and seven hundred and thirty."],
['Numbers','Num','26','008','8',"And the sons of Pallu; Eliab."],
['Numbers','Num','26','009','9',"And the sons of Eliab; Nemuel, and Dathan, and Abiram. This \emph{is} \emph{that} Dathan and Abiram, \emph{which} \emph{were} famous in the congregation, who strove against Moses and against Aaron in the company of Korah, when they strove against the LORD:"],
['Numbers','Num','26','010','10',"And the earth opened her mouth, and swallowed them up together with Korah, when that company died, what time the fire devoured two hundred and fifty men: and they became a sign."],
['Numbers','Num','26','011','11',"Notwithstanding the children of Korah died not."],
['Numbers','Num','26','012','12',"The sons of Simeon after their families: of Nemuel, the family of the Nemuelites: of Jamin, the family of the Jaminites: of Jachin, the family of the Jachinites:"],
['Numbers','Num','26','013','13',"Of Zerah, the family of the Zarhites: of Shaul, the family of the Shaulites."],
['Numbers','Num','26','014','14',"These \emph{are} the families of the Simeonites, twenty and two thousand and two hundred."],
['Numbers','Num','26','015','15',"The children of Gad after their families: of Zephon, the family of the Zephonites: of Haggi, the family of the Haggites: of Shuni, the family of the Shunites:"],
['Numbers','Num','26','016','16',"Of Ozni, the family of the Oznites: of Eri, the family of the Erites:"],
['Numbers','Num','26','017','17',"Of Arod, the family of the Arodites: of Areli, the family of the Arelites."],
['Numbers','Num','26','018','18',"These \emph{are} the families of the children of Gad according to those that were numbered of them, forty thousand and five hundred."],
['Numbers','Num','26','019','19',"The sons of Judah \emph{were} Er and Onan: and Er and Onan died in the land of Canaan."],
['Numbers','Num','26','020','20',"And the sons of Judah after their families were; of Shelah, the family of the Shelanites: of Pharez, the family of the Pharzites: of Zerah, the family of the Zarhites."],
['Numbers','Num','26','021','21',"And the sons of Pharez were; of Hezron, the family of the Hezronites: of Hamul, the family of the Hamulites."],
['Numbers','Num','26','022','22',"These \emph{are} the families of Judah according to those that were numbered of them, threescore and sixteen thousand and five hundred."],
['Numbers','Num','26','023','23',"\emph{Of} the sons of Issachar after their families: \emph{of} Tola, the family of the Tolaites: of Pua, the family of the Punites:"],
['Numbers','Num','26','024','24',"Of Jashub, the family of the Jashubites: of Shimron, the family of the Shimronites."],
['Numbers','Num','26','025','25',"These \emph{are} the families of Issachar according to those that were numbered of them, threescore and four thousand and three hundred."],
['Numbers','Num','26','026','26',"\emph{Of} the sons of Zebulun after their families: of Sered, the family of the Sardites: of Elon, the family of the Elonites: of Jahleel, the family of the Jahleelites."],
['Numbers','Num','26','027','27',"These \emph{are} the families of the Zebulunites according to those that were numbered of them, threescore thousand and five hundred."],
['Numbers','Num','26','028','28',"The sons of Joseph after their families \emph{were} Manasseh and Ephraim."],
['Numbers','Num','26','029','29',"Of the sons of Manasseh: of Machir, the family of the Machirites: and Machir begat Gilead: of Gilead \emph{come} the family of the Gileadites."],
['Numbers','Num','26','030','30',"These \emph{are} the sons of Gilead: \emph{of} Jeezer, the family of the Jeezerites: of Helek, the family of the Helekites:"],
['Numbers','Num','26','031','31',"And \emph{of} Asriel, the family of the Asrielites: and \emph{of} Shechem, the family of the Shechemites:"],
['Numbers','Num','26','032','32',"And \emph{of} Shemida, the family of the Shemidaites: and \emph{of} Hepher, the family of the Hepherites."],
['Numbers','Num','26','033','33',"And Zelophehad the son of Hepher had no sons, but daughters: and the names of the daughters of Zelophehad \emph{were} Mahlah, and Noah, Hoglah, Milcah, and Tirzah."],
['Numbers','Num','26','034','34',"These \emph{are} the families of Manasseh, and those that were numbered of them, fifty and two thousand and seven hundred."],
['Numbers','Num','26','035','35',"These \emph{are} the sons of Ephraim after their families: of Shuthelah, the family of the Shuthalhites: of Becher, the family of the Bachrites: of Tahan, the family of the Tahanites."],
['Numbers','Num','26','036','36',"And these \emph{are} the sons of Shuthelah: of Eran, the family of the Eranites."],
['Numbers','Num','26','037','37',"These \emph{are} the families of the sons of Ephraim according to those that were numbered of them, thirty and two thousand and five hundred. These \emph{are} the sons of Joseph after their families."],
['Numbers','Num','26','038','38',"The sons of Benjamin after their families: of Bela, the family of the Belaites: of Ashbel, the family of the Ashbelites: of Ahiram, the family of the Ahiramites:"],
['Numbers','Num','26','039','39',"Of Shupham, the family of the Shuphamites: of Hupham, the family of the Huphamites."],
['Numbers','Num','26','040','40',"And the sons of Bela were Ard and Naaman: \emph{of} \emph{Ard}, the family of the Ardites: \emph{and} of Naaman, the family of the Naamites."],
['Numbers','Num','26','041','41',"These \emph{are} the sons of Benjamin after their families: and they that were numbered of them \emph{were} forty and five thousand and six hundred."],
['Numbers','Num','26','042','42',"These \emph{are} the sons of Dan after their families: of Shuham, the family of the Shuhamites. These \emph{are} the families of Dan after their families."],
['Numbers','Num','26','043','43',"All the families of the Shuhamites, according to those that were numbered of them, \emph{were} threescore and four thousand and four hundred."],
['Numbers','Num','26','044','44',"\emph{Of} the children of Asher after their families: of Jimna, the family of the Jimnites: of Jesui, the family of the Jesuites: of Beriah, the family of the Beriites."],
['Numbers','Num','26','045','45',"Of the sons of Beriah: of Heber, the family of the Heberites: of Malchiel, the family of the Malchielites."],
['Numbers','Num','26','046','46',"And the name of the daughter of Asher \emph{was} Sarah."],
['Numbers','Num','26','047','47',"These \emph{are} the families of the sons of Asher according to those that were numbered of them; \emph{who} \emph{were} fifty and three thousand and four hundred."],
['Numbers','Num','26','048','48',"\emph{Of} the sons of Naphtali after their families: of Jahzeel, the family of the Jahzeelites: of Guni, the family of the Gunites:"],
['Numbers','Num','26','049','49',"Of Jezer, the family of the Jezerites: of Shillem, the family of the Shillemites."],
['Numbers','Num','26','050','50',"These \emph{are} the families of Naphtali according to their families: and they that were numbered of them \emph{were} forty and five thousand and four hundred."],
['Numbers','Num','26','051','51',"These \emph{were} the numbered of the children of Israel, six hundred thousand and a thousand seven hundred and thirty."],
['Numbers','Num','26','052','52',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','26','053','53',"Unto these the land shall be divided for an inheritance according to the number of names."],
['Numbers','Num','26','054','54',"To many thou shalt give the more inheritance, and to few thou shalt give the less inheritance: to every one shall his inheritance be given according to those that were numbered of him."],
['Numbers','Num','26','055','55',"Notwithstanding the land shall be divided by lot: according to the names of the tribes of their fathers they shall inherit."],
['Numbers','Num','26','056','56',"According to the lot shall the possession thereof be divided between many and few."],
['Numbers','Num','26','057','57',"And these \emph{are} they that were numbered of the Levites after their families: of Gershon, the family of the Gershonites: of Kohath, the family of the Kohathites: of Merari, the family of the Merarites."],
['Numbers','Num','26','058','58',"These \emph{are} the families of the Levites: the family of the Libnites, the family of the Hebronites, the family of the Mahlites, the family of the Mushites, the family of the Korathites. And Kohath begat Amram."],
['Numbers','Num','26','059','59',"And the name of Amram's wife \emph{was} Jochebed, the daughter of Levi, whom \emph{her} \emph{mother} bare to Levi in Egypt: and she bare unto Amram Aaron and Moses, and Miriam their sister."],
['Numbers','Num','26','060','60',"And unto Aaron was born Nadab, and Abihu, Eleazar, and Ithamar."],
['Numbers','Num','26','061','61',"And Nadab and Abihu died, when they offered strange fire before the LORD."],
['Numbers','Num','26','062','62',"And those that were numbered of them were twenty and three thousand, all males from a month old and upward: for they were not numbered among the children of Israel, because there was no inheritance given them among the children of Israel."],
['Numbers','Num','26','063','63',"These \emph{are} they that were numbered by Moses and Eleazar the priest, who numbered the children of Israel in the plains of Moab by Jordan \emph{near} Jericho."],
['Numbers','Num','26','064','64',"But among these there was not a man of them whom Moses and Aaron the priest numbered, when they numbered the children of Israel in the wilderness of Sinai."],
['Numbers','Num','26','065','65',"For the LORD had said of them, They shall surely die in the wilderness. And there was not left a man of them, save Caleb the son of Jephunneh, and Joshua the son of Nun."]]

Numbers27 = [
    ['Numbers','Num','27','001','1',"Then came the daughters of Zelophehad, the son of Hepher, the son of Gilead, the son of Machir, the son of Manasseh, of the families of Manasseh the son of Joseph: and these \emph{are} the names of his daughters; Mahlah, Noah, and Hoglah, and Milcah, and Tirzah."],
['Numbers','Num','27','002','2',"And they stood before Moses, and before Eleazar the priest, and before the princes and all the congregation, \emph{by} the door of the tabernacle of the congregation, saying,"],
['Numbers','Num','27','003','3',"Our father died in the wilderness, and he was not in the company of them that gathered themselves together against the LORD in the company of Korah; but died in his own sin, and had no sons."],
['Numbers','Num','27','004','4',"Why should the name of our father be done away from among his family, because he hath no son? Give unto us \emph{therefore} a possession among the brethren of our father."],
['Numbers','Num','27','005','5',"And Moses brought their cause before the LORD."],
['Numbers','Num','27','006','6',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','27','007','7',"The daughters of Zelophehad speak right: thou shalt surely give them a possession of an inheritance among their father's brethren; and thou shalt cause the inheritance of their father to pass unto them."],
['Numbers','Num','27','008','8',"And thou shalt speak unto the children of Israel, saying, If a man die, and have no son, then ye shall cause his inheritance to pass unto his daughter."],
['Numbers','Num','27','009','9',"And if he have no daughter, then ye shall give his inheritance unto his brethren."],
['Numbers','Num','27','010','10',"And if he have no brethren, then ye shall give his inheritance unto his father's brethren."],
['Numbers','Num','27','011','11',"And if his father have no brethren, then ye shall give his inheritance unto his kinsman that is next to him of his family, and he shall possess it: and it shall be unto the children of Israel a statute of judgment, as the LORD commanded Moses."],
['Numbers','Num','27','012','12',"And the LORD said unto Moses, Get thee up into this mount Abarim, and see the land which I have given unto the children of Israel."],
['Numbers','Num','27','013','13',"And when thou hast seen it, thou also shalt be gathered unto thy people, as Aaron thy brother was gathered."],
['Numbers','Num','27','014','14',"For ye rebelled against my commandment in the desert of Zin, in the strife of the congregation, to sanctify me at the water before their eyes: that \emph{is} the water of Meribah in Kadesh in the wilderness of Zin."],
['Numbers','Num','27','015','15',"And Moses spake unto the LORD, saying,"],
['Numbers','Num','27','016','16',"Let the LORD, the God of the spirits of all flesh, set a man over the congregation,"],
['Numbers','Num','27','017','17',"Which may go out before them, and which may go in before them, and which may lead them out, and which may bring them in; that the congregation of the LORD be not as sheep which have no shepherd."],
['Numbers','Num','27','018','18',"And the LORD said unto Moses, Take thee Joshua the son of Nun, a man in whom \emph{is} the spirit, and lay thine hand upon him;"],
['Numbers','Num','27','019','19',"And set him before Eleazar the priest, and before all the congregation; and give him a charge in their sight."],
['Numbers','Num','27','020','20',"And thou shalt put \emph{some} of thine honour upon him, that all the congregation of the children of Israel may be obedient."],
['Numbers','Num','27','021','21',"And he shall stand before Eleazar the priest, who shall ask \emph{counsel} for him after the judgment of Urim before the LORD: at his word shall they go out, and at his word they shall come in, \emph{both} he, and all the children of Israel with him, even all the congregation."],
['Numbers','Num','27','022','22',"And Moses did as the LORD commanded him: and he took Joshua, and set him before Eleazar the priest, and before all the congregation:"],
['Numbers','Num','27','023','23',"And he laid his hands upon him, and gave him a charge, as the LORD commanded by the hand of Moses."]]

Numbers28 = [
    ['Numbers','Num','28','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','28','002','2',"Command the children of Israel, and say unto them, My offering, \emph{and} my bread for my sacrifices made by fire, \emph{for} a sweet savour unto me, shall ye observe to offer unto me in their due season."],
['Numbers','Num','28','003','3',"And thou shalt say unto them, This \emph{is} the offering made by fire which ye shall offer unto the LORD; two lambs of the first year without spot day by day, \emph{for} a continual burnt offering."],
['Numbers','Num','28','004','4',"The one lamb shalt thou offer in the morning, and the other lamb shalt thou offer at even;"],
['Numbers','Num','28','005','5',"And a tenth \emph{part} of an ephah of flour for a meat offering, mingled with the fourth \emph{part} of an hin of beaten oil."],
['Numbers','Num','28','006','6',"\emph{It} \emph{is} a continual burnt offering, which was ordained in mount Sinai for a sweet savour, a sacrifice made by fire unto the LORD."],
['Numbers','Num','28','007','7',"And the drink offering thereof \emph{shall} \emph{be} the fourth \emph{part} of an hin for the one lamb: in the holy \emph{place} shalt thou cause the strong wine to be poured unto the LORD \emph{for} a drink offering."],
['Numbers','Num','28','008','8',"And the other lamb shalt thou offer at even: as the meat offering of the morning, and as the drink offering thereof, thou shalt offer \emph{it}, a sacrifice made by fire, of a sweet savour unto the LORD."],
['Numbers','Num','28','009','9',"And on the sabbath day two lambs of the first year without spot, and two tenth deals of flour \emph{for} a meat offering, mingled with oil, and the drink offering thereof:"],
['Numbers','Num','28','010','10',"\emph{This} \emph{is} the burnt offering of every sabbath, beside the continual burnt offering, and his drink offering."],
['Numbers','Num','28','011','11',"And in the beginnings of your months ye shall offer a burnt offering unto the LORD; two young bullocks, and one ram, seven lambs of the first year without spot;"],
['Numbers','Num','28','012','12',"And three tenth deals of flour \emph{for} a meat offering, mingled with oil, for one bullock; and two tenth deals of flour \emph{for} a meat offering, mingled with oil, for one ram;"],
['Numbers','Num','28','013','13',"And a several tenth deal of flour mingled with oil \emph{for} a meat offering unto one lamb; \emph{for} a burnt offering of a sweet savour, a sacrifice made by fire unto the LORD."],
['Numbers','Num','28','014','14',"And their drink offerings shall be half an hin of wine unto a bullock, and the third \emph{part} of an hin unto a ram, and a fourth \emph{part} of an hin unto a lamb: this \emph{is} the burnt offering of every month throughout the months of the year."],
['Numbers','Num','28','015','15',"And one kid of the goats for a sin offering unto the LORD shall be offered, beside the continual burnt offering, and his drink offering."],
['Numbers','Num','28','016','16',"And in the fourteenth day of the first month \emph{is} the passover of the LORD."],
['Numbers','Num','28','017','17',"And in the fifteenth day of this month \emph{is} the feast: seven days shall unleavened bread be eaten."],
['Numbers','Num','28','018','18',"In the first day \emph{shall} \emph{be} an holy convocation; ye shall do no manner of servile work \emph{therein}:"],
['Numbers','Num','28','019','19',"But ye shall offer a sacrifice made by fire \emph{for} a burnt offering unto the LORD; two young bullocks, and one ram, and seven lambs of the first year: they shall be unto you without blemish:"],
['Numbers','Num','28','020','20',"And their meat offering \emph{shall} \emph{be} \emph{of} flour mingled with oil: three tenth deals shall ye offer for a bullock, and two tenth deals for a ram;"],
['Numbers','Num','28','021','21',"A several tenth deal shalt thou offer for every lamb, throughout the seven lambs:"],
['Numbers','Num','28','022','22',"And one goat \emph{for} a sin offering, to make an atonement for you."],
['Numbers','Num','28','023','23',"Ye shall offer these beside the burnt offering in the morning, which \emph{is} for a continual burnt offering."],
['Numbers','Num','28','024','24',"After this manner ye shall offer daily, throughout the seven days, the meat of the sacrifice made by fire, of a sweet savour unto the LORD: it shall be offered beside the continual burnt offering, and his drink offering."],
['Numbers','Num','28','025','25',"And on the seventh day ye shall have an holy convocation; ye shall do no servile work."],
['Numbers','Num','28','026','26',"Also in the day of the firstfruits, when ye bring a new meat offering unto the LORD, after your weeks \emph{be} \emph{out}, ye shall have an holy convocation; ye shall do no servile work:"],
['Numbers','Num','28','027','27',"But ye shall offer the burnt offering for a sweet savour unto the LORD; two young bullocks, one ram, seven lambs of the first year;"],
['Numbers','Num','28','028','28',"And their meat offering of flour mingled with oil, three tenth deals unto one bullock, two tenth deals unto one ram,"],
['Numbers','Num','28','029','29',"A several tenth deal unto one lamb, throughout the seven lambs;"],
['Numbers','Num','28','030','30',"\emph{And} one kid of the goats, to make an atonement for you."],
['Numbers','Num','28','031','31',"Ye shall offer \emph{them} beside the continual burnt offering, and his meat offering, (they shall be unto you without blemish) and their drink offerings."]]

Numbers29 = [
    ['Numbers','Num','29','001','1',"And in the seventh month, on the first \emph{day} of the month, ye shall have an holy convocation; ye shall do no servile work: it is a day of blowing the trumpets unto you."],
['Numbers','Num','29','002','2',"And ye shall offer a burnt offering for a sweet savour unto the LORD; one young bullock, one ram, \emph{and} seven lambs of the first year without blemish:"],
['Numbers','Num','29','003','3',"And their meat offering \emph{shall} \emph{be} \emph{of} flour mingled with oil, three tenth deals for a bullock, \emph{and} two tenth deals for a ram,"],
['Numbers','Num','29','004','4',"And one tenth deal for one lamb, throughout the seven lambs:"],
['Numbers','Num','29','005','5',"And one kid of the goats \emph{for} a sin offering, to make an atonement for you:"],
['Numbers','Num','29','006','6',"Beside the burnt offering of the month, and his meat offering, and the daily burnt offering, and his meat offering, and their drink offerings, according unto their manner, for a sweet savour, a sacrifice made by fire unto the LORD."],
['Numbers','Num','29','007','7',"And ye shall have on the tenth \emph{day} of this seventh month an holy convocation; and ye shall afflict your souls: ye shall not do any work \emph{therein}:"],
['Numbers','Num','29','008','8',"But ye shall offer a burnt offering unto the LORD \emph{for} a sweet savour; one young bullock, one ram, \emph{and} seven lambs of the first year; they shall be unto you without blemish:"],
['Numbers','Num','29','009','9',"And their meat offering \emph{shall} \emph{be} \emph{of} flour mingled with oil, three tenth deals to a bullock, \emph{and} two tenth deals to one ram,"],
['Numbers','Num','29','010','10',"A several tenth deal for one lamb, throughout the seven lambs:"],
['Numbers','Num','29','011','11',"One kid of the goats \emph{for} a sin offering; beside the sin offering of atonement, and the continual burnt offering, and the meat offering of it, and their drink offerings."],
['Numbers','Num','29','012','12',"And on the fifteenth day of the seventh month ye shall have an holy convocation; ye shall do no servile work, and ye shall keep a feast unto the LORD seven days:"],
['Numbers','Num','29','013','13',"And ye shall offer a burnt offering, a sacrifice made by fire, of a sweet savour unto the LORD; thirteen young bullocks, two rams, \emph{and} fourteen lambs of the first year; they shall be without blemish:"],
['Numbers','Num','29','014','14',"And their meat offering \emph{shall} \emph{be} \emph{of} flour mingled with oil, three tenth deals unto every bullock of the thirteen bullocks, two tenth deals to each ram of the two rams,"],
['Numbers','Num','29','015','15',"And a several tenth deal to each lamb of the fourteen lambs:"],
['Numbers','Num','29','016','16',"And one kid of the goats \emph{for} a sin offering; beside the continual burnt offering, his meat offering, and his drink offering."],
['Numbers','Num','29','017','17',"And on the second day \emph{ye} \emph{shall} \emph{offer} twelve young bullocks, two rams, fourteen lambs of the first year without spot:"],
['Numbers','Num','29','018','18',"And their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','019','19',"And one kid of the goats \emph{for} a sin offering; beside the continual burnt offering, and the meat offering thereof, and their drink offerings."],
['Numbers','Num','29','020','20',"And on the third day eleven bullocks, two rams, fourteen lambs of the first year without blemish;"],
['Numbers','Num','29','021','21',"And their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','022','22',"And one goat \emph{for} a sin offering; beside the continual burnt offering, and his meat offering, and his drink offering."],
['Numbers','Num','29','023','23',"And on the fourth day ten bullocks, two rams, \emph{and} fourteen lambs of the first year without blemish:"],
['Numbers','Num','29','024','24',"Their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','025','25',"And one kid of the goats \emph{for} a sin offering; beside the continual burnt offering, his meat offering, and his drink offering."],
['Numbers','Num','29','026','26',"And on the fifth day nine bullocks, two rams, \emph{and} fourteen lambs of the first year without spot:"],
['Numbers','Num','29','027','27',"And their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','028','28',"And one goat \emph{for} a sin offering; beside the continual burnt offering, and his meat offering, and his drink offering."],
['Numbers','Num','29','029','29',"And on the sixth day eight bullocks, two rams, \emph{and} fourteen lambs of the first year without blemish:"],
['Numbers','Num','29','030','30',"And their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','031','31',"And one goat \emph{for} a sin offering; beside the continual burnt offering, his meat offering, and his drink offering."],
['Numbers','Num','29','032','32',"And on the seventh day seven bullocks, two rams, \emph{and} fourteen lambs of the first year without blemish:"],
['Numbers','Num','29','033','33',"And their meat offering and their drink offerings for the bullocks, for the rams, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','034','34',"And one goat \emph{for} a sin offering; beside the continual burnt offering, his meat offering, and his drink offering."],
['Numbers','Num','29','035','35',"On the eighth day ye shall have a solemn assembly: ye shall do no servile work \emph{therein}:"],
['Numbers','Num','29','036','36',"But ye shall offer a burnt offering, a sacrifice made by fire, of a sweet savour unto the LORD: one bullock, one ram, seven lambs of the first year without blemish:"],
['Numbers','Num','29','037','37',"Their meat offering and their drink offerings for the bullock, for the ram, and for the lambs, \emph{shall} \emph{be} according to their number, after the manner:"],
['Numbers','Num','29','038','38',"And one goat \emph{for} a sin offering; beside the continual burnt offering, and his meat offering, and his drink offering."],
['Numbers','Num','29','039','39',"These \emph{things} ye shall do unto the LORD in your set feasts, beside your vows, and your freewill offerings, for your burnt offerings, and for your meat offerings, and for your drink offerings, and for your peace offerings."],
['Numbers','Num','29','040','40',"And Moses told the children of Israel according to all that the LORD commanded Moses."]]

Numbers30 = [
    ['Numbers','Num','30','001','1',"And Moses spake unto the heads of the tribes concerning the children of Israel, saying, This \emph{is} the thing which the LORD hath commanded."],
['Numbers','Num','30','002','2',"If a man vow a vow unto the LORD, or swear an oath to bind his soul with a bond; he shall not break his word, he shall do according to all that proceedeth out of his mouth."],
['Numbers','Num','30','003','3',"If a woman also vow a vow unto the LORD, and bind herself by a bond, \emph{being} in her father's house in her youth;"],
['Numbers','Num','30','004','4',"And her father hear her vow, and her bond wherewith she hath bound her soul, and her father shall hold his peace at her: then all her vows shall stand, and every bond wherewith she hath bound her soul shall stand."],
['Numbers','Num','30','005','5',"But if her father disallow her in the day that he heareth; not any of her vows, or of her bonds wherewith she hath bound her soul, shall stand: and the LORD shall forgive her, because her father disallowed her."],
['Numbers','Num','30','006','6',"And if she had at all an husband, when she vowed, or uttered ought out of her lips, wherewith she bound her soul;"],
['Numbers','Num','30','007','7',"And her husband heard \emph{it,} and held his peace at her in the day that he heard \emph{it}: then her vows shall stand, and her bonds wherewith she bound her soul shall stand."],
['Numbers','Num','30','008','8',"But if her husband disallowed her on the day that he heard \emph{it}; then he shall make her vow which she vowed, and that which she uttered with her lips, wherewith she bound her soul, of none effect: and the LORD shall forgive her."],
['Numbers','Num','30','009','9',"But every vow of a widow, and of her that is divorced, wherewith they have bound their souls, shall stand against her."],
['Numbers','Num','30','010','10',"And if she vowed in her husband's house, or bound her soul by a bond with an oath;"],
['Numbers','Num','30','011','11',"And her husband heard \emph{it}, and held his peace at her, \emph{and} disallowed her not: then all her vows shall stand, and every bond wherewith she bound her soul shall stand."],
['Numbers','Num','30','012','12',"But if her husband hath utterly made them void on the day he heard \emph{them;} \emph{then} whatsoever proceeded out of her lips concerning her vows, or concerning the bond of her soul, shall not stand: her husband hath made them void; and the LORD shall forgive her."],
['Numbers','Num','30','013','13',"Every vow, and every binding oath to afflict the soul, her husband may establish it, or her husband may make it void."],
['Numbers','Num','30','014','14',"But if her husband altogether hold his peace at her from day to day; then he establisheth all her vows, or all her bonds, which \emph{are} upon her: he confirmeth them, because he held his peace at her in the day that he heard \emph{them}."],
['Numbers','Num','30','015','15',"But if he shall any ways make them void after that he hath heard \emph{them}; then he shall bear her iniquity."],
['Numbers','Num','30','016','16',"These \emph{are} the statutes, which the LORD commanded Moses, between a man and his wife, between the father and his daughter, \emph{being} \emph{yet} in her youth in her father's house."]]


Numbers31 = [
    ['Numbers','Num','31','001','1',"And the LORD spake unto Moses, saying,"],
    ['Numbers','Num','31','002','2',"Avenge the children of Israel of the Midianites: afterward shalt thou be gathered unto thy people."],
    ['Numbers','Num','31','003','3',"And Moses spake unto the people, saying, Arm some of yourselves unto the war, and let them go against the Midianites, and avenge the LORD of Midian."],
    ['Numbers','Num','31','004','4',"Of every tribe a thousand, throughout all the tribes of Israel, shall ye send to the war."],
    ['Numbers','Num','31','005','5',"So there were delivered out of the thousands of Israel, a thousand of \emph{every} tribe, twelve thousand armed for war."],
    ['Numbers','Num','31','006','6',"And Moses sent them to the war, a thousand of \emph{every} tribe, them and Phinehas the son of Eleazar the priest, to the war, with the holy instruments, and the trumpets to blow in his hand."],
     ['Numbers','Num','31','007','7',"And they warred against the Midianites, as the LORD commanded Moses; and they slew all the males."],
['Numbers','Num','31','008','8',"And they slew the kings of Midian, beside the rest of them that were slain; \emph{namely}, Evi, and Rekem, and Zur, and Hur, and Reba, five kings of Midian: Balaam also the son of Beor they slew with the sword."],
['Numbers','Num','31','009','9',"And the children of Israel took \emph{all} the women of Midian captives, and their little ones, and took the spoil of all their cattle, and all their flocks, and all their goods."],
['Numbers','Num','31','010','10',"And they burnt all their cities wherein they dwelt, and all their goodly castles, with fire."],
['Numbers','Num','31','011','11',"And they took all the spoil, and all the prey, \emph{both} of men and of beasts."],
['Numbers','Num','31','012','12',"And they brought the captives, and the prey, and the spoil, unto Moses, and Eleazar the priest, and unto the congregation of the children of Israel, unto the camp at the plains of Moab, which \emph{are} by Jordan \emph{near} Jericho."],
['Numbers','Num','31','013','13',"And Moses, and Eleazar the priest, and all the princes of the congregation, went forth to meet them without the camp."],
['Numbers','Num','31','014','14',"And Moses was wroth with the officers of the host, \emph{with} the captains over thousands, and captains over hundreds, which came from the battle."],
['Numbers','Num','31','015','15',"And Moses said unto them, Have ye saved all the women alive?"],
['Numbers','Num','31','016','16',"Behold, these caused the children of Israel, through the counsel of Balaam, to commit trespass against the LORD in the matter of Peor, and there was a plague among the congregation of the LORD."],
['Numbers','Num','31','017','17',"Now therefore kill every male among the little ones, and kill every woman that hath known man by lying with him."],
['Numbers','Num','31','018','18',"But all the women children, that have not known a man by lying with him, keep alive for yourselves."],
['Numbers','Num','31','019','19',"And do ye abide without the camp seven days: whosoever hath killed any person, and whosoever hath touched any slain, purify \emph{both} yourselves and your captives on the third day, and on the seventh day."],
['Numbers','Num','31','020','20',"And purify all \emph{your} raiment, and all that is made of skins, and all work of goats' \emph{hair}, and all things made of wood."],
['Numbers','Num','31','021','21',"And Eleazar the priest said unto the men of war which went to the battle, This \emph{is} the ordinance of the law which the LORD commanded Moses;"],
['Numbers','Num','31','022','22',"Only the gold, and the silver, the brass, the iron, the tin, and the lead,"],
['Numbers','Num','31','023','23',"Every thing that may abide the fire, ye shall make \emph{it} go through the fire, and it shall be clean: nevertheless it shall be purified with the water of separation: and all that abideth not the fire ye shall make go through the water."],
['Numbers','Num','31','024','24',"And ye shall wash your clothes on the seventh day, and ye shall be clean, and afterward ye shall come into the camp."],
['Numbers','Num','31','025','25',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','31','026','26',"Take the sum of the prey that was taken, \emph{both} of man and of beast, thou, and Eleazar the priest, and the chief fathers of the congregation:"],
['Numbers','Num','31','027','27',"And divide the prey into two parts; between them that took the war upon them, who went out to battle, and between all the congregation:"],
['Numbers','Num','31','028','28',"And levy a tribute unto the LORD of the men of war which went out to battle: one soul of five hundred, \emph{both} of the persons, and of the beeves, and of the asses, and of the sheep:"],
['Numbers','Num','31','029','29',"Take \emph{it} of their half, and give it unto Eleazar the priest, \emph{for} an heave offering of the LORD."],
['Numbers','Num','31','030','30',"And of the children of Israel's half, thou shalt take one portion of fifty, of the persons, of the beeves, of the asses, and of the flocks, of all manner of beasts, and give them unto the Levites, which keep the charge of the tabernacle of the LORD."],
['Numbers','Num','31','031','31',"And Moses and Eleazar the priest did as the LORD commanded Moses."],
['Numbers','Num','31','032','32',"And the booty, \emph{being} the rest of the prey which the men of war had caught, was six hundred thousand and seventy thousand and five thousand sheep,"],
['Numbers','Num','31','033','33',"And threescore and twelve thousand beeves,"],
['Numbers','Num','31','034','34',"And threescore and one thousand asses,"],
['Numbers','Num','31','035','35',"And thirty and two thousand persons in all, of women that had not known man by lying with him."],
['Numbers','Num','31','036','36',"And the half, \emph{which} \emph{was} the portion of them that went out to war, was in number three hundred thousand and seven and thirty thousand and five hundred sheep:"],
['Numbers','Num','31','037','37',"And the LORD'S tribute of the sheep was six hundred and threescore and fifteen."],
['Numbers','Num','31','038','38',"And the beeves \emph{were} thirty and six thousand; of which the LORD'S tribute \emph{was} threescore and twelve."],
['Numbers','Num','31','039','39',"And the asses \emph{were} thirty thousand and five hundred; of which the LORD'S tribute \emph{was} threescore and one."],
['Numbers','Num','31','040','40',"And the persons \emph{were} sixteen thousand; of which the LORD'S tribute \emph{was} thirty and two persons."],
['Numbers','Num','31','041','41',"And Moses gave the tribute, \emph{which} \emph{was} the LORD'S heave offering, unto Eleazar the priest, as the LORD commanded Moses."],
['Numbers','Num','31','042','42',"And of the children of Israel's half, which Moses divided from the men that warred,"],
['Numbers','Num','31','043','43',"(Now the half \emph{that} \emph{pertained} \emph{unto} the congregation was three hundred thousand and thirty thousand \emph{and} seven thousand and five hundred sheep,"],
['Numbers','Num','31','044','44',"And thirty and six thousand beeves,"],
['Numbers','Num','31','045','45',"And thirty thousand asses and five hundred,"],
['Numbers','Num','31','046','46',"And sixteen thousand persons;)"],
['Numbers','Num','31','047','47',"Even of the children of Israel's half, Moses took one portion of fifty, \emph{both} of man and of beast, and gave them unto the Levites, which kept the charge of the tabernacle of the LORD; as the LORD commanded Moses."],
['Numbers','Num','31','048','48',"And the officers which \emph{were} over thousands of the host, the captains of thousands, and captains of hundreds, came near unto Moses:"],
['Numbers','Num','31','049','49',"And they said unto Moses, Thy servants have taken the sum of the men of war which \emph{are} under our charge, and there lacketh not one man of us."],
['Numbers','Num','31','050','50',"We have therefore brought an oblation for the LORD, what every man hath gotten, of jewels of gold, chains, and bracelets, rings, earrings, and tablets, to make an atonement for our souls before the LORD."],
['Numbers','Num','31','051','51',"And Moses and Eleazar the priest took the gold of them, \emph{even} all wrought jewels."],
['Numbers','Num','31','052','52',"And all the gold of the offering that they offered up to the LORD, of the captains of thousands, and of the captains of hundreds, was sixteen thousand seven hundred and fifty shekels."],
['Numbers','Num','31','053','53',"\emph{(For} the men of war had taken spoil, every man for himself.)"],
['Numbers','Num','31','054','54',"And Moses and Eleazar the priest took the gold of the captains of thousands and of hundreds, and brought it into the tabernacle of the congregation, \emph{for} a memorial for the children of Israel before the LORD."]]

Numbers32 = [
    ['Numbers','Num','32','001','1',"Now the children of Reuben and the children of Gad had a very great multitude of cattle: and when they saw the land of Jazer, and the land of Gilead, that, behold, the place \emph{was} a place for cattle;"],
     ['Numbers','Num','32','002','2',"The children of Gad and the children of Reuben came and spake unto Moses, and to Eleazar the priest, and unto the princes of the congregation, saying,"],
     ['Numbers','Num','32','003','3',"Ataroth, and Dibon, and Jazer, and Nimrah, and Heshbon, and Elealeh, and Shebam, and Nebo, and Beon,"],
     ['Numbers','Num','32','004','4',"\emph{Even} the country which the LORD smote before the congregation of Israel, \emph{is} a land for cattle, and thy servants have cattle:"],
      ['Numbers','Num','32','005','5',"Wherefore, said they, if we have found grace in thy sight, let this land be given unto thy servants for a possession, \emph{and} bring us not over Jordan."],
       ['Numbers','Num','32','006','6',"And Moses said unto the children of Gad and to the children of Reuben, Shall your brethren go to war, and shall ye sit here?"],
       ['Numbers','Num','32','007','7',"And wherefore discourage ye the heart of the children of Israel from going over into the land which the LORD hath given them?"],
       ['Numbers','Num','32','008','8',"Thus did your fathers, when I sent them from Kadesh-barnea to see the land."],
       ['Numbers','Num','32','009','9',"For when they went up unto the valley of Eshcol, and saw the land, they discouraged the heart of the children of Israel, that they should not go into the land which the LORD had given them."],
       ['Numbers','Num','32','010','10',"And the LORD'S anger was kindled the same time, and he sware, saying,"],
       ['Numbers','Num','32','011','11',"Surely none of the men that came up out of Egypt, from twenty years old and upward, shall see the land which I sware unto Abraham, unto Isaac, and unto Jacob; because they have not wholly followed me:"],
       ['Numbers','Num','32','012','12',"Save Caleb the son of Jephunneh the Kenezite, and Joshua the son of Nun: for they have wholly followed the LORD."],
       ['Numbers','Num','32','013','13',"And the LORD'S anger was kindled against Israel, and he made them wander in the wilderness forty years, until all the generation, that had done evil in the sight of the LORD, was consumed."],
       ['Numbers','Num','32','014','14',"And, behold, ye are risen up in your fathers' stead, an increase of sinful men, to augment yet the fierce anger of the LORD toward Israel."],
       ['Numbers','Num','32','015','15',"For if ye turn away from after him, he will yet again leave them in the wilderness; and ye shall destroy all this people."],
       ['Numbers','Num','32','016','16',"And they came near unto him, and said, We will build sheepfolds here for our cattle, and cities for our little ones:"],
       ['Numbers','Num','32','017','17',"But we ourselves will go ready armed before the children of Israel, until we have brought them unto their place: and our little ones shall dwell in the fenced cities because of the inhabitants of the land."],
       ['Numbers','Num','32','018','18',"We will not return unto our houses, until the children of Israel have inherited every man his inheritance."],
       ['Numbers','Num','32','019','19',"For we will not inherit with them on yonder side Jordan, or forward; because our inheritance is fallen to us on this side Jordan eastward."],
       ['Numbers','Num','32','020','20',"And Moses said unto them, If ye will do this thing, if ye will go armed before the LORD to war,"],
       ['Numbers','Num','32','021','21',"And will go all of you armed over Jordan before the LORD, until he hath driven out his enemies from before him,"],
       ['Numbers','Num','32','022','22',"And the land be subdued before the LORD: then afterward ye shall return, and be guiltless before the LORD, and before Israel; and this land shall be your possession before the LORD."],
       ['Numbers','Num','32','023','23',"But if ye will not do so, behold, ye have sinned against the LORD: and be sure your sin will find you out."],
       ['Numbers','Num','32','024','24',"Build you cities for your little ones, and folds for your sheep; and do that which hath proceeded out of your mouth."],
       ['Numbers','Num','32','025','25',"And the children of Gad and the children of Reuben spake unto Moses, saying, Thy servants will do as my lord commandeth."],
       ['Numbers','Num','32','026','26',"Our little ones, our wives, our flocks, and all our cattle, shall be there in the cities of Gilead:"],
       ['Numbers','Num','32','027','27',"But thy servants will pass over, every man armed for war, before the LORD to battle, as my lord saith."],
       ['Numbers','Num','32','028','28',"So concerning them Moses commanded Eleazar the priest, and Joshua the son of Nun, and the chief fathers of the tribes of the children of Israel:"],
       ['Numbers','Num','32','029','29',"And Moses said unto them, If the children of Gad and the children of Reuben will pass with you over Jordan, every man armed to battle, before the LORD, and the land shall be subdued before you; then ye shall give them the land of Gilead for a possession:"],
       ['Numbers','Num','32','030','30',"But if they will not pass over with you armed, they shall have possessions among you in the land of Canaan."],
       ['Numbers','Num','32','031','31',"And the children of Gad and the children of Reuben answered, saying, As the LORD hath said unto thy servants, so will we do."],
       ['Numbers','Num','32','032','32',"We will pass over armed before the LORD into the land of Canaan, that the possession of our inheritance on this side Jordan \emph{may} \emph{be} our's."],
        ['Numbers','Num','32','033','33',"And Moses gave unto them, \emph{even} to the children of Gad, and to the children of Reuben, and unto half the tribe of Manasseh the son of Joseph, the kingdom of Sihon king of the Amorites, and the kingdom of Og king of Bashan, the land, with the cities thereof in the coasts, \emph{even} the cities of the country round about."],
         ['Numbers','Num','32','034','34',"And the children of Gad built Dibon, and Ataroth, and Aroer,"],
         ['Numbers','Num','32','035','35',"And Atroth, Shophan, and Jaazer, and Jogbehah,"],
    ['Numbers','Num','32','036','36',"And Beth-nimrah, and Beth-haran, fenced cities: and folds for sheep."],
    ['Numbers','Num','32','037','37',"And the children of Reuben built Heshbon, and Elealeh, and Kirjathaim,"],
    ['Numbers','Num','32','038','38',"And Nebo, and Baal-meon, (their names being changed,) and Shibmah: and gave other names unto the cities which they builded."],
    ['Numbers','Num','32','039','39',"And the children of Machir the son of Manasseh went to Gilead, and took it, and dispossessed the Amorite which \emph{was} in it."],
    ['Numbers','Num','32','040','40',"And Moses gave Gilead unto Machir the son of Manasseh; and he dwelt therein."],
    ['Numbers','Num','32','041','41',"And Jair the son of Manasseh went and took the small towns thereof, and called them Havoth-jair."],
    ['Numbers','Num','32','042','42',"And Nobah went and took Kenath, and the villages thereof, and called it Nobah, after his own name."]
]

Numbers33 = [
    ['Numbers','Num','33','001','1',"These \emph{are} the journeys of the children of Israel, which went forth out of the land of Egypt with their armies under the hand of Moses and Aaron."],
['Numbers','Num','33','002','2',"And Moses wrote their goings out according to their journeys by the commandment of the LORD: and these \emph{are} their journeys according to their goings out."],
['Numbers','Num','33','003','3',"And they departed from Rameses in the first month, on the fifteenth day of the first month; on the morrow after the passover the children of Israel went out with an high hand in the sight of all the Egyptians."],
['Numbers','Num','33','004','4',"For the Egyptians buried all \emph{their} firstborn, which the LORD had smitten among them: upon their gods also the LORD executed judgments."],
['Numbers','Num','33','005','5',"And the children of Israel removed from Rameses, and pitched in Succoth."],
['Numbers','Num','33','006','6',"And they departed from Succoth, and pitched in Etham, which \emph{is} in the edge of the wilderness."],
['Numbers','Num','33','007','7',"And they removed from Etham, and turned again unto Pi-hahiroth, which \emph{is} before Baal-zephon: and they pitched before Migdol."],
['Numbers','Num','33','008','8',"And they departed from before Pi-hahiroth, and passed through the midst of the sea into the wilderness, and went three days' journey in the wilderness of Etham, and pitched in Marah."],
['Numbers','Num','33','009','9',"And they removed from Marah, and came unto Elim: and in Elim \emph{were} twelve fountains of water, and threescore and ten palm trees; and they pitched there."],
['Numbers','Num','33','010','10',"And they removed from Elim, and encamped by the Red sea."],
['Numbers','Num','33','011','11',"And they removed from the Red sea, and encamped in the wilderness of Sin."],
['Numbers','Num','33','012','12',"And they took their journey out of the wilderness of Sin, and encamped in Dophkah."],
['Numbers','Num','33','013','13',"And they departed from Dophkah, and encamped in Alush."],
['Numbers','Num','33','014','14',"And they removed from Alush, and encamped at Rephidim, where was no water for the people to drink."],
['Numbers','Num','33','015','15',"And they departed from Rephidim, and pitched in the wilderness of Sinai."],
['Numbers','Num','33','016','16',"And they removed from the desert of Sinai, and pitched at Kibroth-hattaavah."],
['Numbers','Num','33','017','17',"And they departed from Kibroth-hattaavah, and encamped at Hazeroth."],
['Numbers','Num','33','018','18',"And they departed from Hazeroth, and pitched in Rithmah."],
['Numbers','Num','33','019','19',"And they departed from Rithmah, and pitched at Rimmon-parez."],
['Numbers','Num','33','020','20',"And they departed from Rimmon-parez, and pitched in Libnah."],
['Numbers','Num','33','021','21',"And they removed from Libnah, and pitched at Rissah."],
['Numbers','Num','33','022','22',"And they journeyed from Rissah, and pitched in Kehelathah."],
['Numbers','Num','33','023','23',"And they went from Kehelathah, and pitched in mount Shapher."],
['Numbers','Num','33','024','24',"And they removed from mount Shapher, and encamped in Haradah."],
['Numbers','Num','33','025','25',"And they removed from Haradah, and pitched in Makheloth."],
['Numbers','Num','33','026','26',"And they removed from Makheloth, and encamped at Tahath."],
['Numbers','Num','33','027','27',"And they departed from Tahath, and pitched at Tarah."],
['Numbers','Num','33','028','28',"And they removed from Tarah, and pitched in Mithcah."],
['Numbers','Num','33','029','29',"And they went from Mithcah, and pitched in Hashmonah."],
['Numbers','Num','33','030','30',"And they departed from Hashmonah, and encamped at Moseroth."],
['Numbers','Num','33','031','31',"And they departed from Moseroth, and pitched in Bene-jaakan."],
['Numbers','Num','33','032','32',"And they removed from Bene-jaakan, and encamped at Hor-hagidgad."],
['Numbers','Num','33','033','33',"And they went from Hor-hagidgad, and pitched in Jotbathah."],
['Numbers','Num','33','034','34',"And they removed from Jotbathah, and encamped at Ebronah."],
['Numbers','Num','33','035','35',"And they departed from Ebronah, and encamped at Ezion-gaber."],
['Numbers','Num','33','036','36',"And they removed from Ezion-gaber, and pitched in the wilderness of Zin, which \emph{is} Kadesh."],
['Numbers','Num','33','037','37',"And they removed from Kadesh, and pitched in mount Hor, in the edge of the land of Edom."],
['Numbers','Num','33','038','38',"And Aaron the priest went up into mount Hor at the commandment of the LORD, and died there, in the fortieth year after the children of Israel were come out of the land of Egypt, in the first \emph{day} of the fifth month."],
['Numbers','Num','33','039','39',"And Aaron \emph{was} an hundred and twenty and three years old when he died in mount Hor."],
['Numbers','Num','33','040','40',"And king Arad the Canaanite, which dwelt in the south in the land of Canaan, heard of the coming of the children of Israel."],
['Numbers','Num','33','041','41',"And they departed from mount Hor, and pitched in Zalmonah."],
['Numbers','Num','33','042','42',"And they departed from Zalmonah, and pitched in Punon."],
['Numbers','Num','33','043','43',"And they departed from Punon, and pitched in Oboth."],
['Numbers','Num','33','044','44',"And they departed from Oboth, and pitched in Ije-abarim, in the border of Moab."],
['Numbers','Num','33','045','45',"And they departed from Iim, and pitched in Dibon-gad."],
['Numbers','Num','33','046','46',"And they removed from Dibon-gad, and encamped in Almon-diblathaim."],
['Numbers','Num','33','047','47',"And they removed from Almon-diblathaim, and pitched in the mountains of Abarim, before Nebo."],
['Numbers','Num','33','048','48',"And they departed from the mountains of Abarim, and pitched in the plains of Moab by Jordan \emph{near} Jericho."],
['Numbers','Num','33','049','49',"And they pitched by Jordan, from Beth-jesimoth \emph{even} unto Abel-shittim in the plains of Moab."],
['Numbers','Num','33','050','50',"And the LORD spake unto Moses in the plains of Moab by Jordan \emph{near} Jericho, saying,"],
['Numbers','Num','33','051','51',"Speak unto the children of Israel, and say unto them, When ye are passed over Jordan into the land of Canaan;"],
['Numbers','Num','33','052','52',"Then ye shall drive out all the inhabitants of the land from before you, and destroy all their pictures, and destroy all their molten images, and quite pluck down all their high places:"],
    ['Numbers','Num','33','053','53',"And ye shall dispossess \emph{the} \emph{inhabitants} of the land, and dwell therein: for I have given you the land to possess it."],
    ['Numbers','Num','33','054','54',"And ye shall divide the land by lot for an inheritance among your families: \emph{and} to the more ye shall give the more inheritance, and to the fewer ye shall give the less inheritance: every man's \emph{inheritance} shall be in the place where his lot falleth; according to the tribes of your fathers ye shall inherit."],
    ['Numbers','Num','33','055','55',"But if ye will not drive out the inhabitants of the land from before you; then it shall come to pass, that those which ye let remain of them \emph{shall} \emph{be} pricks in your eyes, and thorns in your sides, and shall vex you in the land wherein ye dwell."],
    ['Numbers','Num','33','056','56',"Moreover it shall come to pass, \emph{that} I shall do unto you, as I thought to do unto them."]
]

Numbers34 = [
    ['Numbers','Num','34','001','1',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','34','002','2',"Command the children of Israel, and say unto them, When ye come into the land of Canaan; (this \emph{is} the land that shall fall unto you for an inheritance, \emph{even} the land of Canaan with the coasts thereof:)"],
['Numbers','Num','34','003','3',"Then your south quarter shall be from the wilderness of Zin along by the coast of Edom, and your south border shall be the outmost coast of the salt sea eastward:"],
['Numbers','Num','34','004','4',"And your border shall turn from the south to the ascent of Akrabbim, and pass on to Zin: and the going forth thereof shall be from the south to Kadesh-barnea, and shall go on to Hazar-addar, and pass on to Azmon:"],
['Numbers','Num','34','005','5',"And the border shall fetch a compass from Azmon unto the river of Egypt, and the goings out of it shall be at the sea."],
['Numbers','Num','34','006','6',"And \emph{as} \emph{for} the western border, ye shall even have the great sea for a border: this shall be your west border."],
['Numbers','Num','34','007','7',"And this shall be your north border: from the great sea ye shall point out for you mount Hor:"],
['Numbers','Num','34','008','8',"From mount Hor ye shall point out \emph{your} \emph{border} unto the entrance of Hamath; and the goings forth of the border shall be to Zedad:"],
['Numbers','Num','34','009','9',"And the border shall go on to Ziphron, and the goings out of it shall be at Hazar-enan: this shall be your north border."],
['Numbers','Num','34','010','10',"And ye shall point out your east border from Hazar-enan to Shepham:"],
['Numbers','Num','34','011','11',"And the coast shall go down from Shepham to Riblah, on the east side of Ain; and the border shall descend, and shall reach unto the side of the sea of Chinnereth eastward:"],
['Numbers','Num','34','012','12',"And the border shall go down to Jordan, and the goings out of it shall be at the salt sea: this shall be your land with the coasts thereof round about."],
['Numbers','Num','34','013','13',"And Moses commanded the children of Israel, saying, This \emph{is} the land which ye shall inherit by lot, which the LORD commanded to give unto the nine tribes, and to the half tribe:"],
['Numbers','Num','34','014','14',"For the tribe of the children of Reuben according to the house of their fathers, and the tribe of the children of Gad according to the house of their fathers, have received \emph{their} \emph{inheritance}; and half the tribe of Manasseh have received their inheritance:"],
['Numbers','Num','34','015','15',"The two tribes and the half tribe have received their inheritance on this side Jordan \emph{near} Jericho eastward, toward the sunrising."],
['Numbers','Num','34','016','16',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','34','017','17',"These \emph{are} the names of the men which shall divide the land unto you: Eleazar the priest, and Joshua the son of Nun."],
['Numbers','Num','34','018','18',"And ye shall take one prince of every tribe, to divide the land by inheritance."],
['Numbers','Num','34','019','19',"And the names of the men \emph{are} these: Of the tribe of Judah, Caleb the son of Jephunneh."],
['Numbers','Num','34','020','20',"And of the tribe of the children of Simeon, Shemuel the son of Ammihud."],
['Numbers','Num','34','021','21',"Of the tribe of Benjamin, Elidad the son of Chislon."],
['Numbers','Num','34','022','22',"And the prince of the tribe of the children of Dan, Bukki the son of Jogli."],
['Numbers','Num','34','023','23',"The prince of the children of Joseph, for the tribe of the children of Manasseh, Hanniel the son of Ephod."],
['Numbers','Num','34','024','24',"And the prince of the tribe of the children of Ephraim, Kemuel the son of Shiphtan."],
['Numbers','Num','34','025','25',"And the prince of the tribe of the children of Zebulun, Elizaphan the son of Parnach."],
['Numbers','Num','34','026','26',"And the prince of the tribe of the children of Issachar, Paltiel the son of Azzan."],
['Numbers','Num','34','027','27',"And the prince of the tribe of the children of Asher, Ahihud the son of Shelomi."],
['Numbers','Num','34','028','28',"And the prince of the tribe of the children of Naphtali, Pedahel the son of Ammihud."],
['Numbers','Num','34','029','29',"These \emph{are} \emph{they} whom the LORD commanded to divide the inheritance unto the children of Israel in the land of Canaan."]]

Numbers35 = [
    ['Numbers','Num','35','001','1',"And the LORD spake unto Moses in the plains of Moab by Jordan \emph{near} Jericho, saying,"],
['Numbers','Num','35','002','2',"Command the children of Israel, that they give unto the Levites of the inheritance of their possession cities to dwell in; and ye shall give \emph{also} unto the Levites suburbs for the cities round about them."],
['Numbers','Num','35','003','3',"And the cities shall they have to dwell in; and the suburbs of them shall be for their cattle, and for their goods, and for all their beasts."],
['Numbers','Num','35','004','4',"And the suburbs of the cities, which ye shall give unto the Levites, \emph{shall} \emph{reach} from the wall of the city and outward a thousand cubits round about."],
['Numbers','Num','35','005','5',"And ye shall measure from without the city on the east side two thousand cubits, and on the south side two thousand cubits, and on the west side two thousand cubits, and on the north side two thousand cubits; and the city \emph{shall} \emph{be} in the midst: this shall be to them the suburbs of the cities."],
['Numbers','Num','35','006','6',"And among the cities which ye shall give unto the Levites \emph{there} \emph{shall} \emph{be} six cities for refuge, which ye shall appoint for the manslayer, that he may flee thither: and to them ye shall add forty and two cities."],
['Numbers','Num','35','007','7',"\emph{So} all the cities which ye shall give to the Levites \emph{shall} \emph{be} forty and eight cities: them \emph{shall} \emph{ye} \emph{give} with their suburbs."],
['Numbers','Num','35','008','8',"And the cities which ye shall give \emph{shall} \emph{be} of the possession of the children of Israel: from \emph{them} \emph{that} \emph{have} many ye shall give many; but from \emph{them} \emph{that} \emph{have} few ye shall give few: every one shall give of his cities unto the Levites according to his inheritance which he inheriteth."],
['Numbers','Num','35','009','9',"And the LORD spake unto Moses, saying,"],
['Numbers','Num','35','010','10',"Speak unto the children of Israel, and say unto them, When ye be come over Jordan into the land of Canaan;"],
['Numbers','Num','35','011','11',"Then ye shall appoint you cities to be cities of refuge for you; that the slayer may flee thither, which killeth any person at unawares."],
['Numbers','Num','35','012','12',"And they shall be unto you cities for refuge from the avenger; that the manslayer die not, until he stand before the congregation in judgment."],
['Numbers','Num','35','013','13',"And of these cities which ye shall give six cities shall ye have for refuge."],
['Numbers','Num','35','014','14',"Ye shall give three cities on this side Jordan, and three cities shall ye give in the land of Canaan, \emph{which} shall be cities of refuge."],
['Numbers','Num','35','015','15',"These six cities shall be a refuge, \emph{both} for the children of Israel, and for the stranger, and for the sojourner among them: that every one that killeth any person unawares may flee thither."],
['Numbers','Num','35','016','16',"And if he smite him with an instrument of iron, so that he die, he \emph{is} a murderer: the murderer shall surely be put to death."],
['Numbers','Num','35','017','17',"And if he smite him with throwing a stone, wherewith he may die, and he die, he \emph{is} a murderer: the murderer shall surely be put to death."],
['Numbers','Num','35','018','18',"Or \emph{if} he smite him with an hand weapon of wood, wherewith he may die, and he die, he \emph{is} a murderer: the murderer shall surely be put to death."],
['Numbers','Num','35','019','19',"The revenger of blood himself shall slay the murderer: when he meeteth him, he shall slay him."],
['Numbers','Num','35','020','20',"But if he thrust him of hatred, or hurl at him by laying of wait, that he die;"],
['Numbers','Num','35','021','21',"Or in enmity smite him with his hand, that he die: he that smote \emph{him} shall surely be put to death; \emph{for} he \emph{is} a murderer: the revenger of blood shall slay the murderer, when he meeteth him."],
['Numbers','Num','35','022','22',"But if he thrust him suddenly without enmity, or have cast upon him any thing without laying of wait,"],
['Numbers','Num','35','023','23',"Or with any stone, wherewith a man may die, seeing \emph{him} not, and cast \emph{it} upon him, that he die, and \emph{was} not his enemy, neither sought his harm:"],
['Numbers','Num','35','024','24',"Then the congregation shall judge between the slayer and the revenger of blood according to these judgments:"],
['Numbers','Num','35','025','25',"And the congregation shall deliver the slayer out of the hand of the revenger of blood, and the congregation shall restore him to the city of his refuge, whither he was fled: and he shall abide in it unto the death of the high priest, which was anointed with the holy oil."],
['Numbers','Num','35','026','26',"But if the slayer shall at any time come without the border of the city of his refuge, whither he was fled;"],
['Numbers','Num','35','027','27',"And the revenger of blood find him without the borders of the city of his refuge, and the revenger of blood kill the slayer; he shall not be guilty of blood:"],
['Numbers','Num','35','028','28',"Because he should have remained in the city of his refuge until the death of the high priest: but after the death of the high priest the slayer shall return into the land of his possession."],
['Numbers','Num','35','029','29',"So these \emph{things} shall be for a statute of judgment unto you throughout your generations in all your dwellings."],
['Numbers','Num','35','030','30',"Whoso killeth any person, the murderer shall be put to death by the mouth of witnesses: but one witness shall not testify against any person \emph{to} \emph{cause} \emph{him} to die."],
['Numbers','Num','35','031','31',"Moreover ye shall take no satisfaction for the life of a murderer, which \emph{is} guilty of death: but he shall be surely put to death."],
['Numbers','Num','35','032','32',"And ye shall take no satisfaction for him that is fled to the city of his refuge, that he should come again to dwell in the land, until the death of the priest."],
['Numbers','Num','35','033','33',"So ye shall not pollute the land wherein ye \emph{are}: for blood it defileth the land: and the land cannot be cleansed of the blood that is shed therein, but by the blood of him that shed it."],
['Numbers','Num','35','034','34',"Defile not therefore the land which ye shall inhabit, wherein I dwell: for I the LORD dwell among the children of Israel."]
]

Numbers36 = [
    ['Numbers','Num','36','001','1',"And the chief fathers of the families of the children of Gilead, the son of Machir, the son of Manasseh, of the families of the sons of Joseph, came near, and spake before Moses, and before the princes, the chief fathers of the children of Israel:"],
    ['Numbers','Num','36','002','2',"And they said, The LORD commanded my lord to give the land for an inheritance by lot to the children of Israel: and my lord was commanded by the LORD to give the inheritance of Zelophehad our brother unto his daughters."],
    ['Numbers','Num','36','003','3',"And if they be married to any of the sons of the \emph{other} tribes of the children of Israel, then shall their inheritance be taken from the inheritance of our fathers, and shall be put to the inheritance of the tribe whereunto they are received: so shall it be taken from the lot of our inheritance."],
    ['Numbers','Num','36','004','4',"And when the jubile of the children of Israel shall be, then shall their inheritance be put unto the inheritance of the tribe whereunto they are received: so shall their inheritance be taken away from the inheritance of the tribe of our fathers."],
    ['Numbers','Num','36','005','5',"And Moses commanded the children of Israel according to the word of the LORD, saying, The tribe of the sons of Joseph hath said well."],
    ['Numbers','Num','36','006','6',"This \emph{is} the thing which the LORD doth command concerning the daughters of Zelophehad, saying, Let them marry to whom they think best; only to the family of the tribe of their father shall they marry."],
    ['Numbers','Num','36','007','7',"So shall not the inheritance of the children of Israel remove from tribe to tribe: for every one of the children of Israel shall keep himself to the inheritance of the tribe of his fathers."],
    ['Numbers','Num','36','008','8',"And every daughter, that possesseth an inheritance in any tribe of the children of Israel, shall be wife unto one of the family of the tribe of her father, that the children of Israel may enjoy every man the inheritance of his fathers."],
    ['Numbers','Num','36','009','9',"Neither shall the inheritance remove from \emph{one} tribe to another tribe; but every one of the tribes of the children of Israel shall keep himself to his own inheritance."],
    ['Numbers','Num','36','010','10',"Even as the LORD commanded Moses, so did the daughters of Zelophehad:"],
    ['Numbers','Num','36','011','11',"For Mahlah, Tirzah, and Hoglah, and Milcah, and Noah, the daughters of Zelophehad, were married unto their father's brothers' sons:"],
    ['Numbers','Num','36','012','12',"\emph{And} they were married into the families of the sons of Manasseh the son of Joseph, and their inheritance remained in the tribe of the family of their father."],
    ['Numbers','Num','36','013','13',"These \emph{are} the commandments and the judgments, which the LORD commanded by the hand of Moses unto the children of Israel in the plains of Moab by Jordan \emph{near} Jericho."]
]


In [5]:
ProcessVerses('Numbers36',Numbers36)



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And the chief fathers of the families of the children of Gilead, the son of Machir, the son of Manasseh, of the families of the sons of Joseph, came near, and spake before Moses, and before the princes, the chief fathers of the children of Israel:}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And they said, The LORD commanded my lord to give the land for an inheritance by lot to the children of Israel: and my lord was commanded by the LORD to give the inheritance of Zelophehad our brother unto his daughters.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And if they be married to any of the sons of the \emph{other} tribes of the children of Israel, then shall their inheritance be taken from the inheritance of our fathers, and shall be put to the inheritance of the tribe whereunto they are recei